# Imports and installs

Run this two cells before running any other cell in the notebook. The first cell installs the necessary packages and the second cell imports them. Run only the second cell if you have already installed the packages. I suggest creating a virtual enviroment before runing anything, so you don't mess up with your system packages. 

In [ ]:
%pip install qutip --upgrade
%pip install matplotlib --upgrade
# for progress bar
%pip install tqdm --upgrade
# for latex from function to use in plots
%pip install latexify-py --upgrade
# parallel process
%pip install loky --upgrade
# for running julia scripts inside python
#%pip install julia --upgrade
# see https://qutip-jax.readthedocs.io/en/latest/installation.html for installing qutip-jax 
# I recommend creating its own environment

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import qutip as qt
from tqdm.auto import tqdm
from functools import partial
from IPython.display import display
from fractions import Fraction
import joblib
#Before running all make sure to set the constants you want for ODMR and Ramsey in the begining of the Sequences sections
run_odmr=False
run_ramsey=False
run_pls=False
run_pls_bomr=False
run_bomr=False 
run_bang=False
run_b_v_pl=False
load_res=False
run_benchmark = False  # set True to re-run timings, False to load saved
# Strain term 
E_gs=0.0
E_es=70.0
save_res=False
save_odmr=False

## Useful functions - for now, I just use the first section, ignore the other section

### Normalizations

In [ ]:
def norma(arr):
    """
    Normalize an array by scaling its values between 0 and 1.

    Parameters:
    arr (numpy.ndarray): The input array.

    Returns:
    numpy.ndarray: The normalized array.
    """
    return (arr-np.min(arr))/(np.max(arr)-np.min(arr))

def stand(arr):
    """
    Standardizes an array by subtracting the mean and dividing by the standard deviation.

    Parameters:
    arr (array-like): The input array to be standardized.

    Returns:
    array-like: The standardized array.

    """
    return (arr-np.mean(arr))/np.std(arr)
def max_reg(arr):
    """
    Normalize the input array by dividing it by its maximum value.

    Parameters:
    arr (numpy.ndarray): The input array to be normalized.

    Returns:
    numpy.ndarray: The normalized array.

    """
    return arr/np.max(arr)

### Sigmoids for time slices

In [ ]:
def sigmoid_rect(t, t0, t1, amp, step):
    """
    Compute the sigmoidal rectangular function.

    Parameters:
    t (float): The time to be evalutated.
    t0 (float): The lower time threshold value.
    t1 (float): The upper time threshold value.
    amp (float): The amplitude of the function.
    step (float): The step parameter controlling the steepness of the sigmoid.

    Returns:
    float: The computed value of the sigmoidal rectangular function.
    """
    return amp / (
        1
        + np.exp(-step * (t - t0))
        + np.exp(step * (t - t1))
        + np.exp(-step * (t1 - t0))
    )


# Examples of usage
times = np.linspace(0, 10, 200)
plt.figure(figsize=(8, 12))
plt.subplot(311)
for t0 in range(1, 5):
    t1 = t0 + 5
    plt.plot(times, sigmoid_rect(times, t0, t1, 2, 20), label=f"t0={t0},t1={t1}")
    plt.vlines(t0, -0.1, 2.1, colors="green", ls="--")
    plt.vlines(t1, -0.1, 2.1, colors="red", ls="--")
plt.xticks(np.arange(0, 11, 1))

plt.legend(bbox_to_anchor=(1.05, 0.8), loc="upper left")
plt.subplot(312)
times = np.linspace(1.5, 5.5, 200)
for a in range(1, 5):
    plt.plot(times, sigmoid_rect(times, 2, 5, a, 20), label=f"amp={a}")
plt.vlines(2, -0.1, 4.1, colors="green", ls="--", label="t0")
plt.vlines(5, -0.1, 4.1, colors="red", ls="--", label="t1")
plt.legend(bbox_to_anchor=(1.05, 0.8), loc="upper left")
plt.subplot(313)
times = np.linspace(-1, 8, 200)
for b in range(1, 26, 5):
    if b != 1:
        b1 = b - 1
    else:
        b1 = b
    plt.plot(times, sigmoid_rect(times, 2, 5, 2, b1), label=f"step={b1}")
plt.vlines(2, -0.1, 2.1, colors="green", ls="--", label="t0")
plt.vlines(5, -0.1, 2.1, colors="red", ls="--", label="t1")
plt.xticks(np.arange(-1, 9, 1))
plt.legend(bbox_to_anchor=(1.05, 0.8), loc="upper left")
plt.show()

# Model of the moment

This is a implementation, in Qutip, of a model from the paper in this [link](https://arxiv.org/abs/2407.05569). All constants are given in the paper linked above, maybe some are in other papers in the list below.
The models I already implemented correctly are in [this notebook](Codes\SimCompareDiffFrameworks\NV_sims_qutip.ipynb), but their papers are in the list below.

---
Already tested models:
- [Magaletti et al.](https://arxiv.org/abs/2309.06203)

## Hamiltonian and other definitions

In [ ]:
def B0(b_0, phi, theta):
    """
    Calculate the magnetic field vector in Cartesian coordinates.
    Parameters:
    - b_0 (float): The magnitude of the magnetic field.
    - theta (float): The azimuthal angle in radians.
    - phi (float): The polar angle in radians.
    Returns:
    - np.array: The magnetic field vector in Cartesian coordinates np.array([Bx, By, Bz]).
    """
    return np.array([b_0 * np.sin(theta) * np.cos(phi), b_0 * np.sin(theta) * np.sin(phi), b_0 * np.cos(theta)]) #type:ignore

def construct_spin_matrices(basis):
    """
    Constructs the spin matrices for a given basis.
    Parameters:
    - basis (list): A list of basis elements.
    Returns:
    - s_x (Qobj): The x-component of the spin matrix.
    - s_y (Qobj): The y-component of the spin matrix.
    - s_z (Qobj): The z-component of the spin matrix.
    Raises:
    - ValueError: If the number of basis elements is not 3.
    """
    if len(basis) == 3:
        # 3-level system (spin-1)
        s_x = (basis[0] * basis[1].dag() + basis[1] * basis[0].dag() +
               basis[1] * basis[2].dag() + basis[2] * basis[1].dag()) / np.sqrt(2)
        
        s_y = -1j * (basis[0] * basis[1].dag() - basis[1] * basis[0].dag() +
                     basis[1] * basis[2].dag() - basis[2] * basis[1].dag()) / np.sqrt(2)
        
        s_z = basis[0] * basis[0].dag() - basis[2] * basis[2].dag()
        return s_x, s_y, s_z

    else:
        raise ValueError("This function supports only 3 basis elements.")
    
# Dimention of the Hilbert space
dim = 7
# Optical transition rates index
k_ind=2
# dephasing times(microseconds)
t1_gs=1e3
t2_gs=1.5
t1_es=1e3
t2_es=6*1e-3
    
mu_e = 2.8 # (Mhz/G)
B = B0(100.0,0.0,0.0) # (G)
# Constants (MHz)
D_gs = 2870.0
D_es = 1420.0
W_p = 1.9
gamma_gs = 1/t1_gs,1/t2_gs
gamma_es = 1/t1_es,1/t2_es

# Microwave frequency
omega = D_gs-mu_e*B[2]
# Microwave phase
phi = 0.0
# I had to modify this two value to repruduce the results of the paper
# Microwave magnectic field*mu_e 
Om_r = 15.7

# States (I broke them in a few lines to be more readable)
excited = qt.basis(dim, 4), qt.basis(dim, 3),qt.basis(dim, 5) # |+1>_es, |0>_es, |-1>_es
isc = qt.basis(dim, 6)
ground = qt.basis(dim, 1), qt.basis(dim, 0), qt.basis(dim, 2) # |+1>_gs, |0>_gs, |-1>_gs
IdNV=qt.qeye(dim)

n1, n2, n3 = (
    ground[1] * ground[1].dag(),#type: ignore
    ground[2] * ground[2].dag(),#type: ignore
    ground[0] * ground[0].dag(),#type: ignore
)
n4, n5, n6 = (
    excited[1] * excited[1].dag(),#type: ignore
    excited[2] * excited[2].dag(),#type: ignore
    excited[0] * excited[0].dag(),#type: ignore
)
n7, nc = (
    isc * isc.dag(),#type: ignore
    ground[2] * ground[1].dag(),#type: ignore
)

sx_gs, sy_gs, sz_gs = construct_spin_matrices(ground)
sx_es, sy_es, sz_es = construct_spin_matrices(excited)
sm_gs,sp_gs= sx_gs - 1j * sy_gs, sx_gs + 1j * sy_gs
sm_es,sp_es= sx_es - 1j * sy_es, sx_es + 1j * sy_es

S_gs=np.array([sx_gs, sy_gs, sz_gs]) #type:ignore
S_es=np.array([sx_es, sy_es, sz_es]) #type:ignore


def H_no(b, om_r):
    """
    Calculates the Hamiltonian for a given magnetic field and Rabi frequency.
    
    Parameters:
    b (np.array): Magnetic field vector B0(B_amp,phi_B,theta_B).
    om_r (float): Rabi frequency.
    
    Returns:
    list: A list containing the Hamiltonian Qobj terms.
    """
    H_0 = [D_gs*sz_gs**2+E_gs*(sx_gs**2-sy_gs**2)+mu_e*np.dot(b,S_gs)+
           D_es*sz_es**2+E_es*(sx_es**2-sy_es**2)+mu_e*np.dot(b,S_es)]
    H_int = [[np.sqrt(2)*om_r*(sx_gs), "cos(w*t)*cos(p)"],
           [np.sqrt(2)*om_r*(sy_gs), "cos(w*t)*sin(p)"],
           [np.sqrt(2)*om_r*(sx_es), "cos(w*t)*cos(p)"],
           [np.sqrt(2)*om_r*(sy_es), "cos(w*t)*sin(p)"]]
    return [*H_0,*H_int]

def L_no(w_p,k_index=k_ind):
    """
    Returns the Lindblad operators of the system, including optical transitions based on the given k_index.

    Parameters:
    - w_p (float): Laser pump rate.
    - k_index (int, optional): Index for the optical transition rates. Defaults to k_ind.

    Returns:
    - c_ops (list): List of Lindblad operators Qobj.
    """
    #optical transitions
    K_s=[[66.0,0.0,57.0,1.0,0.7],
         [77.0,0.0,30.0,3.3,0.0],
         [62.7,12.97,80.0,3.45,1.08],
         [63.2,10.8,60.7,0.8,0.4],
         [67.4,9.9,96.6,4.83,1.055]]
    k41 = K_s[k_index][0]
    k52 = K_s[k_index][0]
    k63 = K_s[k_index][0]
    k57 = K_s[k_index][2]
    k67 = K_s[k_index][2]
    k47 = K_s[k_index][1]
    k71 = K_s[k_index][3]
    k72 = K_s[k_index][4]
    k73 = K_s[k_index][4]
    
    c_ops = []

    c_ops.append(np.sqrt(w_p) * (excited[1] * ground[1].dag()))  # n1 to n4 #type: ignore
    c_ops.append(np.sqrt(w_p) * (excited[2] * ground[2].dag()))  # n2 to n5 #type: ignore
    c_ops.append(np.sqrt(w_p) * (excited[0] * ground[0].dag())) # n3 to n6 #type: ignore

    c_ops.append(np.sqrt(k41) * (ground[1] * excited[1].dag()))  # n4 to n1 #type: ignore
    c_ops.append(np.sqrt(k71) * (ground[1] * isc.dag()))  # n7 to n1 #type: ignore

    c_ops.append(np.sqrt(k52) * (ground[2] * excited[2].dag()))  # n5 to n2 #type: ignore
    c_ops.append(np.sqrt(k72) * (ground[2] * isc.dag()))  # n7 to n2 #type: ignore

    c_ops.append(np.sqrt(k63) * (ground[0] * excited[0].dag()))  # n6 to n3 #type: ignore
    c_ops.append(np.sqrt(k73) * (ground[0] * isc.dag()))  # n7 to n3 #type: ignore

    c_ops.append(np.sqrt(k47) * (isc * excited[1].dag()))  # n4 to n7 #type: ignore
    c_ops.append(np.sqrt(k57) * (isc * excited[2].dag()))  # n5 to n7 #type: ignore
    c_ops.append(np.sqrt(k67) * (isc * excited[0].dag()))  # n6 to n7 #type: ignore
    # Add collapse operators for decoherence
    c_ops.append(np.sqrt(gamma_gs[1]) * sz_gs)
    c_ops.append(np.sqrt(gamma_gs[0]/2) * (sm_gs))
    c_ops.append(np.sqrt(gamma_gs[0]/2) * (sp_gs))
    c_ops.append(np.sqrt(gamma_es[1]) * sz_es)
    c_ops.append(np.sqrt(gamma_es[0]/2) * (sm_es))
    c_ops.append(np.sqrt(gamma_es[0]/2) * (sp_es))
    return c_ops


def dynamics_no(
    dt,
    init_state,
    b=None,
    om=None,
    p=None,
    om_r=None,
    w_p=None,
    k_index=k_ind,
    #exp_ops=None,
    ti=0.0,    
    mode="Free",
    progress_bar="ON",
    i=0,
):
    """
    Perform dynamics simulation based on the given parameters, including optical transition rates index.
    Where, when using k_index=2 -> K_s[k_index]=[62.7,12.97,80.0,3.45,1.08], and the full K_s list is:
        K_s=[[66.0,0.0,57.0,1.0,0.7],
             [77.0,0.0,30.0,3.3,0.0],
             [62.7,12.97,80.0,3.45,1.08],
             [63.2,10.8,60.7,0.8,0.4],
             [67.4,9.9,96.6,4.83,1.055]]
    Parameters:
    - dt (float): Time step for the calculations.
    - init_state: Initial state for the simulation.
    - b (np.array, optional): Magnetic field vector. Defaults to B0(B_amp,phi_B,theta_B).
    - om (float, optional): Angular frequency of the system. Defaults to omega.
    - om_r (float, optional): Angular frequency for MW-ON evolution. Defaults to Om_r.
    - w_p (float, optional): Frequency for laser-ON evolution. Defaults to W_p.
    - k_index=k_index (int, optional): Index for the optical transition rates. Defaults to k_ind.
    - exp_ops (list, optional): List of expectation operators. Defaults to 
        [n1, n2, n3, n4, n5, n6, n7, nc, 
        sx_gs, sy_gs, sz_gs, 
        sx_es, sy_es, sz_es].
    - ti (float, optional): Initial time for the simulation. Defaults to 0.0.
    - mode (str, optional): Mode of the simulation. Can be "Free", "MW", "Laser", or "Laser-MW". Defaults to "Free".
    - progress_bar (str, optional): Progress bar option. Can be "ON" or "OFF". Defaults to "ON".
    - i (int, optional): Iteration number. Defaults to 0.

    Returns:
    - tf (float): Final time of the simulation.
    - result: Result of the simulation.
    """
    # Default values
    if b is None: b = B
    if om is None: om = omega
    if p is None: p = phi
    if om_r is None: om_r = Om_r
    if w_p is None: w_p = W_p
    #if exp_ops is None: exp_ops = [
    #    n1, n2, n3, n4, n5, n6, n7, nc, 
    #    sx_gs, sy_gs, sz_gs, 
    #    sx_es, sy_es, sz_es
    #]

    # Arguments for the Hamiltonian
    args = {"w": om, "p": p}
    
    # Define the time resolution   
    t_bins = 1000 if dt <= 5 else 5000
    
    tf = ti + dt

    # Define collapse operators and Hamiltonian based on mode
    match mode:
        case "Free":
            c_ops = L_no(0.0, k_index=k_index)
            H = H_no(b, 0.0)
        case "MW":
            c_ops = L_no(0.0, k_index=k_index)
            H = H_no(b, om_r)
        case "Laser":
            c_ops = L_no(w_p, k_index=k_index)
            H = H_no(b, 0.0)
        case "Laser-MW":
            c_ops = L_no(w_p, k_index=k_index)
            H = H_no(b, om_r)
        case _:
            raise ValueError('mode must be one of "Free", "MW", "Laser", or "Laser-MW"')

    # Call the master equation solver
    match progress_bar:
        case "OFF":
            result = qt.mesolve(
                H,
                init_state,
                np.linspace(ti, tf, t_bins + 1),
                c_ops,
                #e_ops=exp_ops,
                args=args,
                options={"store_states": True},
            )
        case "ON":
            print(f"{mode} {int(i + 1)} \n ti    |    tf \n {ti:.2f} | {tf:.2f}")
            result = qt.mesolve(
                H,
                init_state,
                np.linspace(ti, tf, t_bins + 1),
                c_ops,
                #e_ops=exp_ops,
                args=args,
                options={"store_states": True, "progress_bar": "tqdm"},
            )
        case _:
            raise ValueError('progress_bar must be "ON" or "OFF"')
    
    return tf, result

In [ ]:
default_area=mpl.rcParams['figure.figsize'][0]*mpl.rcParams['figure.figsize'][1]
new_figsize=(14,8)
new_area=new_figsize[0]*new_figsize[1]
scale=new_area/default_area
mpl.rcParams['lines.linewidth']=0.75*scale
mpl.rc('xtick', labelsize=18)           # x-tick numbers only
mpl.rc('ytick', labelsize=18)           # y-tick numbers only
mpl.rcParams['figure.dpi']=300
for thetab in [0.0,np.pi/2,2*np.pi/3,np.pi]:
    for phib in [0.0,np.pi/2,2*np.pi/3,np.pi]:
        B1=B0(100,phib,thetab)
        fig,ax=plt.subplots(figsize=(14,8))
        ax.set_title(f"$\\theta_B={Fraction(thetab/np.pi).limit_denominator()}\\pi$, $\\phi_B={Fraction(phib/np.pi).limit_denominator()}\\pi$",fontsize=20)
        bas_x=[f"$\\langle {i+1}|$" for i in range(7)]
        bas_y=[f"$|{i+1}\\rangle$" for i in range(7)]
        qt.hinton(H_no(B1,Om_r)[0],x_basis=bas_x,y_basis=bas_y,fig=fig,ax=ax)
        plt.show()
        if np.isclose(thetab,0) or np.isclose(thetab,np.pi):
            break


### Jax

In [ ]:
#import jax.numpy as jnp
#import jax
#import qutip_jax
#from functools import partial
#
##qutip_jax.set_as_default()
#def B0_jax(b_0, phi, theta):
#    """
#    Calculate the magnetic field vector in Cartesian coordinates.
#    Parameters:
#    - b_0 (float): The magnitude of the magnetic field.
#    - theta (float): The azimuthal angle in radians.
#    - phi (float): The polar angle in radians.
#    Returns:
#    - np.array: The magnetic field vector in Cartesian coordinates np.array([Bx, By, Bz]).
#    """
#    return jnp.array([b_0 * jnp.sin(theta) * jnp.cos(phi), b_0 * jnp.sin(theta) * jnp.sin(phi), b_0 * jnp.cos(theta)])
#with qt.CoreOptions(default_dtype="jax"):
#    excited_j = qt.basis(dim, 4, dtype="jax"), qt.basis(dim, 3, dtype="jax"), qt.basis(dim, 5, dtype="jax")
#    isc_j = qt.basis(dim, 6, dtype="jax")
#    ground_j = qt.basis(dim, 1, dtype="jax"), qt.basis(dim, 0, dtype="jax"), qt.basis(dim, 2, dtype="jax")
#    IdNV_j = qt.qeye(dim, dtype="jax")
#sx_gs_j, sy_gs_j, sz_gs_j = construct_spin_matrices(ground)
#sx_es_j, sy_es_j, sz_es_j = construct_spin_matrices(excited)
#B_j=B0_jax(500.0,0.0,0.0)
#S_gs_j = jnp.array([sx_gs_j.full(), sy_gs_j.full(), sz_gs_j.full()])
#S_es_j = jnp.array([sx_es_j.full(), sy_es_j.full(), sz_es_j.full()])
#n1_j, n2_j, n3_j = (
#    ground_j[1] @ ground_j[1].dag(),
#    ground_j[0] @ ground_j[0].dag(),
#    ground_j[2] @ ground_j[2].dag(),
#)
#n4_j, n5_j, n6_j = (
#    excited_j[1] @ excited_j[1].dag(),
#    excited_j[0] @ excited_j[0].dag(),
#    excited_j[2] @ excited_j[2].dag(),
#)
#n7_j, nc_j = (
#    isc_j @ isc_j.dag(),
#    ground_j[0] @ ground_j[1].dag(),
#)
#
#@partial(jax.jit, static_argnames=("w","p", ))
#def H_int_coeff_cos(t, w, p):
#    return jnp.cos(w * t)*jnp.cos(p)
#@partial(jax.jit, static_argnames=("w","p", ))
#def H_int_coeff_sin(t, w, p):
#    return jnp.cos(w * t)*jnp.sin(p)
## Define the Hamiltonian and collapse operators using JAX dtype
#def H_jax_no(b, om_r):
#    H_0 = [
#        D_gs * sz_gs_j ** 2 + mu_e * qt.Qobj(jnp.dot(S_gs_j.T,b)),
#        D_es * sz_es_j ** 2 + mu_e * qt.Qobj(jnp.dot(S_es_j.T,b))
#    ]
#    H_int = [
#        [jnp.sqrt(2) * om_r * (sx_gs_j), H_int_coeff_cos],
#        [jnp.sqrt(2) * om_r * (sy_gs_j), H_int_coeff_sin],
#        [jnp.sqrt(2) * om_r * (sx_es_j), H_int_coeff_cos],
#        [jnp.sqrt(2) * om_r * (sy_es_j), H_int_coeff_sin]
#    ]
#    
#    return [*H_0, *H_int]
## Define the Lindblad operators using JAX dtype
#def L_jax_no(w_p, k_index=k_ind):
#    #optical transitions
#    K_s=[[66.0,0.0,57.0,1.0,0.7],
#         [77.0,0.0,30.0,3.3,0.0],
#         [62.7,12.97,80.0,3.45,1.08],
#         [63.2,10.8,60.7,0.8,0.4],
#         [67.4,9.9,96.6,4.83,1.055]]
#    k41 = K_s[k_index][0]
#    k52 = K_s[k_index][0]
#    k63 = K_s[k_index][0]
#    k57 = K_s[k_index][2]
#    k67 = K_s[k_index][2]
#    k47 = K_s[k_index][1]
#    k71 = K_s[k_index][3]
#    k72 = K_s[k_index][4]
#    k73 = K_s[k_index][4]
#    c_ops = []
#    c_ops.append(jnp.sqrt(w_p) * (excited_j[1] @ ground_j[1].dag()))  # n1 to n4
#    c_ops.append(jnp.sqrt(w_p) * (excited_j[0] @ ground_j[0].dag()))  # n2 to n5
#    c_ops.append(jnp.sqrt(w_p) * (excited_j[2] @ ground_j[2].dag())) # n3 to n6
#
#    c_ops.append(jnp.sqrt(k41) * (ground_j[1] @ excited_j[1].dag()))  # n4 to n1
#    c_ops.append(jnp.sqrt(k71) * (ground_j[1] @ isc_j.dag()))  # n7 to n1
#
#    c_ops.append(jnp.sqrt(k52) * (ground_j[0] @ excited_j[0].dag()))  # n5 to n2
#    c_ops.append(jnp.sqrt(k72) * (ground_j[0] @ isc_j.dag()))  # n7 to n2
#
#    c_ops.append(jnp.sqrt(k63) * (ground_j[2] @ excited_j[2].dag()))  # n6 to n3
#    c_ops.append(jnp.sqrt(k73) * (ground_j[2] @ isc_j.dag()))  # n7 to n3
#
#    c_ops.append(jnp.sqrt(k47) * (isc_j @ excited_j[1].dag()))  # n4 to n7
#    c_ops.append(jnp.sqrt(k57) * (isc_j @ excited_j[0].dag()))  # n5 to n7
#    c_ops.append(jnp.sqrt(k67) * (isc_j @ excited_j[2].dag()))  # n6 to n7
#    
#    return c_ops
## Modify dynamics_no to use JAX options
#def dynamics_jax_no(
#    dt,
#    init_state,
#    b=None,
#    om=None,
#    p=None,
#    om_r=None,
#    w_p=None,
#    k_index=k_ind,
#    exp_ops=None,
#    ti=0.0,
#    mode="Free",
#    progress_bar="ON",
#    i=0,
#):
#    if b is None: b = B_j
#    if om is None: om = omega
#    if p is None: p = phi
#    if om_r is None: om_r = Om_r
#    if w_p is None: w_p = W_p
#    if exp_ops is None: exp_ops = [
#        n1_j, n2_j, n3_j, n4_j, n5_j, n6_j, n7_j, nc_j, 
#        sx_gs_j, sy_gs_j, sz_gs_j, 
#        sx_es_j, sy_es_j, sz_es_j
#    ]
#    
#    args = {"w": om, "p": p}
#    
#    t_bins = 540
#    if dt <= 1:
#        t_bins = 54 
#    elif dt <= 2:
#        t_bins = 108 
#        
#    tf = ti + dt
#    match mode:
#        case "Free":
#            c_ops = L_jax_no(0.0, k_index=k_index)
#            H = H_jax_no(b, 0.0)
#        case "MW":
#            c_ops = L_jax_no(0.0, k_index=k_index)
#            H = H_jax_no(b, om_r)
#        case "Laser":
#            c_ops = L_jax_no(w_p, k_index=k_index)
#            H = H_jax_no(b, 0.0)
#        case "Laser-MW":
#            c_ops = L_jax_no(w_p, k_index=k_index)
#            H = H_jax_no(b, om_r)
#        case _:
#            raise ValueError('mode must be one of "Free", "MW", "Laser", or "Laser-MW"')
#    c_ops.append(jnp.sqrt(gamma_gs[1]) * sz_gs_j)
#    c_ops.append(jnp.sqrt(gamma_gs[0] / 2) * (sx_gs_j - 1j * sy_gs_j))
#    c_ops.append(jnp.sqrt(gamma_es[1]) * sz_es_j)
#    c_ops.append(jnp.sqrt(gamma_es[0] / 2) * (sx_es_j - 1j * sy_es_j))
#    # Call the master equation solver
#    match progress_bar:
#        case "OFF":
#            result = qt.mesolve(H,
#                                   init_state,
#                                   jnp.linspace(ti, tf, t_bins + 1),
#                                   c_ops,
#                                   
#                                   args=args,
#                                   options={"store_states": True,"method": "diffrax"}
#                                   )
#        case "ON":
#            print(f"{mode} {int(i + 1)} \n ti | tf \n {int(ti)} | {int(tf)}")
#            result = qt.mesolve(H,
#                                   init_state,
#                                   jnp.linspace(ti, tf, t_bins + 1),
#                                   c_ops,
#                                   
#                                   args=args,
#                                   options={"store_states": True, "progress_bar": "tqdm","method": "diffrax"}
#                                   )
#        case _:
#            raise ValueError('progress_bar must be "ON" or "OFF"')
#    return tf, result
#

## Magaletti Sequence

In [ ]:
# Initial state where the ground state is evenly populated
init_state = 1 / 3 * (n1 + n2 + n3)

# Array to collect the result Class
results = np.array([])

# Initial time of the evolution
ti = 0.0

# Store the times of activation and deativation of laser and microwaves for region highlight in plotting
tis = []
tfs = []

# Set the number of times to repeat the Sequence
n = 1
# Set the time intervals for each part of the evolution (with laser, with microwaves and free evolution)
dt_laser = 10.0
dt_free = 1.0
dt_mw = 5.0
print("Starting calculations, this may take a while...")
# This loop is defined to reproduce the Sequence of Magalleti el al
for i in tqdm(range(n)):
    tis.append(ti)
    # Here Laser is ON
    # Run the dynamics_no based on the mode choosen
    tf, result = dynamics_no(
        dt_laser,
        init_state,
        ti=ti,
        mode="Laser",
        progress_bar="ON",
        i=i,
    ) # type:ignore
    # Store the Result class for when the Laser is on
    results = np.append(results, result) # type: ignore 
    # Save the initial state for the next part of the Sequence
    init_state = result.states[-1]
    # Make the initial time the end time of the previous part of the procotol
    ti = tf
    tfs.append(tf)
    # Run the dynamics_no based on the mode choosen
    tf, result = dynamics_no(
        dt_free,
        init_state,
        ti=ti,
        mode="Free",
        progress_bar="ON",
        i=i,
    ) # type:ignore
    # Store the Result class for MW on evolution
    results = np.append(results, result) # type: ignore 
    # Save the times for coloring areas of the plot
    tis.append(ti)
    tfs.append(tf)
    # Make initial time to be the end time of the last part of the Sequence
    ti = tf
    # Make the last state  of this evolution step be the initial states of the next iteration
    init_state = result.states[-1]
    # Run the dynamics_no based on the mode choosen
    tf, result = dynamics_no(dt_mw, 
                          init_state, 
                          ti=ti, 
                          mode="MW", 
                          progress_bar="ON", 
                          i=i) # type:ignore
    # Store the Result class for MW on evolution
    results = np.append(results, result) # type: ignore
    # Save the times for coloring areas of the plot
    tis.append(ti)
    tfs.append(tf)
    # Make initial time to be the end time of the last part of the Sequence
    ti = tf
    # Make the last state  of this evolution step be the initial states of the next iteration
    init_state = result.states[-1]
# Gather the expectation values from the results array
times = np.concatenate([res.times for res in results])
N = [n1, n2, n3, n4, n5, n6, n7, nc]
n_exp = np.array([np.concatenate([qt.expect(M, res.states) for res in results]) for M in N])
S = [sx_gs, sy_gs, sz_gs, sx_es, sy_es, sz_es]
s_exp = np.array([np.concatenate([qt.expect(M, res.states)  for res in results]) for M in S])
#for j in range(len(results[0].expect)):
#    nn_exp = np.array([])
#    for i in range(len(results)):
#        nn_exp = np.append(nn_exp, results[i].expect[j])
#    if j == 0:
#        n_exp = np.array([nn_exp])
#    else:
#        n_exp = np.append(n_exp, [nn_exp], axis=0)
## Gather the times for the plots
#times = np.array([])
#for i in range(len(results)):
#    times = np.append(times, results[i].times)
#if len(n_exp)>8:
#    s_exp=n_exp[len(n_exp)-6:]
#    n_exp=n_exp[:len(n_exp)-6]
print("Calculations are finished!! Go to the next block for plots")

### Plots

#### PL

In [ ]:
c = 0
print("Starting calculations, this may take a while...")
for dt_mw in tqdm([0.0,0.5*np.pi/Om_r,np.pi/Om_r],position=0):# Initial state where the ground state is evenly populated
    init_state = n1 
    if dt_mw==0.5*np.pi/Om_r:
        init_state = (n1 + n2 + ground[1]*ground[2].dag() + ground[2]*ground[1].dag())/2 #type:ignore
    if dt_mw==np.pi/Om_r:   
        init_state = n2
    
    # Array to collect the result Class
    results_t = np.array([])
    
    # Initial time of the evolution
    ti = 0.0
    
    # Store the times of activation and deativation of laser and microwaves for region highlight in plotting
    tis_t = []
    tfs_t = []
    
    # Set the number of times to repeat the Sequence
    n = 1
    # Set the time intervals for each part of the evolution (with laser, with microwaves and free evolution)
    dt_laser = 10.0
    dt_free = 1.0
    # This loop is defined to reproduce the Sequence of Magalleti el al
    for i in tqdm(range(n),position=1,leave=False):
        tis_t.append(ti)
        # Here Laser is ON
        # Run the dynamics_no based on the mode choosen
        tf, result = dynamics_no(
            dt_laser,
            init_state,
            ti=ti,
            mode="Laser",
            progress_bar="ON",
            i=i,
        ) # type:ignore
        # Store the Result class for when the Laser is on
        results_t = np.append(results_t, result) # type: ignore 
        # Save the initial state for the next part of the Sequence
        init_state = result.states[-1]
        # Make the initial time the end time of the previous part of the procotol
        ti = tf
        tfs_t.append(tf)
    times_t = np.concatenate([res.times for res in results_t])
    N = [n1, n2, n3, n4, n5, n6, n7, nc]
    n_exp_t = np.array([np.concatenate([qt.expect(M, res.states) for res in results_t]) for M in N])
    S = [sx_gs, sy_gs, sz_gs, sx_es, sy_es, sz_es]
    s_exp_t = np.array([np.concatenate([qt.expect(M, res.states)  for res in results_t]) for M in S])
    # Gather the expectation values from the results array
    #for j in range(len(results_t[0].expect)):
    #    nn_exp = np.array([])
    #    for i in range(len(results_t)):
    #        nn_exp = np.append(nn_exp, results_t[i].expect[j])
    #    if j == 0:
    #        n_exp_t = np.array([nn_exp])
    #    else:
    #        n_exp_t = np.append(n_exp_t, [nn_exp], axis=0)
    # Gather the times for the plots
    #times_t = np.array([])
    #for i in range(len(results_t)):
    #    times_t = np.append(times_t, results_t[i].times)
    #if len(n_exp)>8:
    #    s_exp_t=n_exp_t[len(n_exp_t)-6:]
    #    n_exp_t=n_exp_t[:len(n_exp_t)-6]
    pl=np.array([])
    pl=np.add(n_exp_t[3],n_exp_t[4])
    pl=np.add(pl,n_exp_t[5])
    if c==0:
        pls_pure = np.array([pl])
        c+=1
    else:
        pls_pure = np.append(pls_pure,[pl],axis=0)
print(f"Calculations are finished!! See the plots below")
# Set the colors of each state
colors = [
    "dodgerblue",
    "chocolate",
    "gold",
    "mediumpurple",
    "mediumseagreen",
    "lightskyblue",
    "magenta",
    "mediumvioletred",
    "forestgreen",
]
# Defines figure size
plt.figure(figsize=(14, 8))
a=0
state=[r'|0\rangle',r'\frac{|0\rangle+|1\rangle}{\sqrt{2}}',r'|1\rangle']
for pl in pls_pure:
    plt.plot(times_t[0:],pl[0:], lw=4, color=colors[a], label=f"init state=${state[a]}$")
    a+=1
times_t_pure=times_t[0:].copy()
eps_y = abs(np.min(pls_pure) - np.max(pls_pure))*5e-2
eps_x = abs(np.min(times_t) - np.max(times_t))*5e-2
plt.ylabel(f"PL(a.u.)",fontsize=20)
plt.xlabel(r"Time($\mu$s)",fontsize=20)

# set the time limits you want to show in the plot
t_lim = (times_t[0]-eps_x, times_t[-1] + eps_x)
plt.xlim(t_lim)
plt.ylim((np.min(pls_pure) - eps_y, np.max(pls_pure) + eps_y))
plt.minorticks_on()

plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left",fontsize=16)
plt.show()

In [ ]:
c = 0
print("Starting calculations, this may take a while...")
for dt_mw in tqdm([0.0,0.5*np.pi/Om_r,np.pi/Om_r],position=0):# Initial state where the ground state is evenly populated
    init_state = 1 / 3 * (n1 + n2 + n3)
    
    # Array to collect the result Class
    results_t = np.array([])
    
    # Initial time of the evolution
    ti = 0.0
    
    # Store the times of activation and deativation of laser and microwaves for region highlight in plotting
    tis_t = []
    tfs_t = []
    
    # Set the number of times to repeat the Sequence
    n = 1
    # Set the time intervals for each part of the evolution (with laser, with microwaves and free evolution)
    dt_laser = 10.0
    dt_free = 1.0
    # This loop is defined to reproduce the Sequence of Magalleti el al
    for i in tqdm(range(n),position=1,leave=False):
        tis_t.append(ti)
        # Here Laser is ON
        # Run the dynamics based on the mode choosen
        tf, result = dynamics_no(
            dt_laser,
            init_state,
            ti=ti,
            mode="Laser",
            progress_bar="ON",
            i=i,
        ) # type:ignore
        # Store the Result class for when the Laser is on
        results_t = np.append(results_t, result) # type: ignore 
        # Save the initial state for the next part of the Sequence
        init_state = result.states[-1]
        # Make the initial time the end time of the previous part of the procotol
        ti = tf
        tfs_t.append(tf)
        # Run the dynamics based on the mode choosen
        tf, result = dynamics_no(
            dt_free,
            init_state,
            ti=ti,
            mode="Free",
            progress_bar="ON",
            i=i,
        ) # type:ignore
        # Store the Result class for MW on evolution
        results_t = np.append(results_t, result) # type: ignore 
        # Save the times for coloring areas of the plot
        tis_t.append(ti)
        tfs_t.append(tf)
        # Make initial time to be the end time of the last part of the Sequence
        ti = tf
        # Make the last state  of this evolution step be the initial states of the next iteration
        init_state = result.states[-1]
        # Run the dynamics based on the mode choosen
        tf, result = dynamics_no(dt_mw, 
                              init_state, 
                              ti=ti, 
                              mode="MW", 
                              progress_bar="ON", 
                              i=i) # type:ignore
        # Store the Result class for MW on evolution
        results_t = np.append(results_t, result) # type: ignore
        # Save the times for coloring areas of the plot
        tis_t.append(ti)
        tfs_t.append(tf)
        # Make initial time to be the end time of the last part of the Sequence
        ti = tf
        # Make the last state  of this evolution step be the initial states of the next iteration
        init_state = result.states[-1]
        tis_t.append(ti)
        # Here Laser is ON
        # Run the dynamics based on the mode choosen
        tf, result = dynamics_no(
            dt_laser,
            init_state,
            ti=ti,
            mode="Laser",
            progress_bar="ON",
            i=i+1,
        ) # type:ignore
        # Store the Result class for when the Laser is on
        results_t = np.append(results_t, result) # type: ignore 
        # Save the initial state for the next part of the Sequence
        init_state = result.states[-1]
        # Make the initial time the end time of the previous part of the procotol
        ti = tf
        tfs_t.append(tf)
    times_t = np.concatenate([res.times for res in results_t])
    N = [n1, n2, n3, n4, n5, n6, n7, nc]
    n_exp_t = np.array([np.concatenate([qt.expect(M, res.states) for res in results_t]) for M in N])
    S = [sx_gs, sy_gs, sz_gs, sx_es, sy_es, sz_es]
    s_exp_t = np.array([np.concatenate([qt.expect(M, res.states)  for res in results_t]) for M in S])
    # Gather the expectation values from the results array
    #for j in range(len(results_t[0].expect)):
    #    nn_exp = np.array([])
    #    for i in range(len(results_t)):
    #        nn_exp = np.append(nn_exp, results_t[i].expect[j])
    #    if j == 0:
    #        n_exp_t = np.array([nn_exp])
    #    else:
    #        n_exp_t = np.append(n_exp_t, [nn_exp], axis=0)
    ## Gather the times for the plots
    #times_t = np.array([])
    #for i in range(len(results_t)):
    #    times_t = np.append(times_t, results_t[i].times)
    #if len(n_exp)>8:
    #    s_exp_t=n_exp_t[len(n_exp_t)-6:]
    #    n_exp_t=n_exp_t[:len(n_exp_t)-6]
    pl=np.array([])
    pl=np.add(n_exp_t[3],n_exp_t[4])
    pl=np.add(pl,n_exp_t[5])
    if c==0:
        pls = np.array([pl])
        c+=1
    else:
        pls = np.append(pls,[pl],axis=0)
print(f"Calculations are finished!! See the plots below")
# Set the colors of each state
colors = [
    "dodgerblue",
    "chocolate",
    "gold",
    "mediumpurple",
    "mediumseagreen",
    "lightskyblue",
    "magenta",
    "mediumvioletred",
    "forestgreen",
]
# Defines figure size
plt.figure(figsize=(14, 8))
a=0
state=[r'|0\rangle',r'\frac{|0\rangle+|1\rangle}{\sqrt{2}}',r'|1\rangle']
for pl in pls:
    plt.plot(times_t[-5200:],pl[-5200:], lw=4, color=colors[a], label=f"init state=${state[a]}$")
    a+=1

eps_y = abs(np.min(pls) - np.max(pls))*5e-2
eps_x = abs(np.min(times_t) - np.max(times_t))*5e-2
plt.ylabel(f"PL(a.u.)",fontsize=20)
plt.xlabel(r"Time($\mu$s)",fontsize=20)

# set the time limits you want to show in the plot
t_lim = (times_t[-5200]-eps_x, times_t[-1] + eps_x)
plt.xlim(t_lim)
plt.ylim((np.min(pls) - eps_y, np.max(pls) + eps_y))
plt.minorticks_on()

plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left",fontsize=16)
plt.show()

In [ ]:
plt.figure(figsize=(14, 8))
a=0
state=[r'|0\rangle',r'\frac{|0\rangle+|1\rangle}{\sqrt{2}}',r'|1\rangle']
for pl in pls_pure:
    plt.plot(times_t_pure[0:],pl[0:], lw=4, ls='--', color=colors[a], label=f"init Pure state=${state[a]}$")
    a+=1
a=0
for pl in pls:
    plt.plot(times_t[-5200:]-11.2,pl[-5200:], lw=4, color=colors[a], label=f"init state=${state[a]}$")
    a+=1
eps_y = abs(np.min(pls_pure) - np.max(pls_pure))*5e-2
eps_x = abs(np.min(times_t[-5200:]) - np.max(times_t[-5200:]))*5e-2
plt.ylabel(f"PL(a.u.)",fontsize=20)
plt.xlabel(r"Time($\mu$s)",fontsize=20)

# set the time limits you want to show in the plot
t_lim = (times_t_pure[0]-eps_x, times_t_pure[-1] + eps_x)
plt.xlim(t_lim)
plt.ylim((np.min(pls_pure) - eps_y, np.max(pls_pure) + eps_y))
plt.minorticks_on()

plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left",fontsize=16)
plt.show()

#### Magaletti Sequence

In [ ]:
# Set the colors of each state
colors = [
    "dodgerblue",
    "chocolate",
    "gold",
    "mediumpurple",
    "mediumseagreen",
    "lightskyblue",
    "magenta",
    "mediumvioletred",
    "forestgreen",
]
# Defines figure size
plt.figure(figsize=(14, 8))
plt.plot(times, n_exp[-1], label="$n_c$", color=colors[-1])
for i in range(len(n_exp)-1):
    plt.plot(times, n_exp[i], label=f"$n_{i+1}$", color=colors[i])
# Highlighting the times where the laser and microwaves are on and the metastable state depletion evolutions
n = 0
eps_y = abs(np.min(n_exp) - np.max(n_exp))*5e-2
eps_x = abs(np.min(times) - np.max(times))*5e-2
for i in zip(tis, tfs):
    if n % 3 == 0:
        plt.fill_betweenx(
            [np.min(n_exp) - eps_y, np.max(n_exp) + eps_y],
            i[0],
            i[1],
            color="palegreen",
            alpha=0.5 ** (int(n / 3) + 1),
            label=f"Laser ON #{int(n/3)+1}",
        )
    else:
        if (n - 1) % 3 == 0:
            plt.fill_betweenx(
                [np.min(n_exp) - eps_y, np.max(n_exp) + eps_y],
                i[0],
                i[1],
                color="lightblue",
                alpha=0.5 ** (int(n / 3) + 1),
                label=f"MS depl #{int(n/3)+1}",
            )
        else:
            plt.fill_betweenx(
                [np.min(n_exp) - eps_y, np.max(n_exp) + eps_y],
                i[0],
                i[1],
                color="orchid",
                alpha=0.5 ** (int(n / 3) + 1),
                label=f"MW ON #{int(n/3)+1}",
            )
    n += 1
plt.ylabel(f"Population",fontsize=20)
plt.xlabel(r"Time($\mu$s)",fontsize=20)

# set the time limits you want to show in the plot
t_lim = (times[0]-eps_x, times[-1] + eps_x)
plt.xlim(t_lim)
plt.ylim((np.min(n_exp) - eps_y, np.max(n_exp) + eps_y))
plt.minorticks_on()

plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left",fontsize=16)
plt.show()

In [ ]:
# Set the colors of each state
colors = [
    "dodgerblue",
    "chocolate",
    "gold",
    "mediumpurple",
    "mediumseagreen",
    "lightskyblue",
    "magenta",
    "mediumvioletred",
    "forestgreen",
]
op_label=["{x,gs}","{y,gs}","{z,gs}","{x,es}","{y,es}","{z,es}"]
# Defines figure size
plt.figure(figsize=(14, 8))
# Plot the population of each state
if dt_mw <=5:
    times_n=times[-1200:]
else:
    times_n=times[-5200:]
n_exp_n=[n_exp[i][-len(times_n):] for i in range(len(n_exp))]
s_exp_n=[s_exp[i][-len(times_n):] for i in range(len(s_exp))]

for i in range(len(s_exp_n)):
    plt.plot(times_n, s_exp_n[i], label=f"$<S_{op_label[i]}>$", color=colors[i])
plt.plot(times_n, n_exp_n[-1], label="$n_c$", color=colors[-2],ls='--')
plt.plot(times_n, n_exp_n[1], label="$n_2$", color=colors[-1],ls='--')
    
# Highlighting the times where the laser and microwaves are on and the metastable state depletion evolutions
n = 0
eps_y = abs(np.min(s_exp_n) - np.max(np.min(np.append(s_exp_n,[n_exp_n[1]],axis=0))))*5e-2
eps_x = abs(np.min(times_n) - np.max(times_n))*5e-2
if times_n[0]-times_n[-1]<=1.0:
    eps_x=eps_y
for i in zip(tis, tfs):
    if n % 3 == 0:
        plt.fill_betweenx(
            [np.min(s_exp_n) - eps_y, np.max(np.append(s_exp_n,[n_exp_n[1]],axis=0)) + eps_y],
            i[0],
            i[1],
            color="palegreen",
            alpha=0.5 ** (int(n / 3) + 1),
            label=f"Laser ON #{int(n/3)+1}",
        )
    else:
        if (n - 1) % 3 == 0:
            plt.fill_betweenx(
                [np.min(s_exp_n) - eps_y, np.max(np.append(s_exp_n,[n_exp_n[1]],axis=0)) + eps_y],
                i[0],
                i[1],
                color="lightblue",
                alpha=0.5 ** (int(n / 3) + 1),
                label=f"MS depl #{int(n/3)+1}",
            )
        else:
            plt.fill_betweenx(
                [np.min(s_exp_n) - eps_y, np.max(np.append(s_exp_n,[n_exp_n[1]],axis=0)) + eps_y],
                i[0],
                i[1],
                color="orchid",
                alpha=0.5 ** (int(n / 3) + 1),
                label=f"MW ON #{int(n/3)+1}",
            )
    n += 1
plt.ylabel(f"Population",fontsize=20)
plt.xlabel(r"Time($\mu$s)",fontsize=20)

# set the time limits you want to show in the plot
t_lim = (times_n[0]-eps_x, times_n[-1] + eps_x)
plt.xlim(t_lim)
plt.ylim((np.min(s_exp_n) - eps_y, np.max(np.append(s_exp_n,[n_exp_n[1]],axis=0)) + eps_y))
plt.minorticks_on()

plt.legend(ncol=2,bbox_to_anchor=(1.05, 1), loc="upper left",fontsize=16)
plt.show()

## Hyperfine hamiltonian

### Doherty

In [ ]:
# Dimention of the Hilbert space
dim = 7

t1_n=1e5
t2_n=1e3
# Constants (MHz)
gamma_n=1/t1_n,1/t2_n
mu_n = 431.7*1e-6 # (MHz/G)

a_gs=3.03,3.65
a_es=-57.8,-39.2

# Nitrogen Nucleous
IdN15 = qt.qeye(2)
nit = qt.basis(2, 0), qt.basis(2, 1) # |-1/2>_n, |+1/2>_n
sx_n, sy_n, sz_n = qt.sigmax()*0.5, qt.sigmay()*0.5, qt.sigmaz()*0.5
sm_n,sp_n=sx_n-1j*sy_n,sx_n+1j*sy_n
S_n=np.array([sx_n,sy_n,sz_n])

def H_doh_hf(b, om_r):
    """
    Calculate the Hamiltonian for a given magnetic field and resonant frequency.
    
    Parameters:
    b (numpy.ndarray): Magnetic field vector [bx,by,bz].
    om_r: Resonant frequency.
    
    Returns:
    list: List of Hamiltonian terms.
    """
    
    H_0 = [((D_gs*sz_gs**2+E_gs*(sx_gs**2-sy_gs**2)+mu_e*np.dot(b,S_gs))&IdN15)+
           ((D_es*sz_es**2+E_es*(sx_es**2-sy_es**2)+mu_e*np.dot(b,S_es))&IdN15)+
           a_gs[0]*(sz_gs&sz_n) + a_gs[1]*((sx_gs&sx_n) + (sy_gs&sy_n))+
           a_es[0]*(sz_es&sz_n) + a_es[1]*((sx_es&sx_n) + (sy_es&sy_n))]
    H_n = [IdNV&(mu_n*np.dot(b,S_n))]
    H_int=[[(np.sqrt(2)*om_r*sx_gs)&IdN15, "cos(w*t)*cos(p)"],
           [(np.sqrt(2)*om_r*sy_gs)&IdN15, "cos(w*t)*sin(p)"],
           [(np.sqrt(2)*om_r*sx_es)&IdN15, "cos(w*t)*cos(p)"],
           [(np.sqrt(2)*om_r*sy_es)&IdN15, "cos(w*t)*sin(p)"],
           [IdNV&(2*om_r/mu_e*mu_n*sx_n), "cos(w*t)*cos(p)"],
           [IdNV&(2*om_r/mu_e*mu_n*sy_n), "cos(w*t)*sin(p)"]]
    return [*H_0,*H_n,*H_int] 

def L_doh_hf(w_p,k_index=k_ind):
    """
    Returns the Lindblad operators of the system, including optical transitions based on the given k_index.

    Parameters:
    - w_p (float): Laser pump rate.
    - k_index (int, optional): Index for the optical transition rates. Defaults to k_ind.

    Returns:
    - c_ops (list): List of Lindblad operators.
    """
    K_s=[[66.0,0.0,57.0,1.0,0.7],
         [77.0,0.0,30.0,3.3,0.0],
         [62.7,12.97,80.0,3.45,1.08],
         [63.2,10.8,60.7,0.8,0.4],
         [67.4,9.9,96.6,4.83,1.055]]
    k41 = K_s[k_index][0]
    k52 = K_s[k_index][0]
    k63 = K_s[k_index][0]
    k57 = K_s[k_index][2]
    k67 = K_s[k_index][2]
    k47 = K_s[k_index][1]
    k71 = K_s[k_index][3]
    k72 = K_s[k_index][4]
    k73 = K_s[k_index][4]
    
    c_ops = []

    c_ops.append((np.sqrt(w_p) * (excited[1] * ground[1].dag()))&IdN15)  # n1 to n4 #type: ignore
    c_ops.append((np.sqrt(w_p) * (excited[2] * ground[2].dag()))&IdN15)  # n2 to n5 #type: ignore
    c_ops.append((np.sqrt(w_p) * (excited[0] * ground[0].dag()))&IdN15) # n3 to n6 #type: ignore

    c_ops.append((np.sqrt(k41) * (ground[1] * excited[1].dag()))&IdN15)  # n4 to n1 #type: ignore
    c_ops.append((np.sqrt(k71) * (ground[1] * isc.dag()))&IdN15)  # n7 to n1 #type: ignore

    c_ops.append((np.sqrt(k52) * (ground[2] * excited[2].dag()))&IdN15)  # n5 to n2 #type: ignore
    c_ops.append((np.sqrt(k72) * (ground[2] * isc.dag()))&IdN15)  # n7 to n2 #type: ignore

    c_ops.append((np.sqrt(k63) * (ground[0] * excited[0].dag()))&IdN15)  # n6 to n3 #type: ignore
    c_ops.append((np.sqrt(k73) * (ground[0] * isc.dag()))&IdN15)  # n7 to n3 #type: ignore

    c_ops.append((np.sqrt(k47) * (isc * excited[1].dag()))&IdN15)  # n4 to n7 #type: ignore
    c_ops.append((np.sqrt(k57) * (isc * excited[2].dag()))&IdN15)  # n5 to n7 #type: ignore
    c_ops.append((np.sqrt(k67) * (isc * excited[0].dag()))&IdN15)  # n6 to n7 #type: ignore
    # Collapse operators for decoherence   
    c_ops.append((np.sqrt(gamma_gs[1]) * sz_gs)&IdN15)
    c_ops.append((np.sqrt(gamma_gs[0]/2) * (sm_gs))&IdN15)
    c_ops.append((np.sqrt(gamma_gs[0]/2) * (sp_gs))&IdN15)
    c_ops.append((np.sqrt(gamma_es[1]) * sz_es)&IdN15)
    c_ops.append((np.sqrt(gamma_es[0]/2) * (sm_es))&IdN15)
    c_ops.append((np.sqrt(gamma_es[0]/2) * (sp_es))&IdN15)
    c_ops.append(IdNV&(np.sqrt(gamma_n[1]) * sz_n))
    c_ops.append(IdNV&(np.sqrt(gamma_n[0]/2) * (sm_n)))
    c_ops.append(IdNV&(np.sqrt(gamma_n[0]/2) * (sp_n)))
    return c_ops

def dynamics_doh_hf(
    dt,
    init_state,
    b=None,
    om_r=None,
    om=None,
    p=None,
    w_p=None,
    k_index=k_ind,
    #exp_ops=None,
    ti=0.0,
    mode="Free",
    progress_bar="ON",
    i=0,
):
    """
    Simulate the dynamics of a quantum system under hyperfine interaction using the Hamiltonian and collapse operators.
    including optical transition rates index.
    Where, when using k_index=2 -> K_s[k_index]=[62.7,12.97,80.0,3.45,1.08], and the full K_s list is:
        K_s=[[66.0,0.0,57.0,1.0,0.7],
             [77.0,0.0,30.0,3.3,0.0],
             [62.7,12.97,80.0,3.45,1.08],
             [63.2,10.8,60.7,0.8,0.4],
             [67.4,9.9,96.6,4.83,1.055]]
    Parameters:
    - dt (float): Total simulation time.
    - init_state (qt.Qobj): Initial quantum state of the system.
    - b (np.array, optional): Magnetic field vector. Defaults to B0(B_amp,phi_B,theta_B).
    - om_r (float, optional): Rabi frequency for microwave interactions. Defaults to Om_r.
    - om (float, optional): Angular frequency of the system. Defaults to omega.
    - p (float, optional): Microwave phase. Defaults to 0.0.
    - w_p (float, optional): Laser frequency. Defaults to W_p.
    - k_index(int, optional): Index for the optical transition rates. Defaults to k_ind.
    - exp_ops (list of qt.Qobj, optional): List of operators for which to compute expectation values. 
        Defaults to a list of tensor products including different Pauli operators and state projectors.
    - ti (float, optional): Initial time of the simulation. Defaults to 0.0.
    - mode (str, optional): Simulation mode. Can be "Free", "MW", "Laser", or "Laser-MW". Defaults to "Free".
    - progress_bar (str, optional): Option to display a progress bar. Can be "ON" or "OFF". Defaults to "ON".
    - i (int, optional): Counter for the progress bar. Defaults to 0.

    Returns:
    - tf (float): Final time of the simulation.
    - result (qutip.solver.Result): Result object containing the simulation output.
    """
    # Default values
    if b is None: b = B
    if om_r is None: om_r = Om_r
    if om is None: om = omega
    if p is None: p = phi
    if w_p is None: w_p = W_p
    #if exp_ops is None: exp_ops = [
    #    n1&IdN15, n2&IdN15, n3&IdN15,
    #    n4&IdN15, n5&IdN15, n6&IdN15,
    #    n7&IdN15, nc&IdN15,
    #    sx_gs&IdN15, sy_gs&IdN15, sz_gs&IdN15,
    #    sx_es&IdN15, sy_es&IdN15, sz_es&IdN15,
    #    IdNV&nit[0] * nit[0].dag(), IdNV&nit[1] * nit[1].dag() #type: ignore
    #]
    # Arguments for the Hamiltonian
    args = {"w": om, "p": p}
    # Time resolution based on dt
    t_bins = 1000 if dt <= 5 else 5000
    
    tf = ti + dt
    
    # Define Hamiltonian and collapse operators based on mode
    match mode:
        case "Free":
            H = H_doh_hf(b, 0.0)
            c_ops = L_doh_hf(0.0, k_index=k_index)
        case "MW":
            H = H_doh_hf(b, om_r)
            c_ops = L_doh_hf(0.0, k_index=k_index)
        case "Laser":
            H = H_doh_hf(b, 0.0)
            c_ops = L_doh_hf(w_p, k_index=k_index)
        case "Laser-MW":
            H = H_doh_hf(b, om_r)
            c_ops = L_doh_hf(w_p, k_index=k_index)
        case _:
            raise ValueError('mode must be one of "Free", "MW", "Laser", or "Laser-MW"')

    # Solve the master equation
    match progress_bar:
        case "OFF":
            result = qt.mesolve(
                H,
                init_state,
                np.linspace(ti, tf, t_bins + 1),
                c_ops,
                #e_ops=exp_ops,
                args=args,
                options={"store_states": True},
            )
        case "ON":
            print(f"{mode} {int(i + 1)} \n ti    |    tf \n {ti:.2f} | {tf:.2f}")
            result = qt.mesolve(
                H,
                init_state,
                np.linspace(ti, tf, t_bins + 1),
                c_ops,
                #e_ops=exp_ops,
                args=args,
                options={"store_states": True, "progress_bar": "tqdm"},
            )
        case _:
            raise ValueError('progress_bar must be "ON" or "OFF"')

    return tf, result

### Duarte

In [ ]:
# Dimention of the Hilbert space of the NV center
dim = 7

t1_n=1e5
t2_n=1e3
# Constants (MHz)
gamma_n=1/t1_n,1/t2_n
mu_n = 431.7*1e-6 # (MHz/G)

A_par = 3.4,-58.1
A_perp = 7.8,-77.0
A_ani = 0.0, 0.0
A_perp_prime = 0.0,0.0
phi_H = 0.0

def H_dua_hf(b, om_r,phi_h):
    """
    Calculate the Hamiltonian for a given magnetic field and resonant frequency, including hyperfine interactions.
    
    Parameters:
    b (numpy.ndarray): Magnetic field vector [bx, by, bz].
    om_r: Resonant frequency.
    
    Returns:
    list: List of Hamiltonian terms.
    """
    H_0 = [((D_gs*sz_gs**2 + E_gs*(sx_gs**2-sy_gs**2) + mu_e*np.dot(b, S_gs))&IdN15)+
           ((D_es*sz_es**2 + E_es*(sx_es**2-sy_es**2) + mu_e*np.dot(b, S_es))&IdN15)+
           A_par[0]*(sz_gs&sz_n) + A_perp[0]/4*((sp_gs&sm_n)+(sm_gs&sp_n))+
           A_perp_prime[0]/4*((sp_gs&sp_n)*np.exp(-2j*phi_h) + (sm_gs&sm_n)*np.exp(2j*phi_h))+
           A_ani[0]/2*((sp_gs&sz_n) + (sz_gs&sp_n)*np.exp(-1j*phi_h) + (sm_gs&sz_n) + (sz_gs&sm_n)*np.exp(1j*phi_h))+
           A_par[1]*(sz_es&sz_n) + A_perp[1]/4*((sp_es&sm_n)+(sm_es&sp_n))+
           A_perp_prime[1]/4*((sp_es&sp_n)*np.exp(-2j*phi_h) + (sm_es&sm_n)*np.exp(2j*phi_h))+
           A_ani[1]/2*((sp_es&sz_n) + (sz_es&sp_n)*np.exp(-1j*phi_h) + (sm_es&sz_n) + (sz_es&sm_n)*np.exp(1j*phi_h))]
    H_n = [IdNV&mu_n*np.dot(b, S_n)]
    H_int = [[(np.sqrt(2)*om_r*sx_gs)&IdN15, "cos(w*t)*cos(p)"],
             [(np.sqrt(2)*om_r*sy_gs)&IdN15, "cos(w*t)*sin(p)"],
             [(np.sqrt(2)*om_r*sx_es)&IdN15, "cos(w*t)*cos(p)"],
             [(np.sqrt(2)*om_r*sy_es)&IdN15, "cos(w*t)*sin(p)"],
             [IdNV&(2*om_r/mu_e*mu_n*sx_n), "cos(w*t)*cos(p)"],
             [IdNV&(2*om_r/mu_e*mu_n*sy_n), "cos(w*t)*sin(p)"]]
    return [*H_0, *H_n, *H_int]

def L_dua_hf(w_p,k_index=k_ind):
    """
    Returns the Lindblad operators of the system, including optical transitions based on the given k_index.

    Parameters:
    - w_p (float): Laser pump rate.
    - k_index (int, optional): Index for the optical transition rates. Defaults to k_ind.

    Returns:
    - c_ops (list): List of Lindblad operators.
    """
    K_s=[[66.0,0.0,57.0,1.0,0.7],
         [77.0,0.0,30.0,3.3,0.0],
         [62.7,12.97,80.0,3.45,1.08],
         [63.2,10.8,60.7,0.8,0.4],
         [67.4,9.9,96.6,4.83,1.055]]
    k41 = K_s[k_index][0]
    k52 = K_s[k_index][0]
    k63 = K_s[k_index][0]
    k57 = K_s[k_index][2]
    k67 = K_s[k_index][2]
    k47 = K_s[k_index][1]
    k71 = K_s[k_index][3]
    k72 = K_s[k_index][4]
    k73 = K_s[k_index][4]
    
    c_ops = []

    c_ops.append((np.sqrt(w_p) * (excited[1] * ground[1].dag()))&IdN15)  # n1 to n4 #type: ignore
    c_ops.append((np.sqrt(w_p) * (excited[2] * ground[2].dag()))&IdN15)  # n2 to n5 #type: ignore
    c_ops.append((np.sqrt(w_p) * (excited[0] * ground[0].dag()))&IdN15) # n3 to n6 #type: ignore

    c_ops.append((np.sqrt(k41) * (ground[1] * excited[1].dag()))&IdN15)  # n4 to n1 #type: ignore
    c_ops.append((np.sqrt(k71) * (ground[1] * isc.dag()))&IdN15)  # n7 to n1 #type: ignore

    c_ops.append((np.sqrt(k52) * (ground[2] * excited[2].dag()))&IdN15)  # n5 to n2 #type: ignore
    c_ops.append((np.sqrt(k72) * (ground[2] * isc.dag()))&IdN15)  # n7 to n2 #type: ignore

    c_ops.append((np.sqrt(k63) * (ground[0] * excited[0].dag()))&IdN15)  # n6 to n3 #type: ignore
    c_ops.append((np.sqrt(k73) * (ground[0] * isc.dag()))&IdN15)  # n7 to n3 #type: ignore

    c_ops.append((np.sqrt(k47) * (isc * excited[1].dag()))&IdN15)  # n4 to n7 #type: ignore
    c_ops.append((np.sqrt(k57) * (isc * excited[2].dag()))&IdN15)  # n5 to n7 #type: ignore
    c_ops.append((np.sqrt(k67) * (isc * excited[0].dag()))&IdN15)  # n6 to n7 #type: ignore
    # Add collapse operators for decoherence   
    c_ops.append((np.sqrt(gamma_gs[1]) * sz_gs)&IdN15)
    c_ops.append((np.sqrt(gamma_gs[0]/2) * (sm_gs))&IdN15)
    c_ops.append((np.sqrt(gamma_gs[0]/2) * (sp_gs))&IdN15)
    c_ops.append((np.sqrt(gamma_es[1]) * sz_es)&IdN15)
    c_ops.append((np.sqrt(gamma_es[0]/2) * (sm_es))&IdN15)
    c_ops.append((np.sqrt(gamma_es[0]/2) * (sp_es))&IdN15)
    c_ops.append(IdNV&(np.sqrt(gamma_n[1]) * sz_n))
    c_ops.append(IdNV&(np.sqrt(gamma_n[0]/2) * (sm_n)))
    c_ops.append(IdNV&(np.sqrt(gamma_n[0]/2) * (sp_n)))
    return c_ops

def dynamics_dua_hf(
    dt,
    init_state,
    b=None,
    om_r=None,
    om=None,
    p=None,
    w_p=None,
    k_index=k_ind,
    #exp_ops=None,
    ti=0.0,
    mode="Free",
    progress_bar="ON",
    i=0,
):
    """
    Simulate the dynamics of a quantum system under hyperfine interaction using the Hamiltonian and collapse operators.
    including optical transition rates index.
    Where, when using k_index=2 -> K_s[k_index]=[62.7,12.97,80.0,3.45,1.08], and the full K_s list is:
        K_s=[[66.0,0.0,57.0,1.0,0.7],
             [77.0,0.0,30.0,3.3,0.0],
             [62.7,12.97,80.0,3.45,1.08],
             [63.2,10.8,60.7,0.8,0.4],
             [67.4,9.9,96.6,4.83,1.055]]
    Parameters:
    - dt (float): Total simulation time.
    - init_state (qt.Qobj): Initial quantum state of the system.
    - b (np.array, optional): Magnetic field vector. Defaults to B0(B_amp,phi_B,theta_B).
    - om_r (float, optional): Rabi frequency for microwave interactions. Defaults to Om_r.
    - om (float, optional): Angular frequency of the system. Defaults to omega.
    - p (float, optional): Microwave phase. Defaults to 0.0.
    - w_p (float, optional): Laser frequency. Defaults to W_p.
    - k_index(int, optional): Index for the optical transition rates. Defaults to k_ind.
    - exp_ops (list of qt.Qobj, optional): List of operators for which to compute expectation values. 
        Defaults to a list of tensor products including different Pauli operators and state projectors.
    - ti (float, optional): Initial time of the simulation. Defaults to 0.0.
    - mode (str, optional): Simulation mode. Can be "Free", "MW", "Laser", or "Laser-MW". Defaults to "Free".
    - progress_bar (str, optional): Option to display a progress bar. Can be "ON" or "OFF". Defaults to "ON".
    - i (int, optional): Counter for the progress bar. Defaults to 0.

    Returns:
    - tf (float): Final time of the simulation.
    - result (qutip.solver.Result): Result object containing the simulation output.
    """
    # Default values
    if b is None: b = B
    if om_r is None: om_r = Om_r
    if om is None: om = omega
    if p is None: p = phi
    if w_p is None: w_p = W_p
    #if exp_ops is None: exp_ops = [
    #    n1&IdN15, n2&IdN15, n3&IdN15,
    #    n4&IdN15, n5&IdN15, n6&IdN15,
    #    n7&IdN15, nc&IdN15,
    #    sx_gs&IdN15, sy_gs&IdN15, sz_gs&IdN15,
    #    sx_es&IdN15, sy_es&IdN15, sz_es&IdN15,
    #    IdNV&nit[0] * nit[0].dag(), IdNV&nit[1] * nit[1].dag() #type: ignore
    #]
    # Arguments for the Hamiltonian
    args = {"w": om, "p": p}
    # Time resolution based on dt
    t_bins = 1000 if dt <= 5 else 5000
    
    tf = ti + dt
    
    # Define Hamiltonian and collapse operators based on mode
    match mode:
        case "Free":
            H = H_dua_hf(b, 0.0, phi_H)
            c_ops = L_dua_hf(0.0, k_index=k_index)
        case "MW":
            H = H_dua_hf(b, om_r, phi_H)
            c_ops = L_dua_hf(0.0, k_index=k_index)
        case "Laser":
            H = H_dua_hf(b, 0.0, phi_H)
            c_ops = L_dua_hf(w_p, k_index=k_index)
        case "Laser-MW":
            H = H_dua_hf(b, om_r, phi_H)
            c_ops = L_dua_hf(w_p, k_index=k_index)
        case _:
            raise ValueError('mode must be one of "Free", "MW", "Laser", or "Laser-MW"')
    # Solve the master equation
    match progress_bar:
        case "OFF":
            result = qt.mesolve(
                H,
                init_state,
                np.linspace(ti, tf, t_bins + 1),
                c_ops,
                #e_ops=exp_ops,
                args=args,
                options={"store_states": True},
            )
        case "ON":
            print(f"{mode} {int(i + 1)} \n ti    |    tf \n {ti:.2f} | {tf:.2f}")
            result = qt.mesolve(
                H,
                init_state,
                np.linspace(ti, tf, t_bins + 1),
                c_ops,
                #e_ops=exp_ops,
                args=args,
                options={"store_states": True, "progress_bar": "tqdm"},
            )
        case _:
            raise ValueError('progress_bar must be "ON" or "OFF"')

    return tf, result

### Sequence

In [ ]:
# Initial state where the ground state is evenly populated
init_state = ((n1+n2+n3)&IdN15)/6

# Array to collect the result Class
results_hf = np.array([])

# Initial time of the evolution
ti = 0.0

# Store the times of activation and deativation of laser and microwaves for region highlight in plotting
tis_hf = []
tfs_hf = []

# Set the number of times to repeat the Sequence
n = 1
# Set the time intervals for each part of the evolution (with laser, with microwaves and free evolution)
dt_laser = 10.0
dt_free = 1.0
dt_mw = 5.0
print("Starting calculations, this may take a while...")
# This loop is defined to reproduce the Sequence of Magalleti el al
for i in tqdm(range(n)):
    tis_hf.append(ti)
    # Here Laser is ON
    # Run the dynamics based on the mode choosen
    tf, result = dynamics_doh_hf(
        dt_laser,
        init_state,
        ti=ti,
        mode="Laser",
        progress_bar="ON",
        i=i,
    ) # type:ignore
    # Store the Result class for when the Laser is on
    results_hf = np.append(results_hf, result) # type: ignore 
    # Save the initial state for the next part of the Sequence
    init_state = result.states[-1]
    # Make the initial time the end time of the previous part of the procotol
    ti = tf
    tfs_hf.append(tf)
    # Run the dynamics based on the mode choosen
    tf, result = dynamics_doh_hf(
        dt_free,
        init_state,
        ti=ti,
        mode="Free",
        progress_bar="ON",
        i=i,
    ) # type:ignore
    # Store the Result class for MW on evolution
    results_hf = np.append(results_hf, result) # type: ignore 
    # Save the times for coloring areas of the plot
    tis_hf.append(ti)
    tfs_hf.append(tf)
    # Make initial time to be the end time of the last part of the Sequence
    ti = tf
    # Make the last state  of this evolution step be the initial states of the next iteration
    init_state = result.states[-1]
    # Run the dynamics based on the mode choosen
    tf, result = dynamics_doh_hf(dt_mw, 
                          init_state, 
                          ti=ti, 
                          mode="MW", 
                          progress_bar="ON", 
                          i=i) # type:ignore
    # Store the Result class for MW on evolution
    results_hf = np.append(results_hf, result) # type: ignore
    # Save the times for coloring areas of the plot
    tis_hf.append(ti)
    tfs_hf.append(tf)
    # Make initial time to be the end time of the last part of the Sequence
    ti = tf
    # Make the last state  of this evolution step be the initial states of the next iteration
    init_state = result.states[-1]
times_hf = np.concatenate([res.times for res in results_hf])
N = [n1&IdN15, n2&IdN15, n3&IdN15,
     n4&IdN15, n5&IdN15, n6&IdN15,
     n7&IdN15, nc&IdN15]
n_exp_hf = np.array([np.concatenate([qt.expect(M, res.states) for res in results_hf]) for M in N])
S = [sx_gs&IdN15, sy_gs&IdN15, sz_gs&IdN15,
     sx_es&IdN15, sy_es&IdN15, sz_es&IdN15]
s_exp_hf = np.array([np.concatenate([qt.expect(M, res.states)  for res in results_hf]) for M in S])
Nit = [IdNV&nit[0] * nit[0].dag(), IdNV&nit[1] * nit[1].dag()] #type: ignore
nit_exp_hf = np.array([np.concatenate([qt.expect(M, res.states)  for res in results_hf]) for M in Nit])
# Gather the expectation values from the results array
#for j in range(len(results_hf[0].expect)):
#    nn_exp = np.array([])
#    for i in range(len(results_hf)):
#        nn_exp = np.append(nn_exp, results_hf[i].expect[j])
#    if j == 0:
#        n_exp_hf = np.array([nn_exp])
#    else:
#        n_exp_hf = np.append(n_exp_hf, [nn_exp], axis=0)
#        
#if len(n_exp_hf)>8:
#    s_exp_hf=n_exp_hf[len(n_exp_hf)-8:len(n_exp_hf)-2]
#    nit_exp=n_exp_hf[len(n_exp_hf)-2:]
#    n_exp_hf=n_exp_hf[:len(n_exp_hf)-8]
## Gather the times for the plots
#times_hf = np.array([])
#for i in range(len(results)):
#    times_hf = np.append(times_hf, results_hf[i].times)
print("Calculations are finished!! Go to the next block for plots")

### Plots

In [ ]:
# Set the colors of each state
colors = [
    "dodgerblue",
    "chocolate",
    "gold",
    "mediumpurple",
    "mediumseagreen",
    "lightskyblue",
    "magenta",
    "mediumvioletred",
    "forestgreen",
]
# Defines figure size
plt.figure(figsize=(14, 8))
hf="{hf}"

# Plot the population of each state
plt.plot(times_hf, n_exp_hf[-1], label="$n^{hf}_c$", color=colors[-1])
for i in range(len(n_exp_hf) - 1):
    plt.plot(times_hf, n_exp_hf[i], label=f"$n^{hf}_{i+1}$", color=colors[i])

# Highlighting the times where the laser and microwaves are on and the metastable state depletion evolutions
n = 0
eps_y = abs(np.min(n_exp_hf) - np.max(n_exp_hf))*5e-2
eps_x = abs(np.min(times_hf) - np.max(times_hf))*5e-2
for i in zip(tis_hf, tfs_hf):
    if n % 3 == 0:
        plt.fill_betweenx(
            [np.min(n_exp_hf) - eps_y, np.max(n_exp_hf) + eps_y],
            i[0],
            i[1],
            color="palegreen",
            alpha=0.5 ** (int(n / 3) + 1),
            label=f"Laser ON ",
        )
    else:
        if (n - 1) % 3 == 0:
            plt.fill_betweenx(
                [np.min(n_exp_hf) - eps_y, np.max(n_exp_hf) + eps_y],
                i[0],
                i[1],
                color="lightblue",
                alpha=0.5 ** (int(n / 3) + 1),
                label=f"MS depl ",
            )
        else:
            plt.fill_betweenx(
                [np.min(n_exp_hf) - eps_y, np.max(n_exp_hf) + eps_y],
                i[0],
                i[1],
                color="orchid",
                alpha=0.5 ** (int(n / 3) + 1),
                label=f"MW ON ",
            )
    n += 1
plt.ylabel(f"Population",fontsize=20)
plt.xlabel(r"Time($\mu$s)",fontsize=20)

# set the time limits you want to show in the plot
t_lim = (times_hf[0]-eps_x, times_hf[-1] + eps_x)
plt.xlim(t_lim)
plt.ylim((np.min(n_exp_hf) - eps_y, np.max(n_exp_hf) + eps_y))
plt.minorticks_on()

plt.legend(ncol=2,bbox_to_anchor=(1.05, 1), loc="upper left",fontsize=16)
plt.show()

In [ ]:
# Set the colors of each state
colors = [
    "dodgerblue",
    "chocolate",
    "gold",
    "mediumpurple",
    "mediumseagreen",
    "lightskyblue",
    "magenta",
    "mediumvioletred",
    "forestgreen",
]
# Defines figure size
plt.figure(figsize=(14, 8))
hf="{hf}"

# Plot the population of each state
st=['{+1/2}', '{-1/2}']
for i in range(len(nit_exp_hf)):
    plt.plot(times_hf, nit_exp_hf[i], label=f"$N^{hf}_{st[i]}$", color=colors[i])

# Highlighting the times where the laser and microwaves are on and the metastable state depletion evolutions
n = 0
eps_y = abs(np.min(nit_exp_hf) - np.max(nit_exp_hf))*5e-2
eps_x = abs(np.min(times_hf) - np.max(times_hf))*5e-2
for i in zip(tis_hf, tfs_hf):
    if n % 3 == 0:
        plt.fill_betweenx(
            [np.min(nit_exp_hf) - eps_y, np.max(nit_exp_hf) + eps_y],
            i[0],
            i[1],
            color="palegreen",
            alpha=0.5 ** (int(n / 3) + 1),
            label=f"Laser ON ",
        )
    else:
        if (n - 1) % 3 == 0:
            plt.fill_betweenx(
                [np.min(nit_exp_hf) - eps_y, np.max(nit_exp_hf) + eps_y],
                i[0],
                i[1],
                color="lightblue",
                alpha=0.5 ** (int(n / 3) + 1),
                label=f"MS depl ",
            )
        else:
            plt.fill_betweenx(
                [np.min(nit_exp_hf) - eps_y, np.max(nit_exp_hf) + eps_y],
                i[0],
                i[1],
                color="orchid",
                alpha=0.5 ** (int(n / 3) + 1),
                label=f"MW ON ",
            )
    n += 1
plt.ylabel(f"Population",fontsize=20)
plt.xlabel(r"Time($\mu$s)",fontsize=20)

# set the time limits you want to show in the plot
t_lim = (times_hf[0]-eps_x, times_hf[-1] + eps_x)
plt.xlim(t_lim)
plt.ylim((np.min(nit_exp_hf) - eps_y, np.max(nit_exp_hf) + eps_y))
plt.minorticks_on()

plt.legend(ncol=2,bbox_to_anchor=(1.05, 1), loc="upper left",fontsize=16)
plt.show()

In [ ]:
# Set the colors of each state
colors = [
    "dodgerblue",
    "chocolate",
    "gold",
    "mediumpurple",
    "mediumseagreen",
    "lightskyblue",
    "magenta",
    "mediumvioletred",
    "forestgreen",
]
op_label=["{x,gs}","{y,gs}","{z,gs}","{x,es}","{y,es}","{z,es}"]
# Defines figure size
plt.figure(figsize=(14, 8))
# Plot the population of each state
if dt_mw <=5:
    times_n=times[-1200:]
else:
    times_n=times[-5200:]
n_exp_n=[n_exp_hf[i][-len(times_n):] for i in range(len(n_exp_hf))]
s_exp_n=[s_exp_hf[i][-len(times_n):] for i in range(len(s_exp_hf))]

for i in range(len(s_exp_n)):
    plt.plot(times_n, s_exp_n[i], label=f"$<S^{hf}_{op_label[i]}>$", color=colors[i])
plt.plot(times_n, n_exp_n[-1], label="$n^{hf}_c$", color=colors[-2],ls='--')
plt.plot(times_n, n_exp_n[1], label="$n^{hf}_2$", color=colors[-1],ls='--')

# Highlighting the times where the laser and microwaves are on and the metastable state depletion evolutions
n = 0
eps_y = abs(np.max(np.append(s_exp_n,[n_exp_n[1]],axis=0)) - np.min(s_exp_n))*5e-2
eps_x = abs(np.max(times_n) - np.min(times_n))*5e-2
if times_n[0]-times_n[-1]<=1.0:
    eps_x=eps_y
for i in zip(tis, tfs):
    if n % 3 == 0:
        plt.fill_betweenx(
            [np.min(s_exp_n) - eps_y, np.max(np.append(s_exp_n,[n_exp_n[1]],axis=0)) + eps_y],
            i[0],
            i[1],
            color="palegreen",
            alpha=0.5 ** (int(n / 3) + 1),
            label=f"Laser ON #{int(n/3)+1}",
        )
    else:
        if (n - 1) % 3 == 0:
            plt.fill_betweenx(
                [np.min(s_exp_n) - eps_y, np.max(np.append(s_exp_n,[n_exp_n[1]],axis=0)) + eps_y],
                i[0],
                i[1],
                color="lightblue",
                alpha=0.5 ** (int(n / 3) + 1),
                label=f"MS depl #{int(n/3)+1}",
            )
        else:
            plt.fill_betweenx(
                [np.min(s_exp_n) - eps_y, np.max(np.append(s_exp_n,[n_exp_n[1]],axis=0)) + eps_y],
                i[0],
                i[1],
                color="orchid",
                alpha=0.5 ** (int(n / 3) + 1),
                label=f"MW ON #{int(n/3)+1}",
            )
    n += 1
plt.ylabel(f"Population")
plt.xlabel(r"Time($\mu$s)")

# set the time limits you want to show in the plot
t_lim = (times_n[0]-eps_x, times_n[-1] + eps_x)
plt.xlim(t_lim)
plt.ylim((np.min(s_exp_n) - eps_y, np.max(np.append(s_exp_n,[n_exp_n[1]],axis=0)) + eps_y))
plt.minorticks_on()

plt.legend(ncol=2,bbox_to_anchor=(1.05, 1), loc="upper left",fontsize=16)
plt.show()

#### PL

In [ ]:
c = 0
print("Starting calculations, this may take a while...")
for dt_mw in tqdm([0.0,0.5*np.pi/Om_r,np.pi/Om_r],position=0):# Initial state where the ground state is evenly populated
    init_state = n1&IdN15/2 
    if dt_mw==0.5*np.pi/Om_r:
        init_state = ((n1 + n2 + ground[1]*ground[2].dag() + ground[2]*ground[1].dag())/2)&IdN15/2 #type:ignore
    if dt_mw==np.pi/Om_r:   
        init_state = n2&IdN15/2
    
    # Array to collect the result Class
    results_t = np.array([])
    
    # Initial time of the evolution
    ti = 0.0
    
    # Store the times of activation and deativation of laser and microwaves for region highlight in plotting
    tis_t = []
    tfs_t = []
    
    # Set the number of times to repeat the Sequence
    n = 1
    # Set the time intervals for each part of the evolution (with laser, with microwaves and free evolution)
    dt_laser = 10.0
    dt_free = 1.0
    # This loop is defined to reproduce the Sequence of Magalleti el al
    for i in tqdm(range(n),position=1,leave=False):
        tis_t.append(ti)
        # Here Laser is ON
        # Run the dynamics based on the mode choosen
        tf, result = dynamics_doh_hf(
            dt_laser,
            init_state,
            ti=ti,
            mode="Laser",
            progress_bar="ON",
            i=i,
        ) # type:ignore
        # Store the Result class for when the Laser is on
        results_t = np.append(results_t, result) # type: ignore 
        # Save the initial state for the next part of the Sequence
        init_state = result.states[-1]
        # Make the initial time the end time of the previous part of the procotol
        ti = tf
        tfs_t.append(tf)
    times_t_pure = np.concatenate([res.times for res in results_t])
    N = [n4&nit[0]*nit[0].dag(),n4&nit[1]*nit[1].dag(), #type: ignore
            n5&nit[0]*nit[0].dag(),n5&nit[1]*nit[1].dag(), #type: ignore
            n6&nit[0]*nit[0].dag(),n6&nit[1]*nit[1].dag(), #type: ignore
            n4&IdN15,
            n5&IdN15,
            n6&IdN15]
    n_exp_t = np.array([np.concatenate([qt.expect(M, res.states) for res in results_t]) for M in N])
    ## Gather the expectation values from the results array
    #for j in range(len(results_t[0].expect)):
    #    nn_exp = np.array([])
    #    for i in range(len(results_t)):
    #        nn_exp = np.append(nn_exp, results_t[i].expect[j])
    #    if j == 0:
    #        n_exp_t = np.array([nn_exp])
    #    else:
    #        n_exp_t = np.append(n_exp_t, [nn_exp], axis=0)
    # Gather the times for the plots
    #times_t_pure = np.array([])
    #for i in range(len(results_t)):
    #    times_t_pure = np.append(times_t_pure, results_t[i].times)
    pl=np.array([])
    pl=np.add(n_exp_t[0],n_exp_t[1])
    pl=np.add(pl,n_exp_t[2])
    pl=np.add(pl,n_exp_t[3])
    pl=np.add(pl,n_exp_t[4])
    pl=np.add(pl,n_exp_t[5])
    pl1=np.array([])
    pl1=np.add(n_exp_t[6],n_exp_t[7])
    pl1=np.add(pl1,n_exp_t[8])
    if c==0:
        pls1_pure = np.array([pl1])
        pls_pure = np.array([pl])
        c+=1
    else:
        pls_pure = np.append(pls_pure,[pl],axis=0)
        pls1_pure = np.append(pls1_pure,[pl1],axis=0)
print(f"Calculations are finished!! See the plots below")
# Set the colors of each state
colors = [
    "dodgerblue",
    "chocolate",
    "gold",
    "mediumpurple",
    "mediumseagreen",
    "lightskyblue",
    "magenta",
    "mediumvioletred",
    "forestgreen",
]
# Defines figure size
plt.figure(figsize=(14, 8))
a=0
state=[r'|0\rangle',r'\frac{|0\rangle+|1\rangle}{\sqrt{2}}',r'|1\rangle']
for pl in pls_pure:
    #plt.plot(times_t_pure,pl, color=colors[a], label=f"PL no trace init state=${state[a]}$")
    a+=1
for pl in pls1_pure:    
    plt.plot(times_t_pure,pl, color=colors[a-3],ls="--",label=f"PL init state=${state[a-3]}$")
    a+=1

eps_y = abs(np.min(pls1_pure) - np.max(pls1_pure))*5e-2
eps_x = abs(np.min(times_t_pure) - np.max(times_t_pure))*5e-2
plt.ylabel(f"PL(a.u.)",fontsize=20)
plt.xlabel(r"Time($\mu$s)",fontsize=20)

# set the time limits you want to show in the plot
t_lim = (times_t_pure[0]-eps_x, times_t_pure[-1] + eps_x)
plt.xlim(t_lim)
plt.ylim((np.min(pls1_pure) - eps_y, np.max(pls1_pure) + eps_y))
plt.minorticks_on()

plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left",fontsize=16)
plt.show()

In [ ]:
c = 0
print("Starting calculations, this may take a while...")
for dt_mw in tqdm([0.0,0.5*np.pi/Om_r,np.pi/Om_r],position=0):# Initial state where the ground state is evenly populated
    init_state = ((n1+n2+n3)&IdN15)/6
    
    # Array to collect the result Class
    results_t = np.array([])
    
    # Initial time of the evolution
    ti = 0.0
    
    # Store the times of activation and deativation of laser and microwaves for region highlight in plotting
    tis_t = []
    tfs_t = []
    
    # Set the number of times to repeat the Sequence
    n = 1
    # Set the time intervals for each part of the evolution (with laser, with microwaves and free evolution)
    dt_laser = 10.0
    dt_free = 1.0
    # This loop is defined to reproduce the Sequence of Magalleti el al
    for i in tqdm(range(n),position=1,leave=False):
        tis_t.append(ti)
        # Here Laser is ON
        # Run the dynamics based on the mode choosen
        tf, result = dynamics_doh_hf(
            dt_laser,
            init_state,
            #exp_ops=[n4&nit[0]*nit[0].dag(),n4&nit[1]*nit[1].dag(), #type: ignore
            #         n5&nit[0]*nit[0].dag(),n5&nit[1]*nit[1].dag(), #type: ignore
            #         n6&nit[0]*nit[0].dag(),n6&nit[1]*nit[1].dag(), #type: ignore
            #         n4&IdN15,
            #         n5&IdN15,
            #         n6&IdN15],
            ti=ti,
            mode="Laser",
            progress_bar="ON",
            i=i,
        ) # type:ignore
        # Store the Result class for when the Laser is on
        results_t = np.append(results_t, result) # type: ignore 
        # Save the initial state for the next part of the Sequence
        init_state = result.states[-1]
        # Make the initial time the end time of the previous part of the procotol
        ti = tf
        tfs_t.append(tf)
        # Run the dynamics based on the mode choosen
        tf, result = dynamics_doh_hf(
            dt_free,
            init_state,
            #exp_ops=[n4&nit[0]*nit[0].dag(),n4&nit[1]*nit[1].dag(), #type: ignore
            #         n5&nit[0]*nit[0].dag(),n5&nit[1]*nit[1].dag(), #type: ignore
            #         n6&nit[0]*nit[0].dag(),n6&nit[1]*nit[1].dag(), #type: ignore
            #         n4&IdN15,
            #         n5&IdN15,
            #         n6&IdN15],
            ti=ti,
            mode="Free",
            progress_bar="ON",
            i=i,
        ) # type:ignore
        # Store the Result class for MW on evolution
        results_t = np.append(results_t, result) # type: ignore 
        # Save the times for coloring areas of the plot
        tis_t.append(ti)
        tfs_t.append(tf)
        # Make initial time to be the end time of the last part of the Sequence
        ti = tf
        # Make the last state  of this evolution step be the initial states of the next iteration
        init_state = result.states[-1]
        # Run the dynamics based on the mode choosen
        tf, result = dynamics_doh_hf(dt_mw, 
                              init_state,
                              #exp_ops=[n4&nit[0]*nit[0].dag(),n4&nit[1]*nit[1].dag(), #type: ignore
                              #          n5&nit[0]*nit[0].dag(),n5&nit[1]*nit[1].dag(), #type: ignore
                              #          n6&nit[0]*nit[0].dag(),n6&nit[1]*nit[1].dag(), #type: ignore
                              #          n4&IdN15,
                              #          n5&IdN15,
                              #          n6&IdN15], 
                              ti=ti, 
                              mode="MW", 
                              progress_bar="ON", 
                              i=i) # type:ignore
        # Store the Result class for MW on evolution
        results_t = np.append(results_t, result) # type: ignore
        # Save the times for coloring areas of the plot
        tis_t.append(ti)
        tfs_t.append(tf)
        # Make initial time to be the end time of the last part of the Sequence
        ti = tf
        # Make the last state  of this evolution step be the initial states of the next iteration
        init_state = result.states[-1]
        tis_t.append(ti)
        # Here Laser is ON
        # Run the dynamics based on the mode choosen
        tf, result = dynamics_doh_hf(
            dt_laser,
            init_state,
            #exp_ops=[n4&nit[0]*nit[0].dag(),n4&nit[1]*nit[1].dag(), #type: ignore
            #         n5&nit[0]*nit[0].dag(),n5&nit[1]*nit[1].dag(), #type: ignore
            #         n6&nit[0]*nit[0].dag(),n6&nit[1]*nit[1].dag(), #type: ignore
            #         n4&IdN15,
            #         n5&IdN15,
            #         n6&IdN15],
            ti=ti,
            mode="Laser",
            progress_bar="ON",
            i=i+1,
        ) # type:ignore
        # Store the Result class for when the Laser is on
        results_t = np.append(results_t, result) # type: ignore 
        # Save the initial state for the next part of the Sequence
        init_state = result.states[-1]
        # Make the initial time the end time of the previous part of the procotol
        ti = tf
        tfs_t.append(tf)
    times_t = np.concatenate([res.times for res in results_t])
    N = [n4&nit[0]*nit[0].dag(),n4&nit[1]*nit[1].dag(), #type: ignore
            n5&nit[0]*nit[0].dag(),n5&nit[1]*nit[1].dag(), #type: ignore
            n6&nit[0]*nit[0].dag(),n6&nit[1]*nit[1].dag(), #type: ignore
            n4&IdN15,
            n5&IdN15,
            n6&IdN15]
    n_exp_t = np.array([np.concatenate([qt.expect(M, res.states) for res in results_t]) for M in N])
    # Gather the expectation values from the results array
    #for j in range(len(results_t[0].expect)):
    #    nn_exp = np.array([])
    #    for i in range(len(results_t)):
    #        nn_exp = np.append(nn_exp, results_t[i].expect[j])
    #    if j == 0:
    #        n_exp_t = np.array([nn_exp])
    #    else:
    #        n_exp_t = np.append(n_exp_t, [nn_exp], axis=0)
    ## Gather the times for the plots
    #times_t = np.array([])
    #for i in range(len(results_t)):
    #    times_t = np.append(times_t, results_t[i].times)
    pl=np.array([])
    pl=np.add(n_exp_t[0],n_exp_t[1])
    pl=np.add(pl,n_exp_t[2])
    pl=np.add(pl,n_exp_t[3])
    pl=np.add(pl,n_exp_t[4])
    pl=np.add(pl,n_exp_t[5])
    pl1=np.array([])
    pl1=np.add(n_exp_t[6],n_exp_t[7])
    pl1=np.add(pl1,n_exp_t[8])
    if c==0:
        pls1 = np.array([pl1])
        pls = np.array([pl])
        c+=1
    else:
        pls = np.append(pls,[pl],axis=0)
        pls1 = np.append(pls1,[pl1],axis=0)
print(f"Calculations are finished!! See the plots below")
# Set the colors of each state
colors = [
    "dodgerblue",
    "chocolate",
    "gold",
    "mediumpurple",
    "mediumseagreen",
    "lightskyblue",
    "magenta",
    "mediumvioletred",
    "forestgreen",
]
# Defines figure size
plt.figure(figsize=(14, 8))
a=0
state=[r'|0\rangle',r'\frac{|0\rangle+|1\rangle}{\sqrt{2}}',r'|1\rangle']
for pl in pls:
    #plt.plot(times_t[-5200:]-11,pl[-5200:], color=colors[a], ls='--', label=f"PL no trace init state=${state[a]}$")
    a+=1
for pl in pls1:    
    plt.plot(times_t[-5200:]-11,pl[-5200:], color=colors[a-3], lw=4,label=f"PL init state=${state[a-3]}$")
    a+=1

eps_y = abs(np.min(pls1) - np.max(pls1))*5e-2
eps_x = abs(np.min(times_t) - np.max(times_t))*5e-2
plt.ylabel(f"PL(a.u.)",fontsize=20)
plt.xlabel(r"Time($\mu$s)",fontsize=20)

# set the time limits you want to show in the plot
t_lim = (times_t[-5200]-eps_x-11, times_t[-1] + eps_x-11)
plt.xlim(t_lim)
plt.ylim((np.min(pls1) - eps_y, np.max(pls1) + eps_y))
plt.minorticks_on()

plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left",fontsize=16)
plt.show()

In [ ]:
plt.figure(figsize=(14, 8))
a=0
state=[r'|0\rangle',r'\frac{|0\rangle+|1\rangle}{\sqrt{2}}',r'|1\rangle']
for pl in pls1_pure:
    plt.plot(times_t_pure,pl, ls='--', lw=4, color=colors[a], label=f"init Pure state=${state[a]}$")
    a+=1
for pl in pls1:
    plt.plot(times_t[-5200:]-11.2,pl[-5200:], lw=4, color=colors[a-3], label=f"init state=${state[a-3]}$")
    a+=1
eps_y = abs(np.min(pls1_pure) - np.max(pls1_pure))*5e-2
eps_x = abs(np.min(times_t[-5200:]) - np.max(times_t[-5200:]))*5e-2
plt.ylabel(f"PL(a.u.)",fontsize=20)
plt.xlabel(r"Time($\mu$s)",fontsize=20)

# set the time limits you want to show in the plot
t_lim = (times_t_pure[0]-eps_x, times_t_pure[-1] + eps_x)
plt.xlim(t_lim)
plt.ylim((np.min(pls1_pure) - eps_y, np.max(pls1_pure) + eps_y))
plt.minorticks_on()

plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left",fontsize=16)
plt.show()

#### Plot functions for Cont vs HF

In [ ]:
def plot_cont_hf_vs_cont(n_exp_hf,times_hf,tis_hf,tfs_hf,n_exp,times,tis,tfs):
    # Set the colors of each state
    colors = [
        "dodgerblue",
        "chocolate",
        "gold",
        "mediumpurple",
        "mediumseagreen",
        "lightskyblue",
        "magenta",
        "forestgreen",
    ]
    hf="{hf}"
    # Defines figure size
    plt.figure(figsize=(14, 8))
    # Plot the population of each state
    plt.plot(times, n_exp[-1], label="$n_c$", color=colors[-1],ls='--')
    plt.plot(times_hf, n_exp_hf[-1], label="$n^{hf}_c$", color=colors[-1])
    for i in range(len(n_exp_hf) - 1):
        plt.plot(times_hf, n_exp_hf[i], label=f"$n^{hf}_{i+1}$", color=colors[i])
    for i in range(len(n_exp) - 1):
        plt.plot(times, n_exp[i], label=f"$n_{i+1}$", color=colors[i],ls='--')
    # Highlighting the times where the laser and microwaves are on and the metastable state depletion evolutions
    n = 0
    eps_y = abs(np.max(np.append(n_exp,n_exp_hf)) - np.min(np.append(n_exp,n_exp_hf)))*5e-2
    eps_x = abs(np.max(np.append(times,times_hf)) - np.min(np.append(times,times_hf)))*5e-2
    for i in zip(tis, tfs):
        if n % 3 == 0:
            plt.fill_betweenx(
                [np.min(np.append(n_exp,n_exp_hf)) - eps_y, np.max(np.append(n_exp,n_exp_hf)) + eps_y],
                i[0],
                i[1],
                color="palegreen",
                alpha=0.5 ** (int(n / 3) + 1),
                label=f"Laser ON #{int(n/3)+1}",
            )
            t_laser=i[1]-i[0]
        else:
            if (n - 1) % 3 == 0:
                plt.fill_betweenx(
                    [np.min(np.append(n_exp,n_exp_hf)) - eps_y, np.max(np.append(n_exp,n_exp_hf)) + eps_y],
                    i[0],
                    i[1],
                    color="lightblue",
                    alpha=0.5 ** (int(n / 3) + 1),
                    label=f"MS depl #{int(n/3)+1}",
                )
                t_free=i[1]-i[0]
            else:
                plt.fill_betweenx(
                    [np.min(np.append(n_exp,n_exp_hf)) - eps_y, np.max(np.append(n_exp,n_exp_hf)) + eps_y],
                    i[0],
                    i[1],
                    color="orchid",
                    alpha=0.5 ** (int(n / 3) + 1),
                    label=f"MW ON #{int(n/3)+1}",
                )
                t_mw=i[1]-i[0]
        n += 1
    plt.ylabel(f"Population",fontsize=20)
    plt.xlabel(r"Time($\mu$s)",fontsize=20)
    plt.title(f"Comparisson between Hyperfine and No-HF evolution\n Laser:{t_laser:.2f} $\\mu$s, MW:{t_mw:.2f} $\\mu$s, Free:{t_free:.2f} $\\mu$s",fontsize=20)
    # set the time limits you want to show in the plot
    t_lim = (np.min(np.append(times,times_hf))-eps_x, np.max(np.append(times,times_hf)) + eps_x)
    plt.xlim(t_lim)
    plt.ylim((np.min(np.append(n_exp,n_exp_hf)) - eps_y, np.max(np.append(n_exp,n_exp_hf)) + eps_y))
    plt.minorticks_on()
    
    plt.legend(ncol=2,bbox_to_anchor=(1.05, 1), loc="upper left",fontsize=16)
    plt.show()

In [ ]:
def plot_cont_hf_vs_cont_bomrs(n_exp_hf,times_hf,tis_hf,tfs_hf,n_exp,times,tis,tfs,b,om):
    # Set the colors of each state
    colors = [
        "dodgerblue",
        "chocolate",
        "gold",
        "mediumpurple",
        "mediumseagreen",
        "lightskyblue",
        "magenta",
        "forestgreen",
    ]
    hf="{hf}"
    # Defines figure size
    plt.figure(figsize=(14, 8))
    # Plot the population of each state
    plt.plot(times, n_exp[-1], label="$n_c$", color=colors[-1],ls='--')
    plt.plot(times_hf, n_exp_hf[-1], label="$n^{hf}_c$", color=colors[-1])
    for i in range(len(n_exp_hf) - 1):
        plt.plot(times_hf, n_exp_hf[i], label=f"$n^{hf}_{i+1}$", color=colors[i])
    for i in range(len(n_exp) - 1):
        plt.plot(times, n_exp[i], label=f"$n_{i+1}$", color=colors[i],ls='--')
    # Highlighting the times where the laser and microwaves are on and the metastable state depletion evolutions
    n = 0
    eps_y = abs(np.max(np.append(n_exp,n_exp_hf)) - np.min(np.append(n_exp,n_exp_hf)))*5e-2
    eps_x = abs(np.max(np.append(times,times_hf)) - np.min(np.append(times,times_hf)))*5e-2
    for i in zip(tis, tfs):
        if n % 3 == 0:
            plt.fill_betweenx(
                [np.min(np.append(n_exp,n_exp_hf)) - eps_y, np.max(np.append(n_exp,n_exp_hf)) + eps_y],
                i[0],
                i[1],
                color="palegreen",
                alpha=0.5 ** (int(n / 3) + 1),
                label=f"Laser ON #{int(n/3)+1}",
            )
            t_laser=i[1]-i[0]
        else:
            if (n - 1) % 3 == 0:
                plt.fill_betweenx(
                    [np.min(np.append(n_exp,n_exp_hf)) - eps_y, np.max(np.append(n_exp,n_exp_hf)) + eps_y],
                    i[0],
                    i[1],
                    color="lightblue",
                    alpha=0.5 ** (int(n / 3) + 1),
                    label=f"MS depl #{int(n/3)+1}",
                )
                t_free=i[1]-i[0]
            else:
                plt.fill_betweenx(
                    [np.min(np.append(n_exp,n_exp_hf)) - eps_y, np.max(np.append(n_exp,n_exp_hf)) + eps_y],
                    i[0],
                    i[1],
                    color="orchid",
                    alpha=0.5 ** (int(n / 3) + 1),
                    label=f"MW ON #{int(n/3)+1}",
                )
                t_mw=i[1]-i[0]
        n += 1
    plt.ylabel(f"Population",fontsize=20)
    plt.xlabel(r"Time($\mu$s)",fontsize=20)
    plt.title(f"Comparisson between Hyperfine and No-HF evolution\n B={b:.2f} G, $\\Omega_r$={om:.2f} MHz",fontsize=20)
    # set the time limits you want to show in the plot
    t_lim = (np.min(np.append(times,times_hf))-eps_x, np.max(np.append(times,times_hf)) + eps_x)
    plt.xlim(t_lim)
    plt.ylim((np.min(np.append(n_exp,n_exp_hf)) - eps_y, np.max(np.append(n_exp,n_exp_hf)) + eps_y))
    plt.minorticks_on()
    
    plt.legend(ncol=2,bbox_to_anchor=(1.05, 1), loc="upper left",fontsize=16)
    plt.show()

In [ ]:
def plot_cont_hf_vs_cont_bangs(n_exp_hf,times_hf,tis_hf,tfs_hf,n_exp,times,tis,tfs,b,p,t,om):
    # Set the colors of each state
    colors = [
        "dodgerblue",
        "chocolate",
        "gold",
        "mediumpurple",
        "mediumseagreen",
        "lightskyblue",
        "magenta",
        "forestgreen",
    ]
    hf="{hf}"
    # Defines figure size
    plt.figure(figsize=(14, 8))
    # Plot the population of each state
    plt.plot(times, n_exp[-1], label="$n_c$", color=colors[-1],ls='--')
    plt.plot(times_hf, n_exp_hf[-1], label="$n^{hf}_c$", color=colors[-1])
    for i in range(len(n_exp_hf) - 1):
        plt.plot(times_hf, n_exp_hf[i], label=f"$n^{hf}_{i+1}$", color=colors[i])
    for i in range(len(n_exp) - 1):
        plt.plot(times, n_exp[i], label=f"$n_{i+1}$", color=colors[i],ls='--')
    # Highlighting the times where the laser and microwaves are on and the metastable state depletion evolutions
    n = 0
    eps_y = abs(np.max(np.append(n_exp,n_exp_hf)) - np.min(np.append(n_exp,n_exp_hf)))*5e-2
    eps_x = abs(np.max(np.append(times,times_hf)) - np.min(np.append(times,times_hf)))*5e-2
    for i in zip(tis, tfs):
        if n % 3 == 0:
            plt.fill_betweenx(
                [np.min(np.append(n_exp,n_exp_hf)) - eps_y, np.max(np.append(n_exp,n_exp_hf)) + eps_y],
                i[0],
                i[1],
                color="palegreen",
                alpha=0.5 ** (int(n / 3) + 1),
                label=f"Laser ON #{int(n/3)+1}",
            )
            t_laser=i[1]-i[0]
        else:
            if (n - 1) % 3 == 0:
                plt.fill_betweenx(
                    [np.min(np.append(n_exp,n_exp_hf)) - eps_y, np.max(np.append(n_exp,n_exp_hf)) + eps_y],
                    i[0],
                    i[1],
                    color="lightblue",
                    alpha=0.5 ** (int(n / 3) + 1),
                    label=f"MS depl #{int(n/3)+1}",
                )
                t_free=i[1]-i[0]
            else:
                plt.fill_betweenx(
                    [np.min(np.append(n_exp,n_exp_hf)) - eps_y, np.max(np.append(n_exp,n_exp_hf)) + eps_y],
                    i[0],
                    i[1],
                    color="orchid",
                    alpha=0.5 ** (int(n / 3) + 1),
                    label=f"MW ON #{int(n/3)+1}",
                )
                t_mw=i[1]-i[0]
        n += 1
    plt.ylabel(f"Population",fontsize=20)
    plt.xlabel(r"Time($\mu$s)",fontsize=20)
    plt.title(
        f"Comparisson between Hyperfine and No-HF evolution\n B={b:.2f} G, $\\theta_B={Fraction(t/np.pi).limit_denominator()}\\pi$,$\\phi_B={Fraction(p/np.pi).limit_denominator()}\\pi$, $\\omega={om:.2f} MHz$",fontsize=20)
    # set the time limits you want to show in the plot
    t_lim = (np.min(np.append(times,times_hf))-eps_x, np.max(np.append(times,times_hf)) + eps_x)
    plt.xlim(t_lim)
    plt.ylim((np.min(np.append(n_exp,n_exp_hf)) - eps_y, np.max(np.append(n_exp,n_exp_hf)) + eps_y))
    plt.minorticks_on()
    
    plt.legend(ncol=2,bbox_to_anchor=(1.05, 1), loc="upper left",fontsize=16)
    plt.show()

#### Plot Cont vs HF Magaletti Sequence

In [ ]:
plot_cont_hf_vs_cont(n_exp_hf,times_hf,tis_hf,tfs_hf,n_exp,times,tis,tfs)

### PLs

In [ ]:
def photolumin_zfs_hf(dyn,model,laser_time, free_time,b=B, om_r=Om_r, om=omega, w_p=W_p,progress_bar="ON"):
    """
    Simulates the evolution of a quantum system under a given Sequence with continuos laser.

    Args:
        laser_time (float): The duration of the laser pulse in each iteration.
        mw_time (float): The duration of the microwave pulse in each iteration.
        free_time (float): The duration of the free evolution in each iteration.
        sequence_repeats (int, optional): The number of times to repeat the Sequence. Defaults to 1.

    Returns:
        tuple: A tuple containing the following arrays:
            - n_exp (numpy.ndarray): An array of expectation values for different operators.
            - times (numpy.ndarray): An array of time values.
            - tis (numpy.ndarray): An array of initial times for each part of the Sequence.
            - tfs (numpy.ndarray): An array of final times for each part of the Sequence.
            - results (numpy.ndarray): An array of the Result class object for the time evolution.
    """
    c = 0
    print("Starting calculations, this may take a while...")
    for dt_mw in tqdm([0.0,0.5*np.pi/om_r,np.pi/om_r]):
        # Initial state where the ground state is evenly populated
        init_state = ((n1+n2+n3)&IdN15)/6

        # Array to collect the result Class
        results_t = np.array([])

        # Initial time of the evolution
        ti = 0.0

        # Store the times of activation and deativation of laser and microwaves for region highlight in plotting
        tis_t = []
        tfs_t = []

        # Set the number of times to repeat the Sequence
        n = 1
        # Set the time intervals for each part of the evolution (with laser, with microwaves and free evolution)
        dt_laser = laser_time
        dt_free = free_time
        # This loop is defined to reproduce the Sequence of Magalleti el al
        for i in tqdm(range(n),position=1,leave=False):
            tis_t.append(ti)
            # Here Laser is ON
            # Run the model based on the mode choosen
            tf, result = model(
                dt_laser,
                init_state,
                b=b,
                om_r=om_r,
                om=om,
                w_p=w_p,
                #exp_ops=[n4&nit[0]*nit[0].dag(),n4&nit[1]*nit[1].dag(), #type: ignore
                #         n5&nit[0]*nit[0].dag(),n5&nit[1]*nit[1].dag(), #type: ignore
                #         n6&nit[0]*nit[0].dag(),n6&nit[1]*nit[1].dag(), #type: ignore
                #         n4&IdN15,
                #         n5&IdN15,
                #         n6&IdN15],
                ti=ti,
                mode="Laser",
                progress_bar=progress_bar,
                i=i,
            ) # type:ignore
            # Store the Result class for when the Laser is on
            results_t = np.append(results_t, result) # type: ignore 
            # Save the initial state for the next part of the Sequence
            init_state = result.states[-1]
            # Make the initial time the end time of the previous part of the procotol
            ti = tf
            tfs_t.append(tf)
            # Run the dynamics based on the mode choosen
            tf, result = model(
                dt_free,
                init_state,
                b=b,
                om_r=om_r,
                om=om,
                w_p=w_p,
                #exp_ops=[n4&nit[0]*nit[0].dag(),n4&nit[1]*nit[1].dag(), #type: ignore
                #         n5&nit[0]*nit[0].dag(),n5&nit[1]*nit[1].dag(), #type: ignore
                #         n6&nit[0]*nit[0].dag(),n6&nit[1]*nit[1].dag(), #type: ignore
                #         n4&IdN15,
                #         n5&IdN15,
                #         n6&IdN15],
                ti=ti,
                mode="Free",
                progress_bar=progress_bar,
                i=i,
            ) # type:ignore
            # Store the Result class for MW on evolution
            results_t = np.append(results_t, result) # type: ignore 
            # Save the times for coloring areas of the plot
            tis_t.append(ti)
            tfs_t.append(tf)
            # Make initial time to be the end time of the last part of the Sequence
            ti = tf
            # Make the last state  of this evolution step be the initial states of the next iteration
            init_state = result.states[-1]
            # Run the dynamics based on the mode choosen
            tf, result = model(dt_mw, 
                                  init_state,
                                  b=b,
                                  om_r=om_r,
                                  om=om,
                                  w_p=w_p,
                                  #exp_ops=[n4&nit[0]*nit[0].dag(),n4&nit[1]*nit[1].dag(), #type: ignore
                                  #          n5&nit[0]*nit[0].dag(),n5&nit[1]*nit[1].dag(), #type: ignore
                                  #          n6&nit[0]*nit[0].dag(),n6&nit[1]*nit[1].dag(), #type: ignore
                                  #          n4&IdN15,
                                  #          n5&IdN15,
                                  #          n6&IdN15], 
                                  ti=ti, 
                                  mode="MW", 
                                  progress_bar=progress_bar, 
                                  i=i) # type:ignore
            # Store the Result class for MW on evolution
            results_t = np.append(results_t, result) # type: ignore
            # Save the times for coloring areas of the plot
            tis_t.append(ti)
            tfs_t.append(tf)
            # Make initial time to be the end time of the last part of the Sequence
            ti = tf
            # Make the last state  of this evolution step be the initial states of the next iteration
            init_state = result.states[-1]
            tis_t.append(ti)
            # Here Laser is ON
            # Run the dynamics based on the mode choosen
            tf, result = model(
                dt_laser,
                init_state,
                b=b,
                om_r=om_r,
                om=om,
                w_p=w_p,
                #exp_ops=[n4&nit[0]*nit[0].dag(),n4&nit[1]*nit[1].dag(), #type: ignore
                #         n5&nit[0]*nit[0].dag(),n5&nit[1]*nit[1].dag(), #type: ignore
                #         n6&nit[0]*nit[0].dag(),n6&nit[1]*nit[1].dag(), #type: ignore
                #         n4&IdN15,
                #         n5&IdN15,
                #         n6&IdN15],
                ti=ti,
                mode="Laser",
                progress_bar=progress_bar,
                i=i+1,
            ) # type:ignore
            # Store the Result class for when the Laser is on
            results_t = np.append(results_t, result) # type: ignore 
            # Save the initial state for the next part of the Sequence
            init_state = result.states[-1]
            # Make the initial time the end time of the previous part of the procotol
            ti = tf
            tfs_t.append(tf)
        times_t = np.concatenate([res.times for res in results_t])
        N = [n4&nit[0]*nit[0].dag(),n4&nit[1]*nit[1].dag(), #type: ignore
                n5&nit[0]*nit[0].dag(),n5&nit[1]*nit[1].dag(), #type: ignore
                n6&nit[0]*nit[0].dag(),n6&nit[1]*nit[1].dag(), #type: ignore
                n4&IdN15,
                n5&IdN15,
                n6&IdN15]
        n_exp_t = np.array([np.concatenate([qt.expect(M, res.states) for res in results_t]) for M in N])
        ## Gather the expectation values from the results array
        #for j in range(len(results_t[0].expect)):
        #    nn_exp = np.array([])
        #    for i in range(len(results_t)):
        #        nn_exp = np.append(nn_exp, results_t[i].expect[j])
        #    if j == 0:
        #        n_exp_t = np.array([nn_exp])
        #    else:
        #        n_exp_t = np.append(n_exp_t, [nn_exp], axis=0)
        ## Gather the times for the plots
        #times_t = np.array([])
        #for i in range(len(results_t)):
        #    times_t = np.append(times_t, results_t[i].times)
        pl=np.array([])
        pl=np.add(n_exp_t[0],n_exp_t[1])
        pl=np.add(pl,n_exp_t[2])
        pl=np.add(pl,n_exp_t[3])
        pl=np.add(pl,n_exp_t[4])
        pl=np.add(pl,n_exp_t[5])
        pl1=np.array([])
        pl1=np.add(n_exp_t[6],n_exp_t[7])
        pl1=np.add(pl1,n_exp_t[8])
        if c==0:
            res=np.array([results_t])
            pls1 = np.array([pl1])
            pls = np.array([pl])
            c+=1
        else:
            res=np.append(res,[results_t],axis=0)
            pls = np.append(pls,[pl],axis=0)
            pls1 = np.append(pls1,[pl1],axis=0)
    print(f"Calculations are finished!! See the plots below")
    # Instead of calling plt.show(), create and return a Figure.
    fig, ax = plt.subplots(figsize=(14, 8))
    # Set the colors of each state
    colors = [
        "dodgerblue",
        "chocolate",
        "gold",
        "mediumpurple",
        "mediumseagreen",
        "lightskyblue",
        "magenta",
        "mediumvioletred",
        "forestgreen",
    ]
    # Defines figure size
    #plt.figure(figsize=(14, 8))
    a=0
    state=[r'|0\rangle',r'\frac{|0\rangle+|1\rangle}{\sqrt{2}}',r'|1\rangle']
    for pl in pls:
        #ax.plot(times_t[-5200:]-11,pl[-5200:], color=colors[a], label=f"PL no trace init state=${state[a]}$")
        a+=1
    for pl in pls1:    
        ax.plot(times_t[-5200:]-11,pl[-5200:], color=colors[a-3],ls="--", lw=4, label=f"PL init state=${state[a-3]}$")
        a+=1
    
    eps_y = abs(np.min(pls1) - np.max(pls1))*5e-2
    eps_x = abs(np.min(times_t) - np.max(times_t))*5e-2
    ax.set_ylabel(f"PL(a.u.)",fontsize=20)
    ax.set_xlabel(r"Time($\mu$s)",fontsize=20)
    ax.set_title(f"{dyn} \n B={b[2]:.2f} G, $\\Omega_r$={om_r:.2f} MHz, $W_p$={w_p:.2f} ",fontsize=20)

    # set the time limits you want to show in the plot
    t_lim = (times_t[-5200]-eps_x-11, times_t[-1] + eps_x-11)
    ax.set_xlim(t_lim)
    ax.set_ylim((np.min(pls1) - eps_y, np.max(pls1) + eps_y))
    ax.minorticks_on()
    ax.tick_params(axis='both', labelsize=14)
    ax.legend(bbox_to_anchor=(1.05, 1), loc="upper left",fontsize=16)
    return fig, res

In [ ]:
def run_photolumin_param(params):
    # params is a tuple: (dynamics, b, omr)
    dyn,dynamics, b, omr = params
    b0 = B0(b, 0.0, 0.0)
    # Here om is set to D_gs - mu_e*b as in your original loop.
    fig, res = photolumin_zfs_hf(dyn,dynamics, 10.0, 1.0, b=b0, om_r=omr, om=D_gs-mu_e*b,
                             progress_bar="OFF")
    return (dyn, b, omr, fig, res)

# Define the parameter arrays.
om_r_pi = np.pi/t2_gs
oms_r = np.array([om_r_pi, 2*om_r_pi, 10*om_r_pi])
bs = np.array([0, 100, 500, 1000])

# Build a list of (dynamics, b, omr) tuples.
params1 = []
for b in bs:
    for omr in oms_r:
        # Run for both dynamics functions.
        params1.append(("Doherty",dynamics_doh_hf, b, omr))
        params1.append(("Duarte",dynamics_dua_hf, b, omr))
# Run in parallel (num_cpus defaults to qutip.settings.num_cpus, 8 in your case).
if run_pls_bomr:
    i=0
    results1 = qt.loky_pmap(run_photolumin_param, params1, progress_bar="tqdm",map_kw={'fail_fast':True})
    for dyn, b, omr, fig, res in results1: #type:ignore
        print(f"Displaying {dyn} figure for b={b} and om_r={omr}")
        display(fig)  # or plt.show() if you wish to display interactively;
        if i == 0:
            result_pls_bomr=np.array([dyn,b,omr,res],dtype=object)
            i+=1
        else:
            result_pls_bomr=np.append(result_pls_bomr,np.array([dyn,b,omr,res],dtype=object),axis=0)
    np.save("NumpyArrays\\results_pl_bomr.npy",result_pls_bomr)
else:
    results_pls_bomr=np.load("NumpyArrays\\results_pl_bomr.npy",allow_pickle=True)   
# In the main process, iterate over the results and display (or save) the figures.


In [ ]:
#om_r_pi=np.pi/t2_gs
#oms_r=np.array([om_r_pi,2*om_r_pi,10*om_r_pi])
#bs=np.array([0,100,500,1000])
#for b in tqdm(bs,desc='Magnectic Field',position=0):
#    b0=B0(b,0.0,0.0)
#    for omr in tqdm(oms_r,desc='Rabi Frequency',position=1):
#        photolumin_zfs_hf(dynamics_doh_hf,10.0,1.0,b=b0,om_r=omr,om=D_gs-mu_e*b,progress_bar="OFF")
#        photolumin_zfs_hf(dynamics_dua_hf,10.0,1.0,b=b0,om_r=omr,om=D_gs-mu_e*b,progress_bar="OFF")

In [ ]:
def run_photolumin_wp(params):
    # params is a tuple: (dynamics, w_p)
    dyn, dynamics, w = params
    fig,res = photolumin_zfs_hf(dyn,dynamics, 10.0, 1.0, w_p=w, progress_bar="OFF")
    return (dyn, w, fig, res)

wps = np.linspace(0.5, 4, 8)
params2 = []
for w in wps:
    params2.append(('Doherty',dynamics_doh_hf, w))
    params2.append(('Duarte',dynamics_dua_hf, w))
if run_pls:
    i=0
    results2 = qt.loky_pmap(run_photolumin_wp, params2, progress_bar="tqdm")
    for dyn, w, fig, res in results2: #type:ignore
        print(f"Displaying {dyn} figure for w={w:.2f}")
        display(fig)  # or plt.show() if you wish to display interactively;
        # Alternatively, you can save the figure:
        # fig.savefig(f"plot_b{b}_omr{omr}.png")
        if i == 0:
            result_pls_w=np.array([dyn,w,res],dtype=object)
            i+=1
        else:
            result_pls_w=np.append(result_pls_w,np.array([dyn,w,res],dtype=object),axis=0)
    np.save("NumpyArrays\\results_pl_w.npy",result_pls_w)
else:
    result_pls_w = np.load("NumpyArrays\\results_pl_w.npy",allow_pickle=True)

In [ ]:
#wps=np.linspace(0.5,4,10)
#for w in tqdm(wps,desc='Laser Power',position=0):
#        photolumin_zfs_hf(dynamics_doh_hf,10.0,1.0,w_p=w,progress_bar="OFF")
#        photolumin_zfs_hf(dynamics_dua_hf,10.0,1.0,w_p=w,progress_bar="OFF")

### Magaletti Sequence for 7 and 14 levels

In [ ]:
def continuous_zfs(model,laser_time, mw_time, free_time,b=B, om_r=Om_r, om=omega, w_p=W_p, 
                      sequence_repeats=1,progress_bar="ON"):
    """
    Simulates the evolution of a quantum system under a given Sequence with continuos laser.

    Args:
        laser_time (float): The duration of the laser pulse in each iteration.
        mw_time (float): The duration of the microwave pulse in each iteration.
        free_time (float): The duration of the free evolution in each iteration.
        sequence_repeats (int, optional): The number of times to repeat the Sequence. Defaults to 1.

    Returns:
        tuple: A tuple containing the following arrays:
            - n_exp (numpy.ndarray): An array of expectation values for different operators.
            - times (numpy.ndarray): An array of time values.
            - tis (numpy.ndarray): An array of initial times for each part of the Sequence.
            - tfs (numpy.ndarray): An array of final times for each part of the Sequence.
            - results (numpy.ndarray): An array of the Result class object for the time evolution.
    """
    # Initial state where the ground state is evenly populated
    init_state = (n1+n2+n3)/3

    # Array to collect the result Class
    results = np.array([])

    # Initial time of the evolution
    ti = 0.0

    # Store the times of activation and deativation of laser and microwaves for region highlight in plotting
    tis = []
    tfs = []

    # Set the number of times to repeat the Sequence
    n = sequence_repeats
    # Set the time intervals for each part of the evolution (with laser, with microwaves and free evolution)
    dt_laser = laser_time
    dt_free = free_time
    dt_mw = mw_time
    print("Starting continuous calculations, this may take a while...")
    # This loop is defined to reproduce the same Sequence as a pulsed Sequence, but with continuous laser
    for i in tqdm(range(n)):
        tis.append(ti)
        # Here Laser is ON
        # Run the dynamics based on the mode choosen
        tf, result = model(dt_laser,
                                     init_state,
                                     b=b,
                                     om_r=om_r,
                                     om=om,
                                     w_p=w_p,
                                     ti=ti,
                                     mode="Laser",
                                     progress_bar=progress_bar,
                                     i=i) # type:ignore
        # Store the Result class for when the Laser is on
        results = np.append(results, result) # type: ignore
        # Save the initial state for the next part of the Sequence
        init_state = result.states[-1]
        # Make the initial time the end time of the previous part of the procotol
        ti = tf
        tfs.append(tf)
        # Run the dynamics based on the mode choosen
        tf, result = model(dt_free,
                                     init_state,
                                     b=b,
                                     om_r=om_r,
                                     om=om,
                                     w_p=w_p,
                                     ti=ti,
                                     mode="Free",
                                     progress_bar=progress_bar,
                                     i=i) # type:ignore
        # Store the Result class for MW on evolution
        results = np.append(results, result) # type: ignore
        # Save the times for coloring areas of the plot
        tis.append(ti)
        tfs.append(tf)
        # Make initial time to be the end time of the last part of the Sequence
        ti = tf
        # Make the last state  of this evolution step be the initial states of the next iteration
        init_state = result.states[-1]
        # Run the dynamics based on the mode choosen
        tf, result = model(dt_mw,
                                     init_state,
                                     b=b,
                                     om_r=om_r,
                                     om=om,
                                     w_p=w_p,
                                     ti=ti,
                                     mode="MW",
                                     progress_bar=progress_bar,
                                     i=i) # type:ignore
        # Store the Result class for MW on evolution
        results = np.append(results, result) # type: ignore
        # Save the times for coloring areas of the plot
        tis.append(ti)
        tfs.append(tf)
        # Make initial time to be the end time of the last part of the Sequence
        ti = tf
        # Make the last state  of this evolution step be the initial states of the next iteration
        init_state = result.states[-1]
    times = np.concatenate([res.times for res in results])
    N = [n1, n2, n3,
         n4, n5, n6,
         n7, nc]
    n_exp = np.array([np.concatenate([qt.expect(M, res.states) for res in results]) for M in N]) #type:ignore
    S = [sx_gs, sy_gs, sz_gs,
         sx_es, sy_es, sz_es]
    s_exp = np.array([np.concatenate([qt.expect(M, res.states)  for res in results]) for M in S])#type:ignore
    # Gather the expectation values from the results array
    #for j in range(len(results[0].expect)):
    #    nn_exp = np.array([])
    #    for i in range(len(results)):
    #        nn_exp = np.append(nn_exp, results[i].expect[j])
    #    if j == 0:
    #        n_exp = np.array([nn_exp])
    #    else:
    #        n_exp = np.append(n_exp, [nn_exp], axis=0) # type: ignore
    #if len(n_exp)>8:
    #    s_exp=n_exp[len(n_exp)-8:len(n_exp)-2]
    #    nit_exp=n_exp[len(n_exp)-2:]
    #    n_exp=n_exp[:len(n_exp)-8]
    ## Gather the times for the plots
    #times = np.array([])
    #for i in range(len(results)):
    #    times = np.append(times, results[i].times)
    print("Calculations are finished!! param order(Expect,times,tis,tfs,results,s_exp)")
    return n_exp, times, tis, tfs, results,s_exp

In [ ]:
def continuous_zfs_hf(model,laser_time, mw_time, free_time,b=B, om_r=Om_r, om=omega, w_p=W_p, 
                      sequence_repeats=1,progress_bar="ON"):
    """
    Simulates the evolution of a quantum system under a given Sequence with continuos laser.

    Args:
        laser_time (float): The duration of the laser pulse in each iteration.
        mw_time (float): The duration of the microwave pulse in each iteration.
        free_time (float): The duration of the free evolution in each iteration.
        sequence_repeats (int, optional): The number of times to repeat the Sequence. Defaults to 1.

    Returns:
        tuple: A tuple containing the following arrays:
            - n_exp (numpy.ndarray): An array of expectation values for different operators.
            - times (numpy.ndarray): An array of time values.
            - tis (numpy.ndarray): An array of initial times for each part of the Sequence.
            - tfs (numpy.ndarray): An array of final times for each part of the Sequence.
            - results (numpy.ndarray): An array of the Result class object for the time evolution.
    """
    # Initial state where the ground state is evenly populated
    init_state = ((n1+n2+n3)&IdN15)/6

    # Array to collect the result Class
    results = np.array([]) #type:ignore

    # Initial time of the evolution
    ti = 0.0

    # Store the times of activation and deativation of laser and microwaves for region highlight in plotting
    tis = []
    tfs = []

    # Set the number of times to repeat the Sequence
    n = sequence_repeats
    # Set the time intervals for each part of the evolution (with laser, with microwaves and free evolution)
    dt_laser = laser_time
    dt_free = free_time
    dt_mw = mw_time
    print("Starting continuous calculations, this may take a while...")
    # This loop is defined to reproduce the same Sequence as a pulsed Sequence, but with continuous laser
    for i in tqdm(range(n)):
        tis.append(ti)
        # Here Laser is ON
        # Run the dynamics based on the mode choosen
        tf, result = model(dt_laser,
                                     init_state,
                                     b=b,
                                     om_r=om_r,
                                     om=om,
                                     w_p=w_p,
                                     ti=ti,
                                     mode="Laser",
                                     progress_bar=progress_bar,
                                     i=i) # type:ignore
        # Store the Result class for when the Laser is on
        results = np.append(results, result) # type: ignore
        # Save the initial state for the next part of the Sequence
        init_state = result.states[-1]
        # Make the initial time the end time of the previous part of the procotol
        ti = tf
        tfs.append(tf)
        # Run the dynamics based on the mode choosen
        tf, result = model(dt_free,
                                     init_state,
                                     b=b,
                                     om_r=om_r,
                                     om=om,
                                     w_p=w_p,
                                     ti=ti,
                                     mode="Free",
                                     progress_bar=progress_bar,
                                     i=i) # type:ignore
        # Store the Result class for MW on evolution
        results = np.append(results, result) # type: ignore
        # Save the times for coloring areas of the plot
        tis.append(ti)
        tfs.append(tf)
        # Make initial time to be the end time of the last part of the Sequence
        ti = tf
        # Make the last state  of this evolution step be the initial states of the next iteration
        init_state = result.states[-1]
        # Run the dynamics based on the mode choosen
        tf, result = model(dt_mw,
                                     init_state,
                                     b=b,
                                     om_r=om_r,
                                     om=om,
                                     w_p=w_p,
                                     ti=ti,
                                     mode="MW",
                                     progress_bar=progress_bar,
                                     i=i) # type:ignore
        # Store the Result class for MW on evolution
        results = np.append(results, result) # type: ignore
        # Save the times for coloring areas of the plot
        tis.append(ti)
        tfs.append(tf)
        # Make initial time to be the end time of the last part of the Sequence
        ti = tf
        # Make the last state  of this evolution step be the initial states of the next iteration
        init_state = result.states[-1]
    times = np.concatenate([res.times for res in results])
    N = [n1&IdN15, n2&IdN15, n3&IdN15,
         n4&IdN15, n5&IdN15, n6&IdN15,
         n7&IdN15, nc&IdN15]
    n_exp = np.array([np.concatenate([qt.expect(M, res.states) for res in results]) for M in N]) #type:ignore
    S = [sx_gs&IdN15, sy_gs&IdN15, sz_gs&IdN15,
         sx_es&IdN15, sy_es&IdN15, sz_es&IdN15]
    s_exp = np.array([np.concatenate([qt.expect(M, res.states)  for res in results]) for M in S]) #type:ignore
    Nit = [IdNV&nit[0] * nit[0].dag(), IdNV&nit[1] * nit[1].dag()] #type: ignore
    nit_exp = np.array([np.concatenate([qt.expect(M, res.states)  for res in results]) for M in Nit]) #type:ignore
    ## Gather the expectation values from the results array
    #for j in range(len(results[0].expect)):
    #    nn_exp = np.array([])
    #    for i in range(len(results)):
    #        nn_exp = np.append(nn_exp, results[i].expect[j])
    #    if j == 0:
    #        n_exp = np.array([nn_exp])
    #    else:
    #        n_exp = np.append(n_exp, [nn_exp], axis=0) # type: ignore
    #if len(n_exp)>8:
    #    s_exp=n_exp[len(n_exp)-8:len(n_exp)-2]
    #    nit_exp=n_exp[len(n_exp)-2:]
    #    n_exp=n_exp[:len(n_exp)-8]
    ## Gather the times for the plots
    #times = np.array([])
    #for i in range(len(results)):
    #    times = np.append(times, results[i].times)
    print("Calculations are finished!! param order(Expect,times,tis,tfs,results,nit_exp,s_exp)")
    return n_exp, times, tis, tfs, results, nit_exp,s_exp

#### 14 levels

In [ ]:
# Define the new expectation operators for 14 levels
def continuous_14_zfs_hf(model, laser_time, mw_time, free_time, b=B, om_r=Om_r, om=omega, w_p=W_p,
                      sequence_repeats=1, progress_bar="ON"):
    """
    Simulates the evolution of a quantum system under a given Sequence with continuous laser.

    Args:
        laser_time (float): The duration of the laser pulse in each iteration.
        mw_time (float): The duration of the microwave pulse in each iteration.
        free_time (float): The duration of the free evolution in each iteration.
        sequence_repeats (int, optional): The number of times to repeat the Sequence. Defaults to 1.

    Returns:
        tuple: A tuple containing the following arrays:
            - n_exp (numpy.ndarray): An array of expectation values for different operators.
            - times (numpy.ndarray): An array of time values.
            - tis (numpy.ndarray): An array of initial times for each part of the Sequence.
            - tfs (numpy.ndarray): An array of final times for each part of the Sequence.
            - results (numpy.ndarray): An array of the Result class object for the time evolution.
            - nit_exp (numpy.ndarray): An empty array (place holder for compatibility).
            - s_exp (numpy.ndarray): An empty array (place holder for compatibility).
    """
    # Initial state where the ground state is evenly populated
    init_state = ((n1+n2+n3)&IdN15)/6
    exp_ops_14 = []
    s_ops_14 = []
    for n_i in [n1, n2, n3, n4, n5, n6, n7, nc]:
        for j in range(2):
            nit_proj = nit[j] * nit[j].dag() #type: ignore
            exp_ops_14.append(n_i&nit_proj)
    for s_i in [sx_gs, sy_gs, sz_gs,sx_es, sy_es, sz_es]:
        for j in range(2):
            nit_proj = nit[j] * nit[j].dag() #type: ignore
            s_ops_14.append(s_i&nit_proj)

    # Array to collect the result Class
    results = np.array([])

    # Initial time of the evolution
    ti = 0.0

    # Store the times of activation and deactivation of laser and microwaves for region highlight in plotting
    tis = []
    tfs = []

    # Set the number of times to repeat the Sequence
    n = sequence_repeats
    # Set the time intervals for each part of the evolution (with laser, with microwaves and free evolution)
    dt_laser = laser_time
    dt_free = free_time
    dt_mw = mw_time
    print("Starting continuous calculations, this may take a while...")
    # This loop is defined to reproduce the same Sequence as a pulsed Sequence, but with continuous laser
    for i in tqdm(range(n)):
        tis.append(ti)
        # Here Laser is ON
        # Run the dynamics with the new expectation operators
        tf, result = model(dt_laser,
                                     init_state,
                                     b=b,
                                     om_r=om_r,
                                     om=om,
                                     w_p=w_p,
                                     ti=ti,
                                     mode="Laser",
                                     progress_bar=progress_bar,
                                     i=i)  # Pass new exp_ops
        # Store the Result class for when the Laser is on
        results = np.append(results, result)
        # Save the initial state for the next part of the Sequence
        init_state = result.states[-1]
        # Update the initial time
        ti = tf
        tfs.append(tf)
        
        # Free evolution
        tf, result = model(dt_free,
                                     init_state,
                                     b=b,
                                     om_r=om_r,
                                     om=om,
                                     w_p=w_p,
                                     ti=ti,
                                     mode="Free",
                                     progress_bar=progress_bar,
                                     i=i)  # Pass new exp_ops
        results = np.append(results, result)
        tis.append(ti)
        tfs.append(tf)
        ti = tf
        init_state = result.states[-1]
        
        # Microwave pulse
        tf, result = model(dt_mw,
                                     init_state,
                                     b=b,
                                     om_r=om_r,
                                     om=om,
                                     w_p=w_p,
                                     ti=ti,
                                     mode="MW",
                                     progress_bar=progress_bar,
                                     i=i)  # Pass new exp_ops
        results = np.append(results, result)
        tis.append(ti)
        tfs.append(tf)
        ti = tf
        init_state = result.states[-1]
    times = np.concatenate([res.times for res in results])
    N = exp_ops_14
    n_exp = np.array([np.concatenate([qt.expect(M, res.states) for res in results]) for M in N])
    S = s_ops_14
    s_exp = np.array([np.concatenate([qt.expect(M, res.states)  for res in results]) for M in S])
    Nit = [IdNV&nit[0] * nit[0].dag(), IdNV&nit[1] * nit[1].dag()] #type: ignore
    nit_exp = np.array([np.concatenate([qt.expect(M, res.states)  for res in results]) for M in Nit])
    ## Gather the expectation values from the results array
    #for j in range(len(results[0].expect)):
    #    nn_exp = np.array([])
    #    for i in range(len(results)):
    #        nn_exp = np.append(nn_exp, results[i].expect[j])
    #    if j == 0:
    #        n_exp = np.array([nn_exp])
    #    else:
    #        n_exp = np.append(n_exp, [nn_exp], axis=0)
#
    ## Removed splitting into s_exp and nit_exp
    #s_exp = np.array([])
    #nit_exp = np.array([])
#
    ## Gather the times for the plots
    #times = np.array([])
    #for i in range(len(results)):
    #    times = np.append(times, results[i].times)

    print("Calculations are finished!! param order(Expect,times,tis,tfs,results,nit_exp,s_exp)")
    return n_exp, times, tis, tfs, results, nit_exp, s_exp

In [ ]:
from matplotlib.lines import Line2D
from matplotlib.patches import Rectangle
def plot_cont_14_ang_doh_vs_cont_dua(n_exp_doh, times_doh, tis_doh, tfs_doh, 
                                      n_exp_dua, times_dua, tis_dua, tfs_dua, 
                                      b, p, t,om):
    """
    Plots population dynamics for 14 levels (7 electronic states × 2 nuclear spin states)
    comparing two datasets (Doherty vs. Duarte).
    """
    # Define colors for electronic states (7 colors)
    e_colors = [
        "dodgerblue", "chocolate", "gold", 
        "mediumpurple", "mediumseagreen", "lightskyblue", "magenta"
    ]
    
    # Line styles for nuclear spin states
    n_marker = ['v', '^']
    m_style = ['-', '--']
    # Build a list of indices:
    # - For the first 5000 points, mark every 500.
    markers1 = list(range(0, 5000, 500))
    # - For the next 1000 points (5000–5999), mark every 400.
    markers2 = list(range(5000, 6000, 400))
    # - For the last 1000 points (6000–6999), mark every 100.
    markers3 = list(range(6000, 7000, 100))

    mark_indices = markers1 + markers2 + markers3
    plt.figure(figsize=(14, 8))

    # Plot Doherty data
    for i in range(14):
        e_idx = i // 2 # Electronic state index (0-6)
        n_idx = i % 2   # Nuclear spin index (0-1)
        
        plt.plot(times_doh, n_exp_doh[i], 
                 color=e_colors[e_idx], 
                 linestyle=m_style[0],
                 marker=n_marker[n_idx],
                 markersize=8,markevery=mark_indices,
                 label=f"$n^{{doh}}_{{{e_idx+1},{n_idx}}}$")
    #for k in range(2):
    #    plt.plot(times_doh, n_exp_doh[14+k], 
    #                color='forestgreen', 
    #                linestyle='-',
    #                marker=n_marker[n_idx],
    #                markersize=4,markevery=50,
    #                linewidth=2,
    #                label=f'$n^{{doh}}_{{{{c}},{n_idx}}}')
    # Plot Duarte data with thicker dashed lines
    for i in range(14):
        e_idx = i // 2
        n_idx = i % 2
        
        plt.plot(times_dua, n_exp_dua[i], 
                 color=e_colors[e_idx], 
                 linestyle=m_style[1],
                 marker=n_marker[n_idx],
                 markersize=8,markevery=mark_indices,
                 linewidth=3, alpha=0.7,
                 label=f"$n^{{dua}}_{{{e_idx+1},{n_idx}}}$")
    #for k in range(2):
    #    plt.plot(times_dua, n_exp_dua[14+k], 
    #                color='forestgreen', 
    #                linestyle='--',
    #                marker=n_marker[n_idx],
    #                markersize=4,markevery=90,
    #                linewidth=2,
    #                label=f'$n^{{dua}}_{{{{c}},{n_idx}}}')
    # Highlight Sequence phases
    n = 0
    
    eps_y = 0.05 * (np.max(np.concatenate([n_exp_doh, n_exp_dua])) - 
                   np.min(np.concatenate([n_exp_doh, n_exp_dua])))
    eps_x = 0.05 * (np.max(np.concatenate([times_doh, times_dua])) - 
                   np.min(np.concatenate([times_doh, times_dua])))
    for i in zip(tis_doh, tfs_doh):
        if n % 3 == 0:
            plt.fill_betweenx(
            [np.min(np.concatenate([n_exp_doh, n_exp_dua]))-eps_y, np.max(np.concatenate([n_exp_doh, n_exp_dua]))+eps_y], 
            i[0], i[1],
            color="palegreen", alpha=0.3,
            label="Laser ON" if n == 0 else "")
            t_laser = i[1]-i[0]
        elif (n-1) % 3 == 0:
            plt.fill_betweenx(
            [np.min(np.concatenate([n_exp_doh, n_exp_dua]))-eps_y, np.max(np.concatenate([n_exp_doh, n_exp_dua]))+eps_y], 
            i[0], i[1],
            color="lightblue", alpha=0.3,
            label="Free Evolution" if n == 1 else "")
            t_free = i[1]-i[0]
        else:
            plt.fill_betweenx(
            [np.min(np.concatenate([n_exp_doh, n_exp_dua]))-eps_y, np.max(np.concatenate([n_exp_doh, n_exp_dua]))+eps_y], 
            i[0], i[1],
            color="orchid", alpha=0.3,
            label="MW ON" if n == 2 else "")
            t_mw = i[1]-i[0]
        n += 1

    # Formatting
    plt.ylabel("Population",fontsize=20)
    plt.xlabel(r"Time ($\mu$s)",fontsize=20)
    plt.title(
        f"Comparisson between Doherty and Duarte Hyperfine evolution\n B={b:.2f} G, $\\theta_B$={Fraction(t/np.pi).limit_denominator()}$\\pi$, $\\phi_B$={Fraction(p/np.pi).limit_denominator()}$\\pi$, $\\omega={om:.2f}$ MHz",
        fontsize=20)
    plt.ylim(np.min(np.concatenate([n_exp_doh, n_exp_dua]))-eps_y, np.max(np.concatenate([n_exp_doh, n_exp_dua]))+eps_y)
    t_lim = (np.min(np.append(times_doh,times_dua))-eps_x, np.max(np.append(times_doh,times_dua)) + eps_x)
    plt.xlim(t_lim)
    
    # Create simplified legend
    handles = []
    # Add electronic state legend
    for idx, color in enumerate(e_colors):
        handles.append(Line2D([], [], color=color, 
                                label=f"$n_{idx+1}$"))
    handles.append(Line2D([], [], color="forestgreen", 
                            label=f"$n_c$"))
    # Add nuclear spin legend
    handles.append(Line2D([], [], color='black', linestyle='-',marker=n_marker[0],
                            label="$n_i |-1/2\\rangle$"))
    handles.append(Line2D([], [], color='black', linestyle='-',marker=n_marker[1],
                            label="$n_i |+1/2\\rangle$"))
    handles.append(Line2D([], [], color='black', linestyle=m_style[0],
                            label="Doherty"))
    handles.append(Line2D([], [], color='black', linestyle=m_style[1],linewidth=3,
                            label="Duarte"))
    # Add Sequence legend
    handles.append(Rectangle((0,0),1,1, fc="palegreen", alpha=0.3,
                               label="Laser ON"))
    handles.append(Rectangle((0,0),1,1, fc="lightblue", alpha=0.3,
                               label="Free Evolution"))
    handles.append(Rectangle((0,0),1,1, fc="orchid", alpha=0.3,
                               label="MW ON"))

    plt.legend(handles=handles, bbox_to_anchor=(1.05, 1), 
             loc='upper left', borderaxespad=0.,fontsize=16)
    plt.minorticks_on()
    
    plt.show()

In [ ]:
def plot_cont_14_doh_vs_cont_dua(n_exp_doh, times_doh, tis_doh, tfs_doh, 
                                      n_exp_dua, times_dua, tis_dua, tfs_dua, 
                                      b, om_r):
    """
    Plots population dynamics for 14 levels (7 electronic states × 2 nuclear spin states)
    comparing two datasets (Doherty vs. Duarte).
    """
    # Define colors for electronic states (7 colors)
    e_colors = [
        "dodgerblue", "chocolate", "gold", 
        "mediumpurple", "mediumseagreen", "lightskyblue", "magenta"
    ]
    
    # Line styles for nuclear spin states
    n_marker = ['v', '^']
    m_style = ['-', '--']
    # Build a list of indices:
    # - For the first 5000 points, mark every 500.
    markers1 = list(range(0, 5000, 500))
    # - For the next 1000 points (5000–5999), mark every 400.
    markers2 = list(range(5000, 6000, 400))
    # - For the last 1000 points (6000–6999), mark every 100.
    markers3 = list(range(6000, 7000, 100))

    mark_indices = markers1 + markers2 + markers3
    plt.figure(figsize=(14, 8))

    # Plot Doherty data
    for i in range(14):
        e_idx = i // 2 # Electronic state index (0-6)
        n_idx = i % 2   # Nuclear spin index (0-1)
        
        plt.plot(times_doh, n_exp_doh[i], 
                 color=e_colors[e_idx], 
                 linestyle=m_style[0],
                 marker=n_marker[n_idx],
                 markersize=8,markevery=mark_indices,
                 label=f"$n^{{doh}}_{{{e_idx+1},{n_idx}}}$")
    #for k in range(2):
    #    plt.plot(times_doh, n_exp_doh[14+k], 
    #                color='forestgreen', 
    #                linestyle='-',
    #                marker=n_marker[n_idx],
    #                markersize=4,markevery=50,
    #                linewidth=2,
    #                label=f'$n^{{doh}}_{{{{c}},{n_idx}}}')
    # Plot Duarte data with thicker dashed lines
    for i in range(14):
        e_idx = i // 2
        n_idx = i % 2
        
        plt.plot(times_dua, n_exp_dua[i], 
                 color=e_colors[e_idx], 
                 linestyle=m_style[1],
                 marker=n_marker[n_idx],
                 markersize=8,markevery=mark_indices,
                 linewidth=3, alpha=0.7,
                 label=f"$n^{{dua}}_{{{e_idx+1},{n_idx}}}$")
    #for k in range(2):
    #    plt.plot(times_dua, n_exp_dua[14+k], 
    #                color='forestgreen', 
    #                linestyle='--',
    #                marker=n_marker[n_idx],
    #                markersize=4,markevery=90,
    #                linewidth=2,
    #                label=f'$n^{{dua}}_{{{{c}},{n_idx}}}')
    # Highlight Sequence phases
    n = 0
    
    eps_y = 0.05 * (np.max(np.concatenate([n_exp_doh, n_exp_dua])) - 
                   np.min(np.concatenate([n_exp_doh, n_exp_dua])))
    eps_x = 0.05 * (np.max(np.concatenate([times_doh, times_dua])) - 
                   np.min(np.concatenate([times_doh, times_dua])))
    for i in zip(tis_doh, tfs_doh):
        if n % 3 == 0:
            plt.fill_betweenx(
            [np.min(np.concatenate([n_exp_doh, n_exp_dua]))-eps_y, np.max(np.concatenate([n_exp_doh, n_exp_dua]))+eps_y], 
            i[0], i[1],
            color="palegreen", alpha=0.3,
            label="Laser ON" if n == 0 else "")
            t_laser = i[1]-i[0]
        elif (n-1) % 3 == 0:
            plt.fill_betweenx(
            [np.min(np.concatenate([n_exp_doh, n_exp_dua]))-eps_y, np.max(np.concatenate([n_exp_doh, n_exp_dua]))+eps_y], 
            i[0], i[1],
            color="lightblue", alpha=0.3,
            label="Free Evolution" if n == 1 else "")
            t_free = i[1]-i[0]
        else:
            plt.fill_betweenx(
            [np.min(np.concatenate([n_exp_doh, n_exp_dua]))-eps_y, np.max(np.concatenate([n_exp_doh, n_exp_dua]))+eps_y], 
            i[0], i[1],
            color="orchid", alpha=0.3,
            label="MW ON" if n == 2 else "")
            t_mw = i[1]-i[0]
        n += 1

    # Formatting
    plt.ylabel("Population",fontsize=20)
    plt.xlabel(r"Time ($\mu$s)",fontsize=20)
    plt.title(f"Comparisson between Doherty and Duarte Hyperfine evolution\n B={b:.2f} G, $\\Omega_r$={om_r:.2f} MHz",fontsize=20)
    plt.ylim(np.min(np.concatenate([n_exp_doh, n_exp_dua]))-eps_y, np.max(np.concatenate([n_exp_doh, n_exp_dua]))+eps_y)
    t_lim = (np.min(np.append(times_doh,times_dua))-eps_x, np.max(np.append(times_doh,times_dua)) + eps_x)
    plt.xlim(t_lim)
    
    # Create simplified legend
    handles = []
    # Add electronic state legend
    for idx, color in enumerate(e_colors):
        handles.append(Line2D([], [], color=color, 
                                label=f"$n_{idx+1}$"))
    #handles.append(Line2D([], [], color="forestgreen", label=f"$n_c$"))
    # Add nuclear spin legend
    handles.append(Line2D([], [], color='black', linestyle='-',marker=n_marker[0],
                            label="$n_i |-1/2\\rangle$"))
    handles.append(Line2D([], [], color='black', linestyle='-',marker=n_marker[1],
                            label="$n_i |+1/2\\rangle$"))
    handles.append(Line2D([], [], color='black', linestyle=m_style[0],
                            label="Doherty"))
    handles.append(Line2D([], [], color='black', linestyle=m_style[1],linewidth=3,
                            label="Duarte"))
    # Add Sequence legend
    handles.append(Rectangle((0,0),1,1, fc="palegreen", alpha=0.3,
                               label="Laser ON"))
    handles.append(Rectangle((0,0),1,1, fc="lightblue", alpha=0.3,
                               label="Free Evolution"))
    handles.append(Rectangle((0,0),1,1, fc="orchid", alpha=0.3,
                               label="MW ON"))

    plt.legend(handles=handles, bbox_to_anchor=(1.05, 1), 
             loc='upper left', borderaxespad=0.,fontsize=16)
    plt.minorticks_on()
    
    plt.show()

#### 14 Levels - Static field and Rabi frequency

In [ ]:
def run_simulation_14_bomr(params):
    b, omr = params
    b0 = B0(b, 0.0, 0.0)
    doherty_14 = continuous_14_zfs_hf(dynamics_doh_hf, 10.0, 3.0*t2_gs, 1.0,
                                      b=b0, om_r=omr, om=D_gs-mu_e*b0[2],
                                      sequence_repeats=1, progress_bar="OFF")
    duarte_14 = continuous_14_zfs_hf(dynamics_dua_hf, 10.0, 3.0*t2_gs, 1.0,
                                     b=b0, om_r=omr, om=D_gs-mu_e*b0[2],
                                     sequence_repeats=1, progress_bar="OFF")
    return b, omr, doherty_14, duarte_14

# Parameter arrays:
om_r_pi = np.pi/t2_gs
oms_r = np.array([om_r_pi, 2*om_r_pi, 10*om_r_pi])
bs = np.array([0, 100, 200, 500, 700, 1000])

# Create list of (b, omr) pairs.
params = [(b, omr) for b in bs for omr in oms_r]

# Use parallel_map (or loky_pmap) with the default num_cpus (8) or specify e.g. num_cpus=4:
if run_bomr:
    results_bomr_14 = qt.loky_pmap(run_simulation_14_bomr, params, progress_bar="tqdm",map_kw={'fail_fast': True})
    np.save('NumpyArrays\\results_bomr_14.npy',np.array(results_bomr_14,dtype=object))
else:
    results_bomr_14=np.load('NumpyArrays\\results_bomr_14.npy',allow_pickle=True)
# For loky_pmap, you could do:
# from qutip.parallel import loky_pmap
# results = loky_pmap(run_simulation, params, progress_bar="tqdm", num_cpus=8)

# Now plot the results:
for b, omr, doherty_14, duarte_14 in results_bomr_14: #type:ignore
    plot_cont_14_doh_vs_cont_dua(doherty_14[0], doherty_14[1], doherty_14[2], doherty_14[3],
                                duarte_14[0], duarte_14[1], duarte_14[2], duarte_14[3],
                                b, omr)

#### 14 Levels - Angles

In [ ]:
def run_simulation_14_bang(params):
    b, p, t = params
    b0 = B0(b, p, t)
    if np.isclose(np.abs(b0[2]),b):
        om=D_gs-mu_e*b0[2]
    else:            
        d_b=mu_e*b/D_gs
        if np.isclose(b0[2],0):
            om=D_gs*(1+3/2*(d_b)**2-d_b**3*(1/8*np.sqrt(b0[0]**2+b0[1]**2)))
        else:    
            om=D_gs*(1-d_b*np.cos(t)+3/2*(d_b*np.sin(t))**2-d_b**3*(1/8*np.sin(t)**3*np.sqrt(b0[0]**2+b0[1]**2)/b0[2]-1/2*np.sin(t)**2*np.cos(t)))
    doherty_14 = continuous_14_zfs_hf(dynamics_doh_hf, 10.0, 3.0*t2_gs, 1.0,
                                      b=b0, om_r=Om_r, om=om,
                                      sequence_repeats=1, progress_bar="OFF")
    duarte_14 = continuous_14_zfs_hf(dynamics_dua_hf, 10.0, 3.0*t2_gs, 1.0,
                                     b=b0, om_r=Om_r, om=om,
                                     sequence_repeats=1, progress_bar="OFF")
    return b, p, t, om, doherty_14, duarte_14

# Parameter arrays:
b14=[200,500]
phibs=np.array([0.0,np.pi/2,2*np.pi/3,np.pi])
thetabs=np.array([0.0,np.pi/2,2*np.pi/3,np.pi])

# Create list of (b, omr) pairs.
params = [(b, p, t) for b in b14 for t in thetabs for p in (phibs if not np.isclose(t,0) and not np.isclose(t,np.pi) else np.array([0.0]))]

if run_bang:
    results_bang_14 = qt.loky_pmap(run_simulation_14_bang, params, progress_bar="tqdm",map_kw={'fail_fast': True})
    np.save("NumpyArrays\\results_bang_14.npy",np.array(results_bang_14,dtype=object))
else:
    results_bang_14=np.load("NumpyArrays\\results_bang_14.npy",allow_pickle=True)
# For loky_pmap, you could do:
# from qutip.parallel import loky_pmap
# results = loky_pmap(run_simulation, params, progress_bar="tqdm", num_cpus=8)

# Now plot the results:
for b, p, t, om, doherty_14, duarte_14 in results_bang_14: #type:ignore
    plot_cont_14_ang_doh_vs_cont_dua(doherty_14[0], doherty_14[1], doherty_14[2], doherty_14[3],
                                duarte_14[0], duarte_14[1], duarte_14[2], duarte_14[3],
                                b, p, t, om)

#### 7 levels

In [ ]:
def plot_cont_ang_doh_vs_cont_dua(n_exp_doh,times_doh,tis_doh,tfs_doh,n_exp_dua,times_dua,tis_dua,tfs_dua,b,p,t,om):
    # Set the colors of each state
    colors = [
        "dodgerblue",
        "chocolate",
        "gold",
        "mediumpurple",
        "mediumseagreen",
        "lightskyblue",
        "magenta",
        "forestgreen",
    ]
    doh="{doh}"
    dua="{dua}"
    # Defines figure size
    plt.figure(figsize=(14, 8))
    # Plot the population of each state
    plt.plot(times_doh, n_exp_doh[-1], label="$n^{doh}_c$", color=colors[-1])
    plt.plot(times_dua, n_exp_dua[-1], label=f"$n^{dua}_c$", color=colors[-1],ls='--')
    for i in range(len(n_exp_doh) - 1):
        plt.plot(times_doh, n_exp_doh[i], label=f"$n^{doh}_{i+1}$", color=colors[i])
    for i in range(len(n_exp_dua) - 1):
        plt.plot(times_dua, n_exp_dua[i], label=f"$n^{dua}_{i+1}$", color=colors[i],ls='--')
    # Highlighting the times where the laser and microwaves are on and the metastable state depletion evolutions
    n = 0
    eps_y = abs(np.max(np.append(n_exp_doh,n_exp_dua)) - np.min(np.append(n_exp_doh,n_exp_dua)))*5e-2
    eps_x = abs(np.max(np.append(times_doh,times_dua)) - np.min(np.append(times_doh,times_dua)))*5e-2
    for i in zip(tis_doh, tfs_doh):
        if n % 3 == 0:
            plt.fill_betweenx(
                [np.min(np.append(n_exp_doh,n_exp_dua)) - eps_y, np.max(np.append(n_exp_doh,n_exp_dua)) + eps_y],
                i[0],
                i[1],
                color="palegreen",
                alpha=0.5 ** (int(n / 3) + 1),
                label=f"Laser ON #{int(n/3)+1}",
            )
            t_laser=i[1]-i[0]
        else:
            if (n - 1) % 3 == 0:
                plt.fill_betweenx(
                    [np.min(np.append(n_exp_doh,n_exp_dua)) - eps_y, np.max(np.append(n_exp_doh,n_exp_dua)) + eps_y],
                    i[0],
                    i[1],
                    color="lightblue",
                    alpha=0.5 ** (int(n / 3) + 1),
                    label=f"MS depl #{int(n/3)+1}",
                )
                t_free=i[1]-i[0]
            else:
                plt.fill_betweenx(
                    [np.min(np.append(n_exp_doh,n_exp_dua)) - eps_y, np.max(np.append(n_exp_doh,n_exp_dua)) + eps_y],
                    i[0],
                    i[1],
                    color="orchid",
                    alpha=0.5 ** (int(n / 3) + 1),
                    label=f"MW ON #{int(n/3)+1}",
                )
                t_mw=i[1]-i[0]
        n += 1
    plt.ylabel(f"Population",fontsize=20)
    plt.xlabel(r"Time($\mu$s)",fontsize=20)
    plt.title(
        f"Comparisson between Doherty and Duarte Hyperfine evolution\n B={b:.2f} G, $\\theta_B$={Fraction(t/np.pi).limit_denominator()}$\\pi$, $\\phi_B$={Fraction(p/np.pi).limit_denominator()}$\\pi$, $\\omega={om:.2f}$ MHz",
        fontsize=20)
    # set the time limits you want to show in the plot
    t_lim = (np.min(np.append(times_doh,times_dua))-eps_x, np.max(np.append(times_doh,times_dua)) + eps_x)
    plt.xlim(t_lim)
    plt.ylim((np.min(np.append(n_exp_doh,n_exp_dua)) - eps_y, np.max(np.append(n_exp_doh,n_exp_dua)) + eps_y))
    plt.minorticks_on()
    
    plt.legend(ncol=2,bbox_to_anchor=(1.05, 1), loc="upper left",fontsize=16)
    plt.show()

In [ ]:
def plot_cont_doh_vs_cont_dua(n_exp_doh,times_doh,tis_doh,tfs_doh,n_exp_dua,times_dua,tis_dua,tfs_dua,b,om_r):
    # Set the colors of each state
    colors = [
        "dodgerblue",
        "chocolate",
        "gold",
        "mediumpurple",
        "mediumseagreen",
        "lightskyblue",
        "magenta",
        "forestgreen",
    ]
    doh="{doh}"
    dua="{dua}"
    # Defines figure size
    plt.figure(figsize=(14, 8))
    # Plot the population of each state
    plt.plot(times_doh, n_exp_doh[-1], label="$n^{doh}_c$", color=colors[-1])
    plt.plot(times_dua, n_exp_dua[-1], label=f"$n^{dua}_c$", color=colors[-1],ls='--')
    for i in range(len(n_exp_doh) - 1):
        plt.plot(times_doh, n_exp_doh[i], label=f"$n^{doh}_{i+1}$", color=colors[i])
    for i in range(len(n_exp_dua) - 1):
        plt.plot(times_dua, n_exp_dua[i], label=f"$n^{dua}_{i+1}$", color=colors[i],ls='--')
    # Highlighting the times where the laser and microwaves are on and the metastable state depletion evolutions
    n = 0
    eps_y = abs(np.max(np.append(n_exp_doh,n_exp_dua)) - np.min(np.append(n_exp_doh,n_exp_dua)))*5e-2
    eps_x = abs(np.max(np.append(times_doh,times_dua)) - np.min(np.append(times_doh,times_dua)))*5e-2
    for i in zip(tis_doh, tfs_doh):
        if n % 3 == 0:
            plt.fill_betweenx(
                [np.min(np.append(n_exp_doh,n_exp_dua)) - eps_y, np.max(np.append(n_exp_doh,n_exp_dua)) + eps_y],
                i[0],
                i[1],
                color="palegreen",
                alpha=0.5 ** (int(n / 3) + 1),
                label=f"Laser ON #{int(n/3)+1}",
            )
            t_laser=i[1]-i[0]
        else:
            if (n - 1) % 3 == 0:
                plt.fill_betweenx(
                    [np.min(np.append(n_exp_doh,n_exp_dua)) - eps_y, np.max(np.append(n_exp_doh,n_exp_dua)) + eps_y],
                    i[0],
                    i[1],
                    color="lightblue",
                    alpha=0.5 ** (int(n / 3) + 1),
                    label=f"MS depl #{int(n/3)+1}",
                )
                t_free=i[1]-i[0]
            else:
                plt.fill_betweenx(
                    [np.min(np.append(n_exp_doh,n_exp_dua)) - eps_y, np.max(np.append(n_exp_doh,n_exp_dua)) + eps_y],
                    i[0],
                    i[1],
                    color="orchid",
                    alpha=0.5 ** (int(n / 3) + 1),
                    label=f"MW ON #{int(n/3)+1}",
                )
                t_mw=i[1]-i[0]
        n += 1
    plt.ylabel(f"Population",fontsize=20)
    plt.xlabel(r"Time($\mu$s)",fontsize=20)
    plt.title(f"Comparisson between Doherty and Duarte Hyperfine evolution\n B={b:.2f} G, $\\Omega_r$={om_r:.2f} MHz",fontsize=20)
    # set the time limits you want to show in the plot
    t_lim = (np.min(np.append(times_doh,times_dua))-eps_x, np.max(np.append(times_doh,times_dua)) + eps_x)
    plt.xlim(t_lim)
    plt.ylim((np.min(np.append(n_exp_doh,n_exp_dua)) - eps_y, np.max(np.append(n_exp_doh,n_exp_dua)) + eps_y))
    plt.minorticks_on()
    
    plt.legend(ncol=2,bbox_to_anchor=(1.05, 1), loc="upper left",fontsize=16)
    plt.show()

#### 7 Levels - Static field and Rabi frequency 

In [ ]:
def run_simulation_bomr(params):
    b, omr = params
    b0 = B0(b, 0.0, 0.0)
    doherty = continuous_zfs_hf(dynamics_doh_hf, 10.0, 3.0*t2_gs, 1.0,
                                      b=b0, om_r=omr, om=D_gs-mu_e*b0[2],
                                      sequence_repeats=1, progress_bar="OFF")
    duarte = continuous_zfs_hf(dynamics_dua_hf, 10.0, 3.0*t2_gs, 1.0,
                                     b=b0, om_r=omr, om=D_gs-mu_e*b0[2],
                                     sequence_repeats=1, progress_bar="OFF")
    norm= continuous_zfs(dynamics_no, 10.0, 3.0*t2_gs, 1.0,
                                     b=b0, om_r=omr, om=D_gs-mu_e*b0[2],
                                     sequence_repeats=1, progress_bar="OFF")
    return b, omr, doherty, duarte, norm

# Parameter arrays:
om_r_pi = np.pi/t2_gs
oms_r = np.array([om_r_pi, 2*om_r_pi, 10*om_r_pi])
bs = np.array([0, 100, 200, 500, 700, 1000])

# Create list of (b, omr) pairs.
params = [(b, omr) for b in bs for omr in oms_r]
if run_bomr:
# Use parallel_map (or loky_pmap) with the default num_cpus (8) or specify e.g. num_cpus=4:
    results = qt.loky_pmap(run_simulation_bomr, params, progress_bar="tqdm",map_kw={'fail_fast': True})
    np.save("NumpyArrays\\results_bomr.npy",np.array(results,dtype=object))
else:
    results=np.load("NumpyArrays\\results_bomr.npy",allow_pickle=True)
# For loky_pmap, you could do:
# from qutip.parallel import loky_pmap
# results = loky_pmap(run_simulation, params, progress_bar="tqdm", num_cpus=8)

# Now plot the results:
for b, omr, doherty, duarte, norm in results: #type:ignore
    plot_cont_doh_vs_cont_dua(doherty[0], doherty[1], doherty[2], doherty[3],duarte[0], duarte[1], duarte[2], duarte[3],b, omr)
    plot_cont_hf_vs_cont_bomrs(doherty[0], doherty[1], doherty[2], doherty[3],norm[0], norm[1], norm[2], norm[3],b, omr)

#### 7 Levels - Angles

In [ ]:
def run_simulation_bang(params):
    b, p, t = params
    b0 = B0(b, p, t)
    if np.isclose(np.abs(b0[2]),b):
        om=D_gs-mu_e*b0[2]
    else:            
        d_b=mu_e*b/D_gs
        if np.isclose(b0[2],0):
            om=D_gs*(1+3/2*(d_b)**2-d_b**3*(1/8*np.sqrt(b0[0]**2+b0[1]**2)))
        else:    
            om=D_gs*(1-d_b*np.cos(t)+3/2*(d_b*np.sin(t))**2-d_b**3*(1/8*np.sin(t)**3*np.sqrt(b0[0]**2+b0[1]**2)/b0[2]-1/2*np.sin(t)**2*np.cos(t)))
    doherty = continuous_zfs_hf(dynamics_doh_hf, 10.0, 3.0*t2_gs, 1.0,
                                      b=b0, om_r=Om_r, om=om,
                                      sequence_repeats=1, progress_bar="OFF")
    duarte = continuous_zfs_hf(dynamics_dua_hf, 10.0, 3.0*t2_gs, 1.0,
                                     b=b0, om_r=Om_r, om=om,
                                     sequence_repeats=1, progress_bar="OFF")
    norm = continuous_zfs(dynamics_no, 10.0, 3.0*t2_gs, 1.0,
                                     b=b0, om_r=Om_r, om=om,
                                     sequence_repeats=1, progress_bar="OFF")
    return b, p, t, om, doherty, duarte, norm

# Parameter arrays:
b14=[200,500]
phibs=np.array([0.0,np.pi/2,2*np.pi/3,np.pi])
thetabs=np.array([0.0,np.pi/2,2*np.pi/3,np.pi])

# Create list of (b, omr) pairs.
params = [(b, p, t) for b in b14 for t in thetabs for p in (phibs if not np.isclose(t,0) and not np.isclose(t,np.pi) else np.array([0.0]))]
if run_bang:
# Use parallel_map (or loky_pmap) with the default num_cpus (8) or specify e.g. num_cpus=4:
    results_bang = qt.loky_pmap(run_simulation_bang, params, progress_bar="tqdm",map_kw={'fail_fast': True})
    np.save("NumpyArrays\\results_bang.npy",np.array(results_bang,dtype=object))
else:
    results_bang=np.load("NumpyArrays\\results_bang.npy",allow_pickle=True)
# For loky_pmap, you could do:
# from qutip.parallel import loky_pmap
# results = loky_pmap(run_simulation, params, progress_bar="tqdm", num_cpus=8)

# Now plot the results:
for b, p, t, om, doherty, duarte, norm in results_bang: #type:ignore
    plot_cont_ang_doh_vs_cont_dua(doherty[0], doherty[1], doherty[2], doherty[3],
                                duarte[0], duarte[1], duarte[2], duarte[3],
                                b, p, t, om)
    plot_cont_hf_vs_cont_bangs(doherty[0], doherty[1], doherty[2], doherty[3],
                               norm[0], norm[1], norm[2], norm[3],
                               b, p, t, om)

### Sequences

#### Constants

In [ ]:
b_odmr=150
n_p_odmr=1
n_p_ramsey=1.5
w_p_odmr=n_p_odmr*1.9
w_p_ramsey=n_p_ramsey*1.9
#om_r_odmr=4*np.pi/t2_gs
om_r_odmr=15.7

#### CW-ODMR

In [ ]:
def cwodmr_non_parallel(t, ws, model, init_state,exp_ops, 
           b=B, om_r=om_r_odmr, w_p=w_p_odmr):
    """
    Calculate the continuous wave optical detected resonance (CWODMR) dynamics.
    
    Parameters:
    - t (float): Time parameter.
    - ws (numpy.ndarray): Array of angular frequencies.
    - model (function): Function that defines the dynamics of the system.
    - init_state (object): Initial state of the system.
    - b (float): Parameter b (default: B).
    - om_r (float): Rabi frequency (default: Om_r).
    - w_p (float): Angular frequency (default: W_p).
    
    Returns:
    - ws (numpy.ndarray): Array of angular frequencies.
    - flur (numpy.ndarray): Array of PL values.
    - times (numpy.ndarray): Array of time values.
    - n_exp (numpy.ndarray): Array of expectation values.
    - results (numpy.ndarray): Array of simulation results.
    """
    #continuous MW and laser appplied together
    results = np.array([])
    flur = np.array([])
    for w in tqdm(ws):
        # Run the dynamics based on the mode choosen
        _, result = model(t, 
                           init_state,
                           b=b,
                           om_r=om_r,
                           om=w,
                           w_p=w_p/n_p_odmr,
                           ti=0.0, 
                           mode="Laser-MW", 
                           progress_bar="OFF") # type:ignore
        # Gather the expectation values from the results array
        results = np.append(results,result)
        fl=np.array([])
        fl=np.add(result.expect[3],result.expect[4])
        fl=np.add(fl,result.expect[5])
        flur=np.append(flur,np.sum(fl))
   
    n_exp = np.array([np.concatenate([qt.expect(M, res.states) for res in results],axis=0) for M in exp_ops])
    #for j in range(len(results[0].expect)):
    #    nn_exp = np.array([])
    #    for i in range(len(results)):
    #        nn_exp = np.append(nn_exp, results[i].expect[j])
    #    if j == 0:
    #        n_exp = np.array([nn_exp])
    #    else:
    #        n_exp = np.append(n_exp, [nn_exp], axis=0)
    # Gather the times for the plots
    times = results[0].times
    return ws,flur,times,n_exp,results
def run_single_frequency(w, model, t, init_state, b, om_r, w_p):
    # Run the dynamics for a single frequency w.
    _, result = model(t, init_state, b=b, om_r=om_r, om=w,
                      w_p=w_p/n_p_odmr, ti=0.0, mode="Laser-MW",
                      progress_bar="OFF")
    # Compute PL from the expectation values.
    return result
def cwodmr(t, ws, model, init_state, exp_ops, b=B, om_r=om_r_odmr, w_p=w_p_odmr):
    # Create a version of run_single_frequency with fixed model, t, etc.
    run_single = partial(run_single_frequency, model=model, t=t,
                         init_state=init_state, b=b, om_r=om_r, w_p=w_p)
    # Run the simulations in parallel over ws.
    outs = qt.loky_pmap(run_single, ws, progress_bar="tqdm",map_kw={'fail_fast': True})
    # Unpack results and PL values. 
    results = [out for out in outs] #type:ignore
    n_exp = np.array([[qt.expect(M, res.states) for M in exp_ops] for res in results]) #type:ignore
    flur = np.array([np.sum(ns[3]+ns[4]+ns[5]) for ns in n_exp])
    # Combine expectation values from each result.
    #for j in range(len(results[0].expect)):
    #    nn_exp = np.array([results[i].expect[j] for i in range(len(results))])
    #    if j == 0:
    #        n_exp = np.array([nn_exp])
    #    else:
    #        n_exp = np.append(n_exp, [nn_exp], axis=0)
    
    # Gather the time arrays.
    times = results[0].times #type:ignore
    return ws, flur, times, n_exp, results

##### Run CW-ODMR

In [ ]:
if run_odmr:
    if b_odmr<500:
        n,m,s=2/5,25,10
        eps=n*b_odmr
    else:
        n,m,s=1/5,25,20
        eps=n*b_odmr
    wslhfl=np.linspace(D_gs-mu_e*b_odmr-mu_e*b_odmr*n,D_gs-mu_e*b_odmr-4*a_gs[0],2*s)
    wslhf=np.linspace(D_gs-mu_e*b_odmr-4*a_gs[0],D_gs-mu_e*b_odmr+4*a_gs[0],m)
    wslhfh=np.linspace(D_gs-mu_e*b_odmr+4*a_gs[0],D_gs-mu_e*b_odmr+mu_e*b_odmr*n,2*s)
    wsm=np.linspace(D_gs-mu_e*b_odmr+mu_e*b_odmr*n+eps,D_gs+mu_e*b_odmr-mu_e*b_odmr*n-eps,s+s//2)
    wshhfl=np.linspace(D_gs+mu_e*b_odmr-mu_e*b_odmr*n,D_gs+mu_e*b_odmr-4*a_gs[0],2*s)
    wshhf=np.linspace(D_gs+mu_e*b_odmr-4*a_gs[0],D_gs+mu_e*b_odmr+4*a_gs[0],m)
    wshhfh=np.linspace(D_gs+mu_e*b_odmr+4*a_gs[0],D_gs+mu_e*b_odmr+mu_e*b_odmr*n,2*s)
    ws=np.unique(np.hstack((wslhfl,wslhf,wslhfh,wsm,wshhfl,wshhf,wshhfh))) if b_odmr>10 else np.linspace(D_gs-100,D_gs+100,200)
    ws_codm,flur_codm,ts_codm,ns_codm,res_codm=\
        cwodmr(10.0,ws,dynamics_no,1/3*(n1+n2+n3),exp_ops=[n1,n2,n3,n4,n5,n6,n7,nc],b=B0(b_odmr,0.0,0.0))
    ws_doh_hf_codm,flur_doh_hf_codm,ts_doh_hf_codm,ns_doh_hf_codm,res_doh_hf_codm=\
        cwodmr(10.0,ws,dynamics_doh_hf,((n1+n2+n3)&IdN15)/6,
               exp_ops=[n1&IdN15,n2&IdN15,n3&IdN15,n4&IdN15,n5&IdN15,n6&IdN15,n7&IdN15,nc&IdN15],#type:ignore
               b=B0(b_odmr,0.0,0.0))
    ws_dua_hf_codm,flur_dua_hf_codm,ts_dua_hf_codm,ns_dua_hf_codm,res_dua_hf_codm=\
        cwodmr(10.0,ws,dynamics_dua_hf,((n1+n2+n3)&IdN15)/6,
               exp_ops=[n1&IdN15,n2&IdN15,n3&IdN15,n4&IdN15,n5&IdN15,n6&IdN15,n7&IdN15,nc&IdN15],#type:ignore
               b=B0(b_odmr,0.0,0.0))

In [ ]:
if not run_odmr:
    if b_odmr<500:
        n,m,s=2/5,25,10
        eps=n*b_odmr
    else:
        n,m,s=1/5,25,20
        eps=n*b_odmr
    flur_codm,ws_codm =(
        np.load(f'NumpyArrays\\flur_cwodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy'),
        np.load(f'NumpyArrays\\ws_cwodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy')
    )
    flur_doh_hf_codm,ws_doh_hf_codm =(
        np.load(f'NumpyArrays\\flur_doh_hf_cwodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy'),
        np.load(f'NumpyArrays\\ws_doh_hf_cwodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy')
    )
    flur_dua_hf_codm,ws_dua_hf_codm =(
        np.load(f'NumpyArrays\\flur_doh_hf_cwodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy'),
        np.load(f'NumpyArrays\\ws_doh_hf_cwodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy')
    )
    flur_podm,ws_podm =(
        np.load(f'NumpyArrays\\flur_puodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy'),
        np.load(f'NumpyArrays\\ws_puodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy')
    )
    flur_doh_hf_podm,ws_doh_hf_podm =(
        np.load(f'NumpyArrays\\flur_doh_hf_puodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy'),
        np.load(f'NumpyArrays\\ws_doh_hf_puodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy')
    )
    flur_dua_hf_podm,ws_dua_hf_podm =(
        np.load(f'NumpyArrays\\flur_doh_hf_puodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy'),
        np.load(f'NumpyArrays\\ws_doh_hf_puodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy')
    )
    if load_res:
        res_codm=np.load(f'NumpyArrays\\res_cwodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy',  allow_pickle=True)
        res_doh_hf_codm=np.load(f'NumpyArrays\\res_doh_hf_cwodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int (100*w_p_odmr)}.npy',allow_pickle=True)
        res_dua_hf_codm=np.load(f'NumpyArrays\\res_doh_hf_cwodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int (100*w_p_odmr)}.npy',allow_pickle=True)
        res_podm=np.load(f'NumpyArrays\\res_puodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy',  allow_pickle=True)
        res_doh_hf_podm=np.load (f'NumpyArrays\\res_doh_hf_puodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int    (100*w_p_odmr)}.npy',allow_pickle=True)
        res_dua_hf_podm=np.load (f'NumpyArrays\\res_doh_hf_puodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int    (100*w_p_odmr)}.npy',allow_pickle=True)

In [ ]:
norm_flur=max_reg(flur_codm)
norm_flur_doh_hf=max_reg(flur_doh_hf_codm)
norm_flur_dua_hf=max_reg(flur_dua_hf_codm)
plt.figure(figsize=(14, 8))
if len(flur_codm)>30 and not np.isclose(b_odmr,0):
    y,x1,x2=min(norm_flur),D_gs-mu_e*b_odmr,D_gs+mu_e*b_odmr
    plt.hlines(y,x1,x2,"r","--")
    plt.annotate(f"$2\\gamma_eB_0={2*mu_e*b_odmr:.2f}$ MHz",xy=(D_gs,y),xytext=(0,4),textcoords='offset points',
                            ha='center', va='bottom',color='red',fontsize=20)
plt.title(f"CW-ODMR\n B={b_odmr:.2f} G, $\\Omega_r$={om_r_odmr:.2f} MHz",fontsize=20)
plt.xlabel("Frequency (MHz)",fontsize=20)
plt.ylabel("Normalized PL (a.u.)",fontsize=20)
plt.plot(ws_codm,norm_flur,label="No-HF",color="orange")
plt.plot(ws_doh_hf_codm,norm_flur_doh_hf,label="HF,Doherty",ls="--")
plt.plot(ws_dua_hf_codm,norm_flur_dua_hf,label="HF,Duarte",ls="dotted",color="green")
plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left",fontsize=16)

plt.minorticks_on()
plt.show()

In [ ]:
if save_odmr:
    # Save no-HF CWODMR
    np.save(f'NumpyArrays\\flur_cwodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy',
            flur_codm)
    np.save(f'NumpyArrays\\ns_cwodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy',
            ns_codm)
    np.save(f'NumpyArrays\\ts_cwodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy',
            ts_codm)
    np.save(f'NumpyArrays\\ws_cwodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy',
            ws_codm)
    
    # Save Doherty CWODMR
    np.save(f'NumpyArrays\\flur_doh_hf_cwodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy',
            flur_doh_hf_codm)
    np.save(f'NumpyArrays\\ns_doh_hf_cwodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy',
            ns_doh_hf_codm)
    np.save(f'NumpyArrays\\ts_doh_hf_cwodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy',
            ts_doh_hf_codm)
    np.save(f'NumpyArrays\\ws_doh_hf_cwodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy',
            ws_doh_hf_codm)
    
    # Save Duarte CWODMR
    np.save(f'NumpyArrays\\flur_dua_hf_cwodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy',
            flur_dua_hf_codm)
    np.save(f'NumpyArrays\\ns_dua_hf_cwodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy',
            ns_dua_hf_codm)
    np.save(f'NumpyArrays\\ts_dua_hf_cwodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy',
            ts_dua_hf_codm)
    np.save(f'NumpyArrays\\ws_dua_hf_cwodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy',
            ws_dua_hf_codm)
    if save_res or b_odmr>=500:
        np.save(f'NumpyArrays\\res_doh_hf_cwodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy',
                res_doh_hf_codm)
        np.save(f'NumpyArrays\\res_cwodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy',
                res_codm)
        np.save(f'NumpyArrays\\res_dua_hf_cwodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy',
                res_dua_hf_codm)

#### Pulsed ODMR

In [ ]:
def puodmr_non_parallel(tp, tw, tr, ws, model, init_state, exp_ops, b=B, om_r=om_r_odmr, w_p=W_p):
    """
    Perform a series of simulations for the puodmr experiment.
    
    Args:
        tp (float): Laser pulse duration.
        tw (float): Free evolution time.
        tr (float): Laser pulse duration.
        ws (array-like): Array of angular frequencies.
        model (function): Function that performs the simulation.
        init_state (object): Initial state of the system.
        b (float, optional): Magnetic field strength. Defaults to B.
        om_r (float, optional): Rabi frequency. Defaults to Om_r.
        w_p (float, optional): Probe frequency. Defaults to W_p.
    
    Returns:
        tuple: A tuple containing the following elements:
            - ws (array-like): Array of angular frequencies.
            - flur (array-like): Array of PL values.
            - times (array-like): Array of time values for the plots.
            - n_exps (array-like): Array of expectation values for each operator.
            - res (array-like): Array of simulation results.
    """
    flur=np.array([])
    for w in tqdm(ws):
        results=np.array([])
        time=np.array([])
        ti=0.0
        #laser tp
        tf,result=model(tp, 
                        init_state,
                        b=b,
                        om_r=om_r,
                        om=w,
                        w_p=w_p, 
                        ti=ti, 
                        mode="Laser", 
                        progress_bar="OFF")
        results=np.append(results,result)
        init_state = result.states[-1]
        #free tw
        ti=tf
        tf,result=model(tw, 
                        init_state,
                        b=b,
                        om_r=om_r,
                        om=w,
                        w_p=w_p,
                        ti=ti, 
                        mode="Free", 
                        progress_bar="OFF")
        results=np.append(results,result)
        init_state = result.states[-1]
        #pi pulse
        ti=tf
        tpi=np.pi/om_r
        tf,result=model(tpi, 
                        init_state,
                        b=b,
                        om_r=om_r,
                        om=w,
                        w_p=w_p, 
                        ti=ti, 
                        mode="MW", 
                        progress_bar="OFF")
        results=np.append(results,result)
        init_state = result.states[-1]
        #laser tr
        ti=tf
        tf,result=model(tr, 
                        init_state,
                        b=b,
                        om_r=om_r,
                        om=w,
                        w_p=w_p_odmr, 
                        ti=ti, 
                        mode="Laser", 
                        progress_bar="OFF")
        results=np.append(results,result)
        n_exp = np.array([np.concatenate([qt.expect(M, res.states) for res in results]) for M in exp_ops])
        #for j in range(len(results[0].expect)):
        #    nn_exp = np.array([])
        #    for i in range(len(results)):
        #        nn_exp = np.append(nn_exp, results[i].expect[j])
        #    if j == 0:
        #        n_exp = np.array([nn_exp])
        #    else:
        #        n_exp = np.append(n_exp, [nn_exp], axis=0)
        fl=n_exp[3]+n_exp[4]+n_exp[5]
        flur=np.append(flur,np.sum(fl))
        # Gather the times for the plots
        for i in range(len(results)):
            time = np.append(time, results[i].times)
        if w == ws[0]:
            times=np.array([time])
            res=np.array([results])
            n_exps=np.array([n_exp])
        else:    
            times=np.append(times, [time], axis=0)
            res=np.append(res, [results], axis=0)
            n_exps=np.append(n_exps, [n_exp], axis=0)
    return ws,flur,times,n_exps,res

In [ ]:
def run_single_frequency_puodmr(w, tp, tw, tr, model, init_state, exp_ops, b, om_r, w_p):
    # Use a local copy of the initial state so that each frequency simulation is independent.
    current_state = init_state  # Consider current_state = init_state.copy() if needed
    results = np.array([])  # to collect simulation result objects
    ti = 0.0
    # 1. Laser pulse (duration tp, mode "Laser")
    tf, result = model(tp, current_state,
                         b=b, om_r=om_r, om=w, w_p=w_p/n_p_odmr,
                         ti=ti, mode="Laser", progress_bar="OFF")
    results = np.append(results, result)
    current_state = result.states[-1]
    # 2. Free evolution (duration tw, mode "Free")
    ti = tf
    tf, result = model(tw, current_state,
                         b=b, om_r=om_r, om=w, w_p=w_p,
                         ti=ti, mode="Free", progress_bar="OFF")
    results = np.append(results, result)
    current_state = result.states[-1]
    # 3. MW (pi pulse): duration pi/om_r, mode "MW"
    ti = tf
    tpi = np.pi/om_r
    tf, result = model(tpi, current_state,
                         b=b, om_r=om_r, om=w, w_p=w_p,
                         ti=ti, mode="MW", progress_bar="OFF")
    results = np.append(results, result)
    current_state = result.states[-1]
    # 4. Laser pulse (duration tr, mode "Laser") with different probe frequency
    ti = tf
    tf, result = model(tr, current_state,
                         b=b, om_r=om_r, om=w, w_p=w_p,  
                         ti=ti, mode="Laser", progress_bar="OFF")
    results = np.append(results, result)
    ## Build n_exp by concatenating the expectation values from each simulation.
    n_exp = np.array([np.concatenate([qt.expect(M, res.states) for res in results]) for M in exp_ops])
    #for j in range(len(results[0].expect)):
    #    nn_exp = np.array([])
    #    for i in range(len(results)):
    #        nn_exp = np.append(nn_exp, results[i].expect[j])
    #    if j == 0:
    #        n_exp = np.array([nn_exp])
    #    else:
    #        n_exp = np.append(n_exp, [nn_exp], axis=0)
    # Define PL as the sum of expectation values for indices 3, 4, and 5.
    fl = qt.expect(exp_ops[3]+exp_ops[4]+exp_ops[5], result.states)
    flur = np.sum(fl)
    # Gather times from each result.
    time_arr = np.concatenate([ress.times for ress in results])
    # Return a tuple: (w, flur, time_arr, n_exp, results)
    return (w, flur, time_arr, n_exp, results)

def puodmr(tp, tw, tr, ws, model, init_state,exp_ops, b=B, om_r=om_r_odmr, w_p=w_p_odmr):
    """
    Perform a series of simulations for the PUODMR experiment in parallel.
    
    Args:
        tp (float): Duration of the first (laser) pulse.
        tw (float): Duration of the free evolution.
        tr (float): Duration of the final (laser) pulse.
        ws (array-like): Array of angular frequencies.
        model (function): The simulation function.
        init_state: The initial state.
        b (float): Magnetic field.
        om_r (float): Rabi frequency.
        w_p (float): Probe frequency.
        
    Returns:
        tuple: (ws, flur, times, n_exps, res) where:
            - ws: Input frequencies.
            - flur: Array of PL values (one per frequency).
            - times: Array (or list) of time arrays.
            - n_exps: Array (or list) of expectation arrays.
            - res: Array (or list) of simulation results.
    """
    # Fix all parameters except frequency using partial.
    run_single = partial(run_single_frequency_puodmr,
                         tp=tp, tw=tw, tr=tr,
                         model=model, init_state=init_state,exp_ops=exp_ops,
                         b=b, om_r=om_r, w_p=w_p)
    # Use loky_pmap to run in parallel over ws.
    outs = qt.loky_pmap(run_single, ws, progress_bar="tqdm", map_kw={'fail_fast': True})
    
    # Now outs is a list of tuples: (w, flur, time_arr, n_exp, results) for each w.
    flur_all = np.array([item[1] for item in outs])         #type:ignore
    times_all = np.array([item[2] for item in outs])        #type:ignore
    n_exps_all = np.array([item[3] for item in outs])       #type:ignore
    res_all = np.array([item[4] for item in outs])          #type:ignore
    
    return ws, flur_all, times_all, n_exps_all, res_all

##### Run PuODMR

In [ ]:
if run_odmr:
    if b_odmr<500:
        n,m,s=2/5,25,10
        eps=n*b_odmr
    else:
        n,m,s=1/5,25,20     
        eps=n*b_odmr
    tI,tR=10,7
    wslhfl=np.linspace(D_gs-mu_e*b_odmr-mu_e*b_odmr*n,D_gs-mu_e*b_odmr-4*a_gs[0],2*s)
    wslhf=np.linspace(D_gs-mu_e*b_odmr-4*a_gs[0],D_gs-mu_e*b_odmr+4*a_gs[0],m)
    wslhfh=np.linspace(D_gs-mu_e*b_odmr+4*a_gs[0],D_gs-mu_e*b_odmr+mu_e*b_odmr*n,2*s)
    wsm=np.linspace(D_gs-mu_e*b_odmr+mu_e*b_odmr*n+eps,D_gs+mu_e*b_odmr-mu_e*b_odmr*n-eps,s)
    wshhfl=np.linspace(D_gs+mu_e*b_odmr-mu_e*b_odmr*n,D_gs+mu_e*b_odmr-4*a_gs[0],2*s)
    wshhf=np.linspace(D_gs+mu_e*b_odmr-4*a_gs[0],D_gs+mu_e*b_odmr+4*a_gs[0],m)
    wshhfh=np.linspace(D_gs+mu_e*b_odmr+4*a_gs[0],D_gs+mu_e*b_odmr+mu_e*b_odmr*n,2*s)
    ws=np.unique(np.hstack((wslhfl,wslhf,wslhfh,wsm,wshhfl,wshhf,wshhfh))) if b_odmr>10 else np.linspace(D_gs-100,D_gs+100,200)
    ws_podm,flur_podm,ts_podm,ns_podm,res_podm=puodmr(tI,1.0,tR,ws,dynamics_no,1/3*(n1+n2+n3),
                                                      [n1,n2,n3,n4,n5,n6,n7,nc],b=B0(b_odmr,0.0,0.0))
    ws_doh_hf_podm,flur_doh_hf_podm,ts_doh_hf_podm,ns_doh_hf_podm,res_doh_hf_podm=\
        puodmr(tI,1.0,tR,ws,dynamics_doh_hf,((n1+n2+n3)&IdN15)/6,
               [n1&IdN15,n2&IdN15,n3&IdN15,n4&IdN15,n5&IdN15,n6&IdN15,n7&IdN15,nc&IdN15],#type:ignore
               b=B0(b_odmr,0.0,0.0))
    ws_dua_hf_podm,flur_dua_hf_podm,ts_dua_hf_podm,ns_dua_hf_podm,res_dua_hf_podm=\
        puodmr(tI,1.0,tR,ws,dynamics_dua_hf,((n1+n2+n3)&IdN15)/6,
               [n1&IdN15,n2&IdN15,n3&IdN15,n4&IdN15,n5&IdN15,n6&IdN15,n7&IdN15,nc&IdN15],#type:ignore
               b=B0(b_odmr,0.0,0.0))

In [ ]:
norm_flur=max_reg(flur_podm)
norm_flur_doh_hf=max_reg(flur_doh_hf_podm)
norm_flur_dua_hf=max_reg(flur_dua_hf_podm)
plt.figure(figsize=(14, 8))
if len(flur_podm)>30 and not np.isclose(b_odmr,0):
    y,x1,x2=min(norm_flur),D_gs-mu_e*b_odmr,D_gs+mu_e*b_odmr
    plt.hlines(y,x1,x2,"r","--")
    plt.annotate(f"$2\\gamma_eB_0={2*mu_e*b_odmr:.2f}$ MHz",xy=(D_gs,y),xytext=(0,4),textcoords='offset points',
                            ha='center', va='bottom',color='r',fontsize=20)
plt.plot(ws_podm,norm_flur,label="No-HF",color="orange")
plt.title(f"PuODMR\n B={b_odmr:.2f} G, $\\Omega_r$={om_r_odmr:.2f} MHz",fontsize=20)
plt.xlabel("Frequency (MHz)",fontsize=20)
plt.ylabel("Normalized PL (a.u.)",fontsize=20)
plt.plot(ws_doh_hf_podm,norm_flur_doh_hf,label="HF,Doherty",ls="--")
plt.plot(ws_dua_hf_podm,norm_flur_dua_hf,label="HF,Duarte",ls="dotted",color="green")
plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left",fontsize=16)
plt.minorticks_on()

plt.show()

In [ ]:
if save_odmr:
    # Save no-HF PuODMR
    np.save(f'NumpyArrays\\flur_puodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy',
            flur_podm)
    np.save(f'NumpyArrays\\ns_puodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy',
            ns_podm)
    np.save(f'NumpyArrays\\ts_puodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy',
            ts_podm)
    np.save(f'NumpyArrays\\ws_puodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy',
            ws_podm)
    
    # Save Doherty PuODMR
    np.save(f'NumpyArrays\\flur_doh_hf_puodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy',
            flur_doh_hf_podm)
    np.save(f'NumpyArrays\\ns_doh_hf_puodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy',
            ns_doh_hf_podm)
    np.save(f'NumpyArrays\\ts_doh_hf_puodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy',
            ts_doh_hf_podm)
    np.save(f'NumpyArrays\\ws_doh_hf_puodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy',
            ws_doh_hf_podm)
    
    # Save Duarte PuODMR
    np.save(f'NumpyArrays\\flur_dua_hf_puodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy',
            flur_dua_hf_podm)
    np.save(f'NumpyArrays\\ns_dua_hf_puodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy',
            ns_dua_hf_podm)
    np.save(f'NumpyArrays\\ts_dua_hf_puodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy',
            ts_dua_hf_podm)
    np.save(f'NumpyArrays\\ws_dua_hf_puodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy',
            ws_dua_hf_podm)
    if save_res or b_odmr>500:
        np.save(f'NumpyArrays\\res_puodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy',
                res_podm)
        np.save(f'NumpyArrays\\res_doh_hf_puodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy',
                res_doh_hf_podm)
        np.save(f'NumpyArrays\\res_dua_hf_puodmr_B_{int(b_odmr)}_om_{int(100*om_r_odmr)}_W_{int(100*w_p_odmr)}.npy',
                res_dua_hf_podm)

In [ ]:
plt.figure(figsize=(14, 8))
plt.title(f"B={b_odmr:.2f} G, $\\Omega_r$={om_r_odmr:.2f} MHz",fontsize=20)
plt.xlabel("Frequency (MHz)",fontsize=20)
plt.ylabel("Normalized PL (a.u.)",fontsize=20)
if not np.isclose(b_odmr,0):
    #y,x1,x2=min(max_reg(flur_codm)),D_gs-mu_e*b_odmr,D_gs+mu_e*b_odmr
    plt.hlines(y,x1,x2,"r","--")
    plt.annotate(f"$2\\gamma_eB_0={2*mu_e*b_odmr:.2f}$ MHz",xy=(D_gs,y),xytext=(0,4),textcoords='offset points',
                            ha='center', va='bottom',color='r',fontsize=20)
plt.plot(ws_codm,max_reg(flur_codm),label="no-HF,CW-ODMR")
plt.plot(ws_podm,max_reg(flur_podm),label="no-HF,PuODMR")
plt.plot(ws_doh_hf_codm,max_reg(flur_doh_hf_codm),label="HF,CW-ODMR\nDoherty",ls="--")
plt.plot(ws_doh_hf_podm,max_reg(flur_doh_hf_podm),label="HF,PuODMR\nDoherty",ls="--")
plt.plot(ws_dua_hf_codm,max_reg(flur_dua_hf_codm),label="HF,CW-ODMR\nDuarte",ls="dotted")
plt.plot(ws_dua_hf_podm,max_reg(flur_dua_hf_podm),label="HF,PuODMR\nDuarte",ls="dotted")
plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left",fontsize=16)
plt.minorticks_on()

plt.show()

#### Ramsey

In [ ]:
def ramsey(ts, tp, tw, tr, model, init_state,exp_ops,
           b=B, om_r=Om_r, w_p=W_p,om=omega):
    """
    Perform Ramsey experiment simulation.
    
    Parameters:
    - tp (float): Laser pulse duration for preparation.
    - tw (float): Free evolution time between the laser pulse and first pi/2 pulse.
    - ts (float): Free evolution time between the first and second pi/2 pulses.
    - tr (float): Laser pulse duration for readout.
    - model (function): Function that defines the simulation model.
    - init_state (object): Initial state of the system.
    - b (array): Parameter b.
    - om_r (float): Rabi frequency.
    - w_p (float): Parameter w_p.
    
    Returns:
    - ws (array): Array of frequencies.
    - flur (array): Array of PL values.
    - times (array): Array of times for the plots.
    - n_exps (array): Array of expectation values.
    - res (array): Array of simulation results.
    """
    flur=np.array([])
    #for t in tqdm(ts):
    results=np.array([])
    time=np.array([])
    ti=0.0
    #laser tp
    tf,result=model(tp, 
                    init_state,
                    b=b,
                    om_r=om_r,
                    om=om,
                    w_p=w_p, 
                    ti=ti, 
                    mode="Laser", 
                    progress_bar="OFF")
    results=np.append(results,result)
    init_state = result.states[-1]
    #free tw
    ti=tf
    tf,result=model(tw, 
                    init_state,
                    b=b,
                    om_r=om_r,
                    om=om,
                    w_p=w_p,
                    ti=ti, 
                    mode="Free", 
                    progress_bar="OFF")
    results=np.append(results,result)
    init_state = result.states[-1]
    #pi/2 pulse
    ti=tf
    tpi2=0.5*np.pi/om_r
    tf,result=model(tpi2, 
                    init_state,
                    b=b,
                    om_r=om_r,
                    om=om,
                    w_p=w_p,
                    ti=ti, 
                    mode="MW", 
                    progress_bar="OFF")
    results=np.append(results,result)
    init_state = result.states[-1]
    #free ts
    ti=tf
    tf,result=model(ts, 
                    init_state,
                    b=b,
                    om_r=om_r,
                    om=om,
                    w_p=w_p,
                    ti=ti, 
                    mode="Free", 
                    progress_bar="OFF")
    results=np.append(results,result)
    init_state = result.states[-1]
    #pi/2 pulse
    ti=tf
    tpi2=0.5*np.pi/om_r
    tf,result=model(tpi2, 
                    init_state,
                    b=b,
                    om_r=om_r,
                    om=om,
                    w_p=w_p,
                    ti=ti, 
                    mode="MW", 
                    progress_bar="OFF")
    results=np.append(results,result)
    init_state = result.states[-1]
    #laser tr
    ti=tf
    tf,result=model(tr, 
                    init_state,
                    b=b,
                    om_r=om_r,
                    om=om,
                    w_p=w_p_ramsey,
                    ti=ti, 
                    mode="Laser", 
                    progress_bar="OFF")
    results=np.append(results,result)
    n_exp = np.array([np.concatenate([qt.expect(M, res.states) for res in results]) for M in exp_ops])
    fl=qt.expect(exp_ops[3]+exp_ops[4]+exp_ops[5], result.states) 
    flur=np.append(flur,np.sum(fl))
    # Gather the times for the plots
    time = np.concatenate([res.times for res in results])

    return ts,flur,time,n_exp,results

In [ ]:
def ramsey_parallel(tp, tw, ts, tr, model, init_state, exp_ops, b, om_r, w_p, om):
    """
    Run the Ramsey simulation in parallel over the free evolution time values in ts.
    
    Returns a tuple:
       (ts_out, flur_out, times_out, n_exps_out, res_out)
    where each element is an array (or list) with one entry per t.
    """
    # Create a partial function that fixes the parameters except t.
    run_ramsey = partial(ramsey, tp=tp, tw=tw, tr=tr,
                          model=model, init_state=init_state,exp_ops=exp_ops,
                          b=b, om_r=om_r, w_p=w_p, om=om)
    # Use loky_pmap to run in parallel over ts.
    outs = qt.loky_pmap(run_ramsey, ts, progress_bar="tqdm", map_kw={'fail_fast': True})
    
    # outs is a list of tuples: (t, flur, times, n_exp, results) for each t.
    ts_out = np.array([item[0] for item in outs])       #type:ignore
    flur_out = np.array([item[1] for item in outs])     #type:ignore
    times_out = np.array([item[2] for item in outs])    #type:ignore
    n_exps_out = np.array([item[3] for item in outs])   #type:ignore
    res_out = np.array([item[4] for item in outs])      #type:ignore
    
    return ts_out, flur_out, times_out, n_exps_out, res_out

In [ ]:
om_r_pi=np.pi/t2_gs
init_t=5
tf=10
oms_r=np.array([om_r_pi,2*om_r_pi,10*om_r_pi])
bs=np.array([0,100,510,1020])
for b in tqdm(bs,desc='Magnectic Field',position=0):
    b0=B0(b,0.0,0.0)
    for omr in tqdm(oms_r,desc='Rabi Frequency',position=1):
        if run_ramsey:
            tI,tR=10.0,6.0
            tf=10.0
            prop=5/200
            init_t=5.0
            ts=np.linspace(init_t+prop,tf,int((tf-init_t)/prop))
            ts_ramsey,flur_ramsey,times_ramsey,ns_ramsey,res_ramsey=\
                ramsey_parallel(tI,1.0,ts,tR,dynamics_no,1/3*(n1+n2+n3),
                                [n1,n2,n3,n4,n5,n6,n7,nc],b0,omr,W_p,D_gs-mu_e*b)
            ts_doh_hf_ramsey,flur_doh_hf_ramsey,times_doh_hf_ramsey,ns_doh_hf_ramsey,res_doh_hf_ramsey=\
                ramsey_parallel(tI,1.0,ts,tR,dynamics_doh_hf,((n1+n2+n3)&IdN15)/6,
                                [n1&IdN15,n2&IdN15,n3&IdN15,n4&IdN15,n5&IdN15,n6&IdN15,n7&IdN15,nc&IdN15],#type:ignore
                                b0,omr,W_p,D_gs-mu_e*b)
            ts_dua_hf_ramsey,flur_dua_hf_ramsey,times_dua_hf_ramsey,ns_dua_hf_ramsey,res_dua_hf_ramsey=\
                ramsey_parallel(tI,1.0,ts,tR,dynamics_dua_hf,((n1+n2+n3)&IdN15)/6,
                                [n1&IdN15,n2&IdN15,n3&IdN15,n4&IdN15,n5&IdN15,n6&IdN15,n7&IdN15,nc&IdN15],#type:ignore
                                b0,omr,W_p,D_gs-mu_e*b)
            # Save no_HF Ramsey
            np.save(f'NumpyArrays\\flur_ramsey_B_{int(b)}_om_{int(100*omr)}_W_{int(100*w_p_ramsey)}_ti_{int(init_t)}_tf_{int(tf)}.npy',
                    flur_ramsey)
            np.save(f'NumpyArrays\\times_ramsey_B_{int(b)}_om_{int(100*omr)}_W_{int(100*w_p_ramsey)}_ti_{int(init_t)}_tf_{int(tf)}.npy',
                    times_ramsey)
            np.save(f'NumpyArrays\\ns_ramsey_B_{int(b)}_om_{int(100*omr)}_W_{int(100*w_p_ramsey)}_ti_{int(init_t)}_tf_{int(tf)}.npy',
                    ns_ramsey)
            np.save(f'NumpyArrays\\ts_ramsey_B_{int(b)}_om_{int(100*omr)}_W_{int(100*w_p_ramsey)}_ti_{int(init_t)}_tf_{int(tf)}.npy',
                    ts_ramsey)
            
            # Save Doherty ramsey
            np.save(f'NumpyArrays\\flur_doh_hf_ramsey_B_{int(b)}_om_{int(100*omr)}_W_{int(100*w_p_ramsey)}_ti_{int(init_t)}_tf_{int(tf)}.npy',
                    flur_doh_hf_ramsey)
            np.save(f'NumpyArrays\\times_doh_hf_ramsey_B_{int(b)}_om_{int(100*omr)}_W_{int(100*w_p_ramsey)}_ti_{int(init_t)}_tf_{int(tf)}.npy',
                    times_doh_hf_ramsey)
            np.save(f'NumpyArrays\\ns_doh_hf_ramsey_B_{int(b)}_om_{int(100*omr)}_W_{int(100*w_p_ramsey)}_ti_{int(init_t)}_tf_{int(tf)}.npy',
                    ns_doh_hf_ramsey)
            np.save(f'NumpyArrays\\ts_doh_hf_ramsey_B_{int(b)}_om_{int(100*omr)}_W_{int(100*w_p_ramsey)}_ti_{int(init_t)}_tf_{int(tf)}.npy',
                    ts_doh_hf_ramsey)

            # Save Duarte Ramsey
            np.save(f'NumpyArrays\\flur_dua_hf_ramsey_B_{int(b)}_om_{int(100*omr)}_W_{int(100*w_p_ramsey)}_ti_{int(init_t)}_tf_{int(tf)}.npy',
                    flur_dua_hf_ramsey)
            np.save(f'NumpyArrays\\times_dua_hf_ramsey_B_{int(b)}_om_{int(100*omr)}_W_{int(100*w_p_ramsey)}_ti_{int(init_t)}_tf_{int(tf)}.npy',
                    times_dua_hf_ramsey)
            np.save(f'NumpyArrays\\ns_dua_hf_ramsey_B_{int(b)}_om_{int(100*omr)}_W_{int(100*w_p_ramsey)}_ti_{int(init_t)}_tf_{int(tf)}.npy',
                    ns_dua_hf_ramsey)
            np.save(f'NumpyArrays\\ts_dua_hf_ramsey_B_{int(b)}_om_{int(100*omr)}_W_{int(100*w_p_ramsey)}_ti_{int(init_t)}_tf_{int(tf)}.npy',
                    ts_dua_hf_ramsey)
            if save_res or (b>500 and omr==10*om_r_pi):
                np.save(f'NumpyArrays\\res_ramsey_B_{int(b)}_om_{int(100*omr)}_W_{int(100*w_p_ramsey)}_ti_{int(init_t)}_tf_{int(tf)}.npy',
                        res_ramsey)
                np.save(f'NumpyArrays\\res_doh_hf_ramsey_B_{int(b)}_om_{int(100*omr)}_W_{int(100*w_p_ramsey)}_ti_{int(init_t)}_tf_{int(tf)}.npy',
                        res_doh_hf_ramsey)
                np.save(f'NumpyArrays\\res_dua_hf_ramsey_B_{int(b)}_om_{int(100*omr)}_W_{int(100*w_p_ramsey)}_ti_{int(init_t)}_tf_{int(tf)}.npy',
                        res_dua_hf_ramsey)
        else:
            # Load no_HF Ramsey
            flur_ramsey,times_ramsey,ns_ramsey,ts_ramsey=(
                np.load(f'NumpyArrays\\flur_ramsey_B_{int(b)}_om_{int(100*omr)}_W_{int(100*w_p_ramsey)}_ti_{int(init_t)}_tf_{int(tf)}.npy'),
                np.load(f'NumpyArrays\\times_ramsey_B_{int(b)}_om_{int(100*omr)}_W_{int(100*w_p_ramsey)}_ti_{int(init_t)}_tf_{int(tf)}.npy'),
                np.load(f'NumpyArrays\\ns_ramsey_B_{int(b)}_om_{int(100*omr)}_W_{int(100*w_p_ramsey)}_ti_{int(init_t)}_tf_{int(tf)}.npy'),
                np.load(f'NumpyArrays\\ts_ramsey_B_{int(b)}_om_{int(100*omr)}_W_{int(100*w_p_ramsey)}_ti_{int(init_t)}_tf_{int(tf)}.npy'),
                )
            # Load Doherty ramsey
            flur_doh_hf_ramsey,times_doh_hf_ramsey,ns_doh_hf_ramsey,ts_doh_hf_ramsey=(
                np.load(f'NumpyArrays\\flur_doh_hf_ramsey_B_{int(b)}_om_{int(100*omr)}_W_{int(100*w_p_ramsey)}_ti_{int(init_t)}_tf_{int(tf)}.npy'),
                np.load(f'NumpyArrays\\times_doh_hf_ramsey_B_{int(b)}_om_{int(100*omr)}_W_{int(100*w_p_ramsey)}_ti_{int(init_t)}_tf_{int(tf)}.npy'),
                np.load(f'NumpyArrays\\ns_doh_hf_ramsey_B_{int(b)}_om_{int(100*omr)}_W_{int(100*w_p_ramsey)}_ti_{int(init_t)}_tf_{int(tf)}.npy'),
                np.load(f'NumpyArrays\\ts_doh_hf_ramsey_B_{int(b)}_om_{int(100*omr)}_W_{int(100*w_p_ramsey)}_ti_{int(init_t)}_tf_{int(tf)}.npy'),
                )
            # Load Duarte Ramsey
            flur_dua_hf_ramsey,times_dua_hf_ramsey,ns_dua_hf_ramsey,ts_dua_hf_ramsey=(                
                np.load(f'NumpyArrays\\flur_dua_hf_ramsey_B_{int(b)}_om_{int(100*omr)}_W_{int(100*w_p_ramsey)}_ti_{int(init_t)}_tf_{int(tf)}.npy'),
                np.load(f'NumpyArrays\\times_dua_hf_ramsey_B_{int(b)}_om_{int(100*omr)}_W_{int(100*w_p_ramsey)}_ti_{int(init_t)}_tf_{int(tf)}.npy'),
                np.load(f'NumpyArrays\\ns_dua_hf_ramsey_B_{int(b)}_om_{int(100*omr)}_W_{int(100*w_p_ramsey)}_ti_{int(init_t)}_tf_{int(tf)}.npy'),
                np.load(f'NumpyArrays\\ts_dua_hf_ramsey_B_{int(b)}_om_{int(100*omr)}_W_{int(100*w_p_ramsey)}_ti_{int(init_t)}_tf_{int(tf)}.npy'),
                )
            if load_res:
                res_ramsey=np.load(f'NumpyArrays\\res_ramsey_B_{int(b)}_om_{int(100*omr)}_W_{int(100*w_p_ramsey)}_ti_{int(init_t)}_tf_{int(tf)}.npy', allow_pickle=True)
                res_doh_hf_ramsey=np.load(f'NumpyArrays\\res_doh_hf_ramsey_B_{int(b)}_om_{int(100*omr)}_W_{int  (100*w_p_ramsey)}_ti_{int(init_t)}_tf_{int(tf)}.npy',allow_pickle=True)
                res_dua_hf_ramsey=np.load(f'NumpyArrays\\res_dua_hf_ramsey_B_{int(b)}_om_{int(100*omr)}_W_{int  (100*w_p_ramsey)}_ti_{int(init_t)}_tf_{int(tf)}.npy',allow_pickle=True)
        plt.figure(figsize=(14, 8))
        plt.title(f"Ramsey\n B={b:.2f} G, $\\Omega_r$={omr:.2f} MHz",fontsize=20)
        plt.plot(ts_ramsey,max_reg(flur_ramsey),label='no-HF')
        plt.plot(ts_dua_hf_ramsey,max_reg(flur_dua_hf_ramsey),ls="dotted",label="HF Duarte")
        plt.plot(ts_doh_hf_ramsey,max_reg(flur_doh_hf_ramsey),ls="--",label="HF Doherty")
        plt.minorticks_on()
        
        plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left",fontsize=16)
        plt.show()

#### Spin echo

In [ ]:
def spin_echo(tp, tw, ts1, ts2, tr, model, init_state, exp_ops,
              b=B, om_r=Om_r, w_p=W_p):
    """
    Perform a spin echo experiment.
    
    Parameters:
    - tp (float): Duration of the laser pulse.
    - tw (float): Duration of the free evolution period.
    - ts1 (float): Duration of the first free evolution period.
    - ts2 (float): Duration of the second free evolution period.
    - tr (float): Duration of the laser pulse.
    - model (function): Function that models the system dynamics.
    - init_state (object): Initial state of the system.
    - b (float): Magnetic field strength (default: B).
    - om_r (float): Resonance frequency (default: Om_r).
    - w_p (float): Rabi frequency (default: W_p).
    
    Returns:
    - ws (array): Array of frequencies.
    - flur (array): Array of PL values.
    - times (array): Array of time values.
    - n_exps (array): Array of expectation values.
    - res (array): Array of simulation results.
    """
    flur=np.array([])
    for w in tqdm(ws):
        results=np.array([])
        time=np.array([])
        ti=0.0
        #laser tp
        tf,result=model(tp, 
                        init_state,
                        b=b,
                        om_r=om_r,
                        om=w,
                        w_p=w_p,
                        ti=ti, 
                        mode="Laser", 
                        progress_bar="OFF")
        results=np.append(results,result)
        init_state = result.states[-1]
        #free tw
        ti=tf
        tf,result=model(tw, 
                        init_state,
                        b=b,
                        om_r=om_r,
                        om=w,
                        w_p=w_p,
                        ti=ti, 
                        mode="Free", 
                        progress_bar="OFF")
        results=np.append(results,result)
        init_state = result.states[-1]
        #pi/2 pulse
        ti=tf
        tpi2=0.5*np.pi/om_r
        tf,result=model(tpi2, 
                        init_state,
                        b=b,
                        om_r=om_r,
                        om=w,
                        w_p=w_p, 
                        ti=ti, 
                        mode="MW", 
                        progress_bar="OFF")
        results=np.append(results,result)
        init_state = result.states[-1]
        #free ts1
        ti=tf
        tf,result=model(ts1, 
                        init_state,
                        b=b,
                        om_r=om_r,
                        om=w,
                        w_p=w_p,
                        ti=ti, 
                        mode="Free", 
                        progress_bar="OFF")
        results=np.append(results,result)
        init_state = result.states[-1]
        #pi pulse
        ti=tf
        tpi=np.pi/om_r
        tf,result=model(tpi, 
                        init_state,
                        b=b,
                        om_r=om_r,
                        om=w,
                        w_p=w_p,
                        ti=ti, 
                        mode="MW", 
                        progress_bar="OFF")
        results=np.append(results,result)
        init_state = result.states[-1]
        #free ts2
        ti=tf
        tf,result=model(ts2, 
                        init_state,
                        b=b,
                        om_r=om_r,
                        om=w,
                        w_p=w_p, 
                        ti=ti, 
                        mode="Free", 
                        progress_bar="OFF")
        results=np.append(results,result)
        init_state = result.states[-1]
        #pi/2 pulse
        ti=tf
        tpi2=0.5*np.pi/om_r
        tf,result=model(tpi2, 
                        init_state,
                        b=b,
                        om_r=om_r,
                        om=w,
                        w_p=w_p,
                        ti=ti, 
                        mode="MW", 
                        progress_bar="OFF")
        results=np.append(results,result)
        init_state = result.states[-1]
        #laser tr
        ti=tf
        tf,result=model(tr, 
                        init_state,
                        b=b,
                        om_r=om_r,
                        om=w,
                        w_p=w_p, 
                        ti=ti, 
                        mode="Laser", 
                        progress_bar="OFF")
        results=np.append(results,result)
        n_exp = np.array([np.concatenate([qt.expect(M, res.states) for res in results]) for M in exp_ops])
        #for j in range(len(results[0].expect)):
        #    nn_exp = np.array([])
        #    for i in range(len(results)):
        #        nn_exp = np.append(nn_exp, results[i].expect[j])
        #    if j == 0:
        #        n_exp = np.array([nn_exp])
        #    else:
        #        n_exp = np.append(n_exp, [nn_exp], axis=0)
        fl=n_exp[3]+n_exp[4]+n_exp[5]
        flur=np.append(flur,np.sum(fl))
        # Gather the times for the plots
        for i in range(len(results)):
            time = np.append(time, results[i].times)
        if w == ws[0]:
            times=np.array([time])
            res=np.array([results])
            n_exps=np.array([n_exp])
        else:    
            times=np.append(times, [time], axis=0)
            res=np.append(res, [results], axis=0)
            n_exps=np.append(n_exps, [n_exp], axis=0)
    return ws,flur,times,n_exps,res

## Magnectic orientation vs PL

In [ ]:
final_b=1300
delt_b=5
ang_degs  = [0, 0.5, 1, 4, 8, 10]
if run_b_v_pl:
    for ang_deg in tqdm(ang_degs,desc="Angle",position=0):
        final_states = np.array([])
        final_states_hf = np.array([])
        #results = np.array([])
        #results_hf = np.array([])
        ang_rad = np.deg2rad(ang_deg)
        Bs=range(0,final_b,delt_b)
        for b_pl in tqdm(Bs,desc='Magnectic Field',position=1):
            Om_r = 15.7
            W_p = 2
            bs=B0(b_pl,0.0,ang_rad)
            omega = D_gs-mu_e*bs[2]
            init_state= 1/3*(n1+n2+n3)
            init_state_hf = ((n1+n2+n3)&IdN15)/6
            ti=0
            _,result = dynamics_no(10, init_state, b=bs, om_r=Om_r, om=omega, w_p=W_p, ti=0.0, mode="Laser", progress_bar="OFF")
            _,result_hf= dynamics_doh_hf(10, init_state_hf, b=bs, om_r=Om_r, om=omega, w_p=W_p, ti=0.0, mode="Laser",   progress_bar="OFF")
            #results=np.append(results,result)
            #results_hf.append(result_hf)
            tf = result.times[-1]
            init_state= result.states[-1]
            init_state_hf = result_hf.states[-1]
            _,result = dynamics_no(1, init_state, b=bs, om_r=Om_r, om=omega, w_p=W_p, ti=tf, mode="Free", progress_bar="OFF")
            _,result_hf = dynamics_doh_hf(1, init_state_hf, b=bs, om_r=Om_r, om=omega, w_p=W_p, ti=tf, mode="Free",     progress_bar="OFF")
            #results.append(result)
            #results_hf.append(result_hf)
            tf = result.times[-1]
            init_state = result.states[-1] 
            init_state_hf = result_hf.states[-1]
            _,result = dynamics_no(10, init_state, b=bs, om_r=Om_r, om=omega, w_p=W_p, ti=tf, mode="Laser", progress_bar="OFF")
            _,result_hf = dynamics_doh_hf(10, init_state_hf, b=bs, om_r=Om_r, om=omega, w_p=W_p, ti=tf, mode="Laser",   progress_bar="OFF")
            #results.append(result)
            #results_hf.append(result_hf)
            final_states = np.append(final_states, result.states[-1]) #type:ignore
            final_states_hf = np.append(final_states_hf, result_hf.states[-1]) #type:ignore
                
        plt.figure(figsize=(14, 8))
        plt.plot(Bs, [qt.expect(n4+n5+n6, state) for state in final_states], label="No HF")
        #plt.plot(range(0,final_b), [qt.expect(n4+n5+n6, state) for state in final_states], label="Hirose No HF")
        plt.plot(Bs, [qt.expect(((n4+n5+n6)&IdN15), state) for state in final_states_hf], label="HF")
        plt.title(f"Final States for ang_deg={ang_deg} and b_pl={b_pl}", fontsize=20)
        plt.xlabel("B (G)", fontsize=20)
        plt.ylabel("PL (a.u.)", fontsize=20)
        plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left",fontsize=16)
        plt.minorticks_on()
        plt.show()
        np.save(f'NumpyArrays\\final_states_PL_v_B_{int(final_b)}_ang_deg_{int(10*ang_deg)}.npy', final_states)
        np.save(f'NumpyArrays\\final_states_hf_PL_v_B_{int(final_b)}_ang_deg_{int(10*ang_deg)}.npy', final_states_hf)
        #if save_res:
        #    # ------------------------------------------------------------
        #    # Estimate compressed size of one element in MB
        #    # ------------------------------------------------------------
        #    def estimate_avg_size_mb(sample_list, compress_level):
        #        import io
        #        total = 0
        #        for item in sample_list:
        #            f = io.BytesIO()
        #            joblib.dump([item], f, compress=('xz', compress_level), protocol=5) #type:ignore
        #            total += len(f.getvalue())
        #        return (total / len(sample_list)) / 1024 / 1024  # MB
    #
    #
    #
        #    # ------------------------------------------------------------
        #    # Set chunk size target (in MB) and compute estimated chunk size
        #    # ------------------------------------------------------------
        #    target_file_size_mb = 200
        #    compress_lvl = 5
    #
        #    # Sample 5–10 items (adjust if list is small)
        #    sample_size = min(10, len(results))
        #    sample_items = results[:sample_size]
        #    item_size_mb = estimate_avg_size_mb(sample_items, compress_lvl)
        #    chunk_size = max(1, int(target_file_size_mb // item_size_mb))
        #    for i in range(0, len(results), chunk_size):
        #        batch = results[i : i + chunk_size]
        #        joblib.dump(
        #            batch,
        #            f'NumpyArrays/results_PL_v_B_{int(final_b)}_ang_deg_{int(10*ang_deg)}_part{i//chunk_size}.xz',
        #            compress=5,
        #            protocol=5
        #        )
        #    # Sample 5–10 items (adjust if list is small)
        #    sample_size = min(10, len(results_hf))
        #    sample_items = results_hf[:sample_size]
        #    item_size_mb = estimate_avg_size_mb(sample_items, compress_lvl)
        #    chunk_size = max(1, int(target_file_size_mb // item_size_mb))
        #    for i in range(0, len(results_hf), chunk_size):
        #        batch_hf = results_hf[i : i + chunk_size]
        #        joblib.dump(
        #            batch_hf,
        #            f'NumpyArrays/results_hf_PL_v_B_{int(final_b)}_ang_deg_{int(10*ang_deg)}_part{i//chunk_size}.xz',
        #            compress=5,
        #            protocol=5
        #        )
else:
    #import glob
    #import re
    # -------------------------------------------------------------------
    # Helper to load and concatenate chunked joblib files in order
    # -------------------------------------------------------------------
    #def load_joblib_chunks(pattern):
    #    files = glob.glob(pattern)
    #    files_sorted = sorted(
    #        files,
    #        key=lambda fn: int(re.search(r'_part(\d+)\.xz$', fn).group(1))
    #    )
    #    data = []
    #    for fn in files_sorted:
    #        data.extend(joblib.load(fn))
    #    return data

    

    # -------------------------------------------------------------------
    # 2) Container for all loaded data
    # -------------------------------------------------------------------
    all_data = {}

    # -------------------------------------------------------------------
    # 3) Iterate over angles and reload files
    # -------------------------------------------------------------------
    for ang_deg in ang_degs:
        # match the save naming convention
        adeg = int(10 * ang_deg)
        key  = f"{ang_deg}"
        all_data[key] = {}

        # --- 3.1 Load final_states arrays ---
        all_data[key]['final_states']    = np.load(
            f'NumpyArrays/final_states_PL_v_B_{final_b}_ang_deg_{adeg}.npy',
            allow_pickle=True
        )
        all_data[key]['final_states_hf'] = np.load(
            f'NumpyArrays/final_states_hf_PL_v_B_{final_b}_ang_deg_{adeg}.npy',
            allow_pickle=True
        )
        #if load_res:
        #    # --- 3.2 Load chunked results (parallel and HF) ---
        #    all_data[key]['results']    = load_joblib_chunks(
        #        f'NumpyArrays/results_PL_v_B_{final_b}_ang_deg_{adeg}_part*.xz'
        #    )
        #    all_data[key]['results_hf'] = load_joblib_chunks(
        #        f'NumpyArrays/results_hf_PL_v_B_{final_b}_ang_deg_{adeg}_part*.xz'
        #    )
    # assume all_data from your loader, and final_b defined
    angles = [0, 0.5, 1, 4, 8, 10]

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(25, 8))

    # No‑HF panel
    for ang in angles:
        key = f"{ang}"
        states = all_data[key]['final_states']
        y = [qt.expect(n4 + n5 + n6, state) for state in states]
        ax1.plot(range(0,final_b,delt_b), y, label=rf"{ang}$^\circ$")
    ax1.set_title("No‑HF regime",fontsize=20)
    ax1.set_xlabel("B (G)",fontsize=20)
    ax1.set_ylabel("PL (a.u.)",fontsize=20)
    #ax1.legend(title="angle", bbox_to_anchor=(1.05, 1), loc="upper left", fontsize=16)

    # HF panel
    for ang in angles:
        key = f"{ang}"
        states_hf = all_data[key]['final_states_hf']
        yhf = [qt.expect((n4 + n5 + n6) & IdN15, state) for state in states_hf]
        ax2.plot(range(0,final_b,delt_b), yhf, label=rf"{ang}$^\circ$")
    ax2.set_title("HF regime",fontsize=20)
    ax2.set_xlabel("B (G)",fontsize=20)
    ax2.legend( bbox_to_anchor=(1.05, 1), loc="upper left", fontsize=20)

    fig.suptitle("Photoluminescence (PL) vs Magnetic Field (B) \nfor each angle in No‑HF (left) and HF (right)",fontsize=20)
    #fig.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.show()


## Run time comparison

In [ ]:
import multiprocessing as mp 
def perform(init_state,dynamics,timings_only=False):
    #Initial state where the ground state is evenly populated
    #init_state = ((n1+n2+n3)&IdN15)/6
    # decide whether to show a progress bar for this worker
    show_bar = (not timings_only) and (mp.current_process().name == "MainProcess")
    pb_flag  = "ON" if show_bar else "OFF"
    # Array to collect the result Class
    results_hf = np.array([])

    # Initial time of the evolution
    ti = 0.0

    # Store the times of activation and deativation of laser and microwaves for region highlight in plotting
    tis_hf = []
    tfs_hf = []

    # Set the number of times to repeat the Sequence
    n = 1
    # Set the time intervals for each part of the evolution (with laser, with microwaves and free evolution)
    dt_laser = 10.0
    dt_free = 1.0
    dt_mw = 5.0
    #print("Starting calculations, this may take a while...")
    # This loop is defined to reproduce the Sequence of Magalleti el al
    for i in tqdm(range(n), disable=(pb_flag == "OFF")):
        tis_hf.append(ti)
        # Here Laser is ON
        # Run the dynamics based on the mode choosen
        tf, result = dynamics(
            dt_laser,
            init_state,
            ti=ti,
            mode="Laser",
            progress_bar=pb_flag,
            i=i,
        ) # type:ignore
        # Store the Result class for when the Laser is on
        results_hf = np.append(results_hf, result) # type: ignore 
        # Save the initial state for the next part of the Sequence
        init_state = result.states[-1]
        # Make the initial time the end time of the previous part of the procotol
        ti = tf
        tfs_hf.append(tf)
        # Run the dynamics based on the mode choosen
        tf, result = dynamics(
            dt_free,
            init_state,
            ti=ti,
            mode="Free",
            progress_bar=pb_flag,
            i=i,
        ) # type:ignore
        # Store the Result class for MW on evolution
        results_hf = np.append(results_hf, result) # type: ignore 
        # Save the times for coloring areas of the plot
        tis_hf.append(ti)
        tfs_hf.append(tf)
        # Make initial time to be the end time of the last part of the Sequence
        ti = tf
        # Make the last state  of this evolution step be the initial states of the next iteration
        init_state = result.states[-1]
        # Run the dynamics based on the mode choosen
        tf, result = dynamics(dt_mw, 
                              init_state, 
                              ti=ti, 
                              mode="MW", 
                              progress_bar=pb_flag, 
                              i=i) # type:ignore
        # Store the Result class for MW on evolution
        results_hf = np.append(results_hf, result) # type: ignore
        # Save the times for coloring areas of the plot
        tis_hf.append(ti)
        tfs_hf.append(tf)
        # Make initial time to be the end time of the last part of the Sequence
        ti = tf
        # Make the last state  of this evolution step be the initial states of the next iteration
        init_state = result.states[-1]
    #print("Calculations are finished!! Go to the next block for plots")
    if timings_only:
        return

In [ ]:
import time, statistics, joblib

# -------------------------------------------------------------------
# 0) Filename for saving/loading summary
# -------------------------------------------------------------------
runs     = 8
summary_file = f'NumpyArrays/benchmark_summary_{runs}_runs.joblib'
run_benchmark = False  # Set to False to load pre-computed results
if run_benchmark:
    # -------------------------------------------------------------------
    # 1) Helper that does ONE timed evolution and returns only the runtime
    # -------------------------------------------------------------------
    def _timed_perform(args):
        init_state, dynamics = args
        t0 = time.perf_counter()
        perform(init_state, dynamics, timings_only=True)
        return time.perf_counter() - t0

    # -------------------------------------------------------------------
    # 2) Helpers that measure TOTAL time for a list of runs
    # -------------------------------------------------------------------
    def total_time_seq(args_list):
        t0 = time.perf_counter()
        for args in args_list:
            _timed_perform(args)
        return time.perf_counter() - t0

    def total_time_par(args_list, map_opts):
        t0 = time.perf_counter()
        qt.loky_pmap(_timed_perform, args_list, map_kw=map_opts, progress_bar="tqdm")
        return time.perf_counter() - t0

    # -------------------------------------------------------------------
    # 3) Build the (runs)× argument lists for each dynamics variant
    # -------------------------------------------------------------------
    mag_args = [((n1 + n2 + n3) / 3,           dynamics_no    )] * runs
    doh_args = [(((n1 + n2 + n3) & IdN15) / 6, dynamics_doh_hf)] * runs
    dua_args = [(((n1 + n2 + n3) & IdN15) / 6, dynamics_dua_hf)] * runs

    # -------------------------------------------------------------------
    # 4) Total‑time benchmarks (repeat to get statistics)
    # -------------------------------------------------------------------
    repeats  = 5
    map_opts = {"num_cpus": 8}

    seq_totals_mag = [total_time_seq(mag_args) for _ in range(repeats)]
    par_totals_mag = [total_time_par(mag_args, map_opts) for _ in range(repeats)]

    seq_totals_doh = [total_time_seq(doh_args) for _ in range(repeats)]
    par_totals_doh = [total_time_par(doh_args, map_opts) for _ in range(repeats)]

    seq_totals_dua = [total_time_seq(dua_args) for _ in range(repeats)]
    par_totals_dua = [total_time_par(dua_args, map_opts) for _ in range(repeats)]

    # -------------------------------------------------------------------
    # 5) Numerical summary: means and standard deviations
    # -------------------------------------------------------------------
    summary = {}
    for label, seq_list, par_list in [
        ("no-hf", seq_totals_mag, par_totals_mag),
        ("doh-hf", seq_totals_doh, par_totals_doh),
        ("dua-hf", seq_totals_dua, par_totals_dua)
    ]:
        summary[label] = {
            "seq_mean": statistics.mean(seq_list),
            "seq_std":  statistics.stdev(seq_list),
            "par_mean": statistics.mean(par_list),
            "par_std":  statistics.stdev(par_list)
        }

    # -------------------------------------------------------------------
    # 6) Save summary to disk
    # -------------------------------------------------------------------
    joblib.dump(summary, summary_file)

else:
    # -------------------------------------------------------------------
    # Load pre‑computed summary
    # -------------------------------------------------------------------
    summary = joblib.load(summary_file)

# -------------------------------------------------------------------
# 7) Print results
# -------------------------------------------------------------------
print(f"\nTotal time over {runs} runs (seconds):")
for lbl, stats in summary.items():
    print(f"{lbl:<6} seq = {stats['seq_mean']:.4g} ± {stats['seq_std']:.4g}   "
          f"par = {stats['par_mean']:.4g} ± {stats['par_std']:.4g}")

# -------------------------------------------------------------------
# 8) Bar plot with error bars
# -------------------------------------------------------------------
labels    = ["No‑HF", "Model 3", "Model 2"]
means_seq = [summary[l]["seq_mean"] for l in ["no-hf","doh-hf","dua-hf"]]
errs_seq  = [summary[l]["seq_std"]  for l in ["no-hf","doh-hf","dua-hf"]]
means_par = [summary[l]["par_mean"] for l in ["no-hf","doh-hf","dua-hf"]]
errs_par  = [summary[l]["par_std"]  for l in ["no-hf","doh-hf","dua-hf"]]

x     = np.arange(len(labels))
width = 0.35

fig, ax = plt.subplots(figsize=(8,5))
ax.bar(x - width/2, means_seq, width, yerr=errs_seq, capsize=4,
       label="Sequential", alpha=0.7)
ax.bar(x + width/2, means_par, width, yerr=errs_par, capsize=4,
       label="Parallel", hatch="//", edgecolor="black")

ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.set_ylabel(f"Total time for {runs} runs (s)",fontsize=16)
ax.set_title(f"Sequential vs Parallel \nAverage (5 rep. of {runs} runs) total execution time",fontsize=16)
ax.legend()
ax.grid(axis="y", linestyle=":", linewidth=0.5)

plt.tight_layout()
plt.show()

# End